### Import Dependencies

In [56]:
import pandas as pd
import numpy as np
import os

### Set Formulas

In [57]:
def clean_yearlies(df, base_team=''):
    
    # Add a 'base team' column
    df['Base_Team'] = base_team
    
    # Unify the data so all null rows are set to zero
    df['PF'].fillna(0, inplace = True)
    df['PA'].fillna(0, inplace = True)
    df['OPassY'].fillna(0, inplace = True)
    df['ORushY'].fillna(0, inplace = True)
    df['TO_lost'].fillna(0, inplace = True)
    df['DPassY'].fillna(0, inplace = True)
    df['DRushY'].fillna(0, inplace = True)
    df['TO_won'].fillna(0, inplace = True)
    df['OTotYd'].fillna(0, inplace = True)
    df['DTotYd'].fillna(0, inplace = True)

    # Caluculate the cumulative sum of each stat by week, EXCLUDING that week's numbers. Should reflect the
    # team's states going into the game, not coming out of it.
    df['PF Tally'] = df['PF'].cumsum() - df['PF']
    df['PA Tally'] = df['PA'].cumsum() - df['PA']
    df['OPassY Tally'] = df['OPassY'].cumsum() - df['OPassY']
    df['ORushY Tally'] = df['ORushY'].cumsum() - df['ORushY']
    df['TO_lost Tally'] = df['TO_lost'].cumsum() - df['TO_lost']
    df['TO_won Tally'] = df['TO_won'].cumsum() - df['TO_won']
    df['DPassY Tally'] = df['DPassY'].cumsum() - df['DPassY']
    df['DRushY Tally'] = df['DRushY'].cumsum() - df['DRushY']
    df['OTotYd Tally'] = df['OTotYd'].cumsum() - df['OTotYd']
    df['DTotYd Tally'] = df['DTotYd'].cumsum() - df['DTotYd']
    
    # Clean 'Results' column so wins have a value of 1 and losses/ties have a value of 0
    df['Results'] = df['Results'].replace(['L', 'T'], 0)
    df['Results'] = df['Results'].replace('W', 1)

    # Caluculate win tallies going into each game
    df = df.rename(columns={"Entering Wins": "Wins Tally"})
    df['Wins Tally'] = 0
    df['Wins Tally'] = df['Results'].cumsum() - df['Results']
    
    # Clean 'week_after_bye' column so only the 'Bye Week' row and the row after it have a value of 1.
    # 'Bye Week' row will be dropped later.
    df['week_after_bye'] = 0
    df.loc[df['Opp'] == 'Bye Week', 'week_after_bye'] = 1
    df['after_bye'] = df['week_after_bye'].cumsum()
    df['week_after_bye'].values[df['week_after_bye'] >= 2] = 0
    df['week_after_bye'] = df['after_bye'].cumsum() - df['after_bye']
    df['after_bye'].values[df['week_after_bye'] >= 2] = 0
    
    df = df.drop(columns=['week_after_bye', 'Rec', 'Date', 'PF', 'PA', 'OPassY', 'ORushY', 'TO_lost', 'TO_won', 'DPassY', 'DRushY', 'OTotYd', 'DTotYd'])
    
    # Delete 'Bye' weeks and change column name
    df = df[df['Opp'] != 'Bye Week']

    # Clean 'Home' column so home games have a value of 1 and away games have a value of 0
    df['Home'] = df['Home'].replace(['@', 'N'], 0)
    df['Home'].fillna(1, inplace = True)
    
    return df

In [58]:
def yearly_data(df, year, HC2_range=0, QB2_range=0, HC='', HC2='none', OC='', DC='', QB='', QB2='none'):
    df['year'] = year
    df = clean_yearlies(df)
    df['Head_Coach'] = HC
    if HC2 == 'none':
        pass
    else:
        df.loc[df['Week'] == HC2_range, 'Head_Coach'] = HC2
    df['OC'] = OC
    df['DC'] = DC
    df['QB'] = QB
    if QB2 == 'none':
        pass
    else:
        df.loc[df['Week'] == QB2_range, 'QB'] = QB2
        
    df = df[df['Week'] <= 17]
        
    return df

QB data from: https://www.espn.com/nfl/team/stats/_/name/
End that URL with the team abbreviation

### San Francisco 49ers Data, 2010-2019

In [59]:
# Data Source: https://www.pro-football-reference.com/teams/sfo/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

sf_df2010 = pd.read_csv('Resources/Niners_Raw/sf_sportsref_download2010.csv')
sf_df2011 = pd.read_csv('Resources/Niners_Raw/sf_sportsref_download2011.csv')
sf_df2012 = pd.read_csv('Resources/Niners_Raw/sf_sportsref_download2012.csv')
sf_df2013 = pd.read_csv('Resources/Niners_Raw/sf_sportsref_download2013.csv')
sf_df2014 = pd.read_csv('Resources/Niners_Raw/sf_sportsref_download2014.csv')
sf_df2015 = pd.read_csv('Resources/Niners_Raw/sf_sportsref_download2015.csv')
sf_df2016 = pd.read_csv('Resources/Niners_Raw/sf_sportsref_download2016.csv')
sf_df2017 = pd.read_csv('Resources/Niners_Raw/sf_sportsref_download2017.csv')
sf_df2018 = pd.read_csv('Resources/Niners_Raw/sf_sportsref_download2018.csv')
sf_df2019 = pd.read_csv('Resources/Niners_Raw/sf_sportsref_download2019.csv')
niners_2020 = pd.read_csv('Resources/2020data/niners_sportsref_download2020.csv')

sf_df2010 = yearly_data(df=sf_df2010, year=2010, 
                        HC='Mike Singletary', HC2_range=17, HC2='Jim Tomsula', 
                        OC='Jimmy Raye', 
                        DC='Greg Manusky', 
                        QB='Alex Smith', 
                        QB2='David Carr', QB2_range=7)
sf_df2010.loc[sf_df2010['Week'] == range(8,13,1), 'QB'] = "Troy Smith"

sf_df2011 = yearly_data(df=sf_df2011, year=2011, 
                        HC='Jim Harbaugh', 
                        OC='Greg Roman', 
                        DC='Vic Fangio', 
                        QB='Alex Smith')
sf_df2012 = yearly_data(df=sf_df2012, year=2012, 
                        HC='Jim Harbaugh', 
                        OC='Greg Roman', 
                        DC='Vic Fangio',
                        QB='Alex Smith', 
                        QB2='Colin Kaepernick', QB2_range=range(10,18,1))
sf_df2013 = yearly_data(df=sf_df2013, year=2013, 
                        HC='Jim Harbaugh', 
                        OC='Greg Roman', 
                        DC='Vic Fangio', 
                        QB='Colin Kaepernick')
sf_df2014 = yearly_data(df=sf_df2014, year=2014, 
                        HC='Jim Harbaugh', 
                        OC='Greg Roman', 
                        DC='Vic Fangio', 
                        QB='Colin Kaepernick')
sf_df2015 = yearly_data(df=sf_df2015, year=2015, 
                        HC='Jim Tomsula', 
                        OC='Geep Chryst', 
                        DC='Eric Mangini', 
                        QB='Colin Kaepernick',
                        QB2='Blaine Gabbert', QB2_range=range(10,18,1))
sf_df2016 = yearly_data(df=sf_df2016, year=2016, 
                        HC='Chip Kelly', 
                        OC='Curtis Modkins', 
                        DC="Jim O'Neil", 
                        QB='Blaine Gabbert',
                        QB2='Colin Kaepernick', QB2_range=(6,18,1))
sf_df2017 = yearly_data(df=sf_df2017, year=2017, 
                        HC='Kyle Shanahan', 
                        OC='Kyle Shanahan', 
                        DC='Robert Saleh', 
                        QB='Brian Hoyer',
                        QB2='C.J. Beathard', QB2_range=range(7,13,1))
sf_df2017.loc[sf_df2017['Week'] == range(12,18,1), 'QB'] = "Jimmy Garoppolo"

sf_df2018 = yearly_data(df=sf_df2018, year=2018, 
                        HC='Kyle Shanahan', 
                        OC='Kyle Shanahan', 
                        DC='Robert Saleh', 
                        QB='Jimmy Garoppolo',
                        QB2='C.J. Beathard', QB2_range=range(3,9,1))
sf_df2018.loc[sf_df2018['Week'] == range(8,18,1), 'QB'] = "Nick Mullens"

sf_df2019 = yearly_data(df=sf_df2019, year=2019, 
                        HC='Kyle Shanahan', 
                        OC='Kyle Shanahan', 
                        DC='Robert Saleh', 
                        QB='Jimmy Garoppolo')

niners_2020 = yearly_data(df=niners_2020, year=2020, 
                        HC='Kyle Shanahan', 
                        OC='Kyle Shanahan', 
                        DC='Robert Saleh', 
                        QB='Jimmy Garoppolo', 
                        QB2='Nick Mullens', QB2_range=range(3,5,1))

sf_df = niners_2020.append(sf_df2019, ignore_index=True)
sf_df = sf_df.append(sf_df2018, ignore_index=True)
sf_df = sf_df.append(sf_df2017, ignore_index=True)
sf_df = sf_df.append(sf_df2016, ignore_index=True)
sf_df = sf_df.append(sf_df2015, ignore_index=True)
sf_df = sf_df.append(sf_df2014, ignore_index=True)
sf_df = sf_df.append(sf_df2013, ignore_index=True)
sf_df = sf_df.append(sf_df2012, ignore_index=True)
sf_df = sf_df.append(sf_df2011, ignore_index=True)
sf_df = sf_df.append(sf_df2010, ignore_index=True)

sf_df['Base_Team'] = '49ers'
sf_df = sf_df.sort_values(by=['year', 'Week'], ascending=[False, True])

sf_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Sun,0.0,0.0,1.0,Arizona Cardinals,2020,49ers,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Kyle Shanahan,Kyle Shanahan,Robert Saleh,Jimmy Garoppolo
1,2,Sun,1.0,0.0,0.0,New York Jets,2020,49ers,20.0,24.0,...,1.0,224.0,180.0,366.0,404.0,0,Kyle Shanahan,Kyle Shanahan,Robert Saleh,Jimmy Garoppolo
2,3,Sun,1.0,1.0,0.0,New York Giants,2020,49ers,51.0,37.0,...,1.0,397.0,284.0,725.0,681.0,0,Kyle Shanahan,Kyle Shanahan,Robert Saleh,Jimmy Garoppolo
3,4,Sun,0.0,2.0,1.0,Philadelphia Eagles,2020,49ers,87.0,46.0,...,4.0,562.0,350.0,1145.0,912.0,0,Kyle Shanahan,Kyle Shanahan,Robert Saleh,Jimmy Garoppolo
4,5,Sun,NaN,NaN,1.0,Miami Dolphins,2020,49ers,107.0,71.0,...,5.0,736.0,443.0,1562.0,1179.0,0,Kyle Shanahan,Kyle Shanahan,Robert Saleh,Jimmy Garoppolo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,13,Sun,0.0,4.0,0.0,Green Bay Packers,2010,49ers,187.0,225.0,...,14.0,2404.0,1081.0,3523.0,3485.0,0,Mike Singletary,Jimmy Raye,Greg Manusky,Alex Smith
172,14,Sun,1.0,4.0,1.0,Seattle Seahawks,2010,49ers,203.0,259.0,...,14.0,2678.0,1217.0,3792.0,3895.0,0,Mike Singletary,Jimmy Raye,Greg Manusky,Alex Smith
173,15,Thu,0.0,5.0,0.0,San Diego Chargers,2010,49ers,243.0,280.0,...,19.0,2955.0,1301.0,4128.0,4256.0,0,Mike Singletary,Jimmy Raye,Greg Manusky,Alex Smith
174,16,Sun,0.0,5.0,0.0,St. Louis Rams,2010,49ers,250.0,314.0,...,19.0,3221.0,1409.0,4320.0,4630.0,0,Mike Singletary,Jimmy Raye,Greg Manusky,Alex Smith


In [60]:
niners_df = sf_df.copy()
niners_df.to_csv('Resources/indv_team_csvs/niners.csv')

### Rams Data, 2010-2019

In [61]:
# Data Source: https://www.pro-football-reference.com/teams/ram/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

rams_df2010 = pd.read_csv('Resources/Rams_Raw/rams_sportsref_download2010.csv')
rams_df2011 = pd.read_csv('Resources/Rams_Raw/rams_sportsref_download2011.csv')
rams_df2012 = pd.read_csv('Resources/Rams_Raw/rams_sportsref_download2012.csv')
rams_df2013 = pd.read_csv('Resources/Rams_Raw/rams_sportsref_download2013.csv')
rams_df2014 = pd.read_csv('Resources/Rams_Raw/rams_sportsref_download2014.csv')
rams_df2015 = pd.read_csv('Resources/Rams_Raw/rams_sportsref_download2015.csv')
rams_df2016 = pd.read_csv('Resources/Rams_Raw/rams_sportsref_download2016.csv')
rams_df2017 = pd.read_csv('Resources/Rams_Raw/rams_sportsref_download2017.csv')
rams_df2018 = pd.read_csv('Resources/Rams_Raw/rams_sportsref_download2018.csv')
rams_df2019 = pd.read_csv('Resources/Rams_Raw/rams_sportsref_download2019.csv')
rams_2020 = pd.read_csv('Resources/2020data/rams_sportsref_download2020.csv')

rams_df2010 = yearly_data(df=rams_df2010, year=2010, 
                        HC='Steve Spagnuolo', 
                        OC='Pat Shurmur', 
                        DC='Ken Flajole', 
                        QB='Sam Bradford')

rams_df2011 = yearly_data(df=rams_df2011, year=2011, 
                          HC='Steve Spagnuolo', 
                          OC='Josh McDaniels', 
                          DC='Ken Flajole', 
                          QB='Sam Bradford',
                          QB2='A.J. Feeley', QB2_range=range(7,9,1))
rams_df2011.loc[rams_df2011['Week'] == 12, 'QB'] = "Brandon Lloyd"
rams_df2011.loc[rams_df2011['Week'] == 13, 'QB'] = "A.J. Feeley"
rams_df2011.loc[rams_df2011['Week'] == range(15,18,1), 'QB'] = "Kellen Clemens"

rams_df2012 = yearly_data(df=rams_df2012, year=2012, 
                        HC='Jeff Fisher', 
                        OC='Brian Schottenheimer', 
                        DC='Jeff Fisher',
                        QB='Sam Bradford')

rams_df2013 = yearly_data(df=rams_df2013, year=2013, 
                        HC='Jeff Fisher', 
                        OC='Brian Schottenheimer', 
                        DC='Tim Walton', 
                        QB='Sam Bradford')
rams_df2013.loc[rams_df2013['Week'] == range(8,18,1), 'QB'] = "Kellen Clemens"

rams_df2014 = yearly_data(df=rams_df2014, year=2014, 
                        HC='Jeff Fisher', 
                        OC='Brian Schottenheimer', 
                        DC='Gregg Williams', 
                        QB='Shaun Hill',
                        QB2='Austin Davis', QB2_range=(2,11,1))

rams_df2015 = yearly_data(df=rams_df2015, year=2015, 
                        HC='Jeff Fisher', 
                        OC='Rob Boras', 
                        DC='Gregg Williams', 
                        QB='Nick Foles',
                        QB2='Case Keenum', QB2_range=11)
rams_df2015.loc[rams_df2015['Week'] == range(14,18,1), 'QB'] = "Case Keenum"

rams_df2016 = yearly_data(df=rams_df2016, year=2016, 
                        HC='Jeff Fisher', HC2='John Fassel', HC2_range=range(14,18,1),
                        OC='Rob Boras', 
                        DC="Gregg Williams", 
                        QB='Case Keenum',
                        QB2='Jared Goff', QB2_range=range(11,18,1))

rams_df2017 = yearly_data(df=rams_df2017, year=2017, 
                        HC='Sean McVay', 
                        OC='Matt LaFleur', 
                        DC='Wade Phillips', 
                        QB='Jared Goff',
                        QB2='Sean Mannion', QB2_range=17)

rams_df2018 = yearly_data(df=rams_df2018, year=2018, 
                        HC='Sean McVay', 
                        OC='Sean McVay', 
                        DC='Wade Phillips', 
                        QB='Jared Goff')

rams_df2019 = yearly_data(df=rams_df2019, year=2019, 
                        HC='Sean McVay', 
                        OC='Sean McVay', 
                        DC='Wade Phillips', 
                        QB='Jared Goff')

rams_2020 = yearly_data(df=rams_2020, year=2020, 
                        HC='Sean McVay', 
                        OC="Kevin O'Connell", 
                        DC='Brandon Staley', 
                        QB='Jared Goff')

# Make master dataframe for team
rams_df = rams_2020.append(rams_df2019, ignore_index=True)
rams_df = rams_df.append(rams_df2018, ignore_index=True)
rams_df = rams_df.append(rams_df2017, ignore_index=True)
rams_df = rams_df.append(rams_df2016, ignore_index=True)
rams_df = rams_df.append(rams_df2015, ignore_index=True)
rams_df = rams_df.append(rams_df2014, ignore_index=True)
rams_df = rams_df.append(rams_df2013, ignore_index=True)
rams_df = rams_df.append(rams_df2012, ignore_index=True)
rams_df = rams_df.append(rams_df2011, ignore_index=True)
rams_df = rams_df.append(rams_df2010, ignore_index=True)

rams_df['Base_Team'] = 'Rams'
rams_df = rams_df.sort_values(by=['year', 'Week'], ascending=[False, True])

rams_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Sun,1.0,0.0,1.0,Dallas Cowboys,2020,Rams,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Sean McVay,Kevin O'Connell,Brandon Staley,Jared Goff
1,2,Sun,1.0,1.0,0.0,Philadelphia Eagles,2020,Rams,20.0,17.0,...,0.0,244.0,136.0,422.0,380.0,0,Sean McVay,Kevin O'Connell,Brandon Staley,Jared Goff
2,3,Sun,0.0,2.0,0.0,Buffalo Bills,2020,Rams,57.0,36.0,...,3.0,486.0,257.0,871.0,743.0,0,Sean McVay,Kevin O'Connell,Brandon Staley,Jared Goff
3,4,Sun,1.0,2.0,1.0,New York Giants,2020,Rams,89.0,71.0,...,5.0,760.0,358.0,1349.0,1118.0,0,Sean McVay,Kevin O'Connell,Brandon Staley,Jared Goff
4,5,Sun,NaN,NaN,0.0,Washington Football Team,2020,Rams,106.0,80.0,...,6.0,919.0,494.0,1589.0,1413.0,0,Sean McVay,Kevin O'Connell,Brandon Staley,Jared Goff
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,13,Sun,1.0,5.0,0.0,Arizona Cardinals,2010,Rams,213.0,231.0,...,17.0,2635.0,1137.0,3488.0,3772.0,0,Steve Spagnuolo,Pat Shurmur,Ken Flajole,Sam Bradford
172,14,Sun,0.0,6.0,0.0,New Orleans Saints,2010,Rams,232.0,237.0,...,19.0,2754.0,1242.0,3776.0,3996.0,0,Steve Spagnuolo,Pat Shurmur,Ken Flajole,Sam Bradford
173,15,Sun,0.0,6.0,1.0,Kansas City Chiefs,2010,Rams,245.0,268.0,...,22.0,2967.0,1374.0,4103.0,4341.0,0,Steve Spagnuolo,Pat Shurmur,Ken Flajole,Sam Bradford
174,16,Sun,1.0,6.0,1.0,San Francisco 49ers,2010,Rams,258.0,295.0,...,23.0,3140.0,1584.0,4327.0,4724.0,0,Steve Spagnuolo,Pat Shurmur,Ken Flajole,Sam Bradford


In [62]:
rams_df.to_csv('Resources/indv_team_csvs/rams.csv')

### Chargers Data, 2010-2019

In [63]:
# Data Source: https://www.pro-football-reference.com/teams/sdg/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

chargers_df2010 = pd.read_csv('Resources/Chargers_Raw/chargers_sportsref_download2010.csv')
chargers_df2011 = pd.read_csv('Resources/Chargers_Raw/chargers_sportsref_download2011.csv')
chargers_df2012 = pd.read_csv('Resources/Chargers_Raw/chargers_sportsref_download2012.csv')
chargers_df2013 = pd.read_csv('Resources/Chargers_Raw/chargers_sportsref_download2013.csv')
chargers_df2014 = pd.read_csv('Resources/Chargers_Raw/chargers_sportsref_download2014.csv')
chargers_df2015 = pd.read_csv('Resources/Chargers_Raw/chargers_sportsref_download2015.csv')
chargers_df2016 = pd.read_csv('Resources/Chargers_Raw/chargers_sportsref_download2016.csv')
chargers_df2017 = pd.read_csv('Resources/Chargers_Raw/chargers_sportsref_download2017.csv')
chargers_df2018 = pd.read_csv('Resources/Chargers_Raw/chargers_sportsref_download2018.csv')
chargers_df2019 = pd.read_csv('Resources/Chargers_Raw/chargers_sportsref_download2019.csv')
chargers_2020 = pd.read_csv('Resources/2020data/chargers_sportsref_download2020.csv')

chargers_df2010 = yearly_data(df=chargers_df2010, year=2010, 
                        HC='Norv Turner', 
                        OC='Clarence Shelmon', 
                        DC='Ron Rivera', 
                        QB='Philip Rivers')

chargers_df2011 = yearly_data(df=chargers_df2011, year=2011, 
                        HC='Norv Turner', 
                        OC='Clarence Shelmon', 
                        DC='Greg Manusky', 
                        QB='Philip Rivers')

chargers_df2012 = yearly_data(df=chargers_df2012, year=2012, 
                        HC='Norv Turner', 
                        OC='Hal Hunter', 
                        DC='John Pagano',
                        QB='Philip Rivers')

chargers_df2013 = yearly_data(df=chargers_df2013, year=2013, 
                        HC='Mike McCoy', 
                        OC='Ken Whisenhunt', 
                        DC='John Pagano', 
                        QB='Philip Rivers')

chargers_df2014 = yearly_data(df=chargers_df2014, year=2014, 
                        HC='Mike McCoy', 
                        OC='Frank Reich', 
                        DC='John Pagano', 
                        QB='Philip Rivers')

chargers_df2015 = yearly_data(df=chargers_df2015, year=2015, 
                        HC='Mike McCoy', 
                        OC='Frank Reich', 
                        DC='John Pagano', 
                        QB='Philip Rivers')

chargers_df2016 = yearly_data(df=chargers_df2016, year=2016, 
                        HC='Mike McCoy', 
                        OC='Ken Whisenhunt', 
                        DC='John Pagano', 
                        QB='Philip Rivers')

chargers_df2017 = yearly_data(df=chargers_df2017, year=2017, 
                        HC='Anthony Lynn', 
                        OC='Ken Whisenhunt', 
                        DC='Gus Bradley', 
                        QB='Philip Rivers')

chargers_df2018 = yearly_data(df=chargers_df2018, year=2018, 
                        HC='Anthony Lynn', 
                        OC='Ken Whisenhunt', 
                        DC='Gus Bradley', 
                        QB='Philip Rivers')

chargers_df2019 = yearly_data(df=chargers_df2019, year=2019, 
                        HC='Anthony Lynn', 
                        OC='Ken Whisenhunt', 
                        DC='Gus Bradley', 
                        QB='Philip Rivers')

chargers_2020 = yearly_data(df=chargers_2020, year=2020, 
                        HC='Anthony Lynn', 
                        OC='Shane Steichen', 
                        DC='Gus Bradley', 
                        QB='Tyrod Taylor', 
                        QB2='Justin Herbert', QB2_range=range(2,5,1))

# Make master dataframe for team
chargers_df = chargers_2020.append(chargers_df2019, ignore_index=True)
chargers_df = chargers_df.append(chargers_df2018, ignore_index=True)
chargers_df = chargers_df.append(chargers_df2017, ignore_index=True)
chargers_df = chargers_df.append(chargers_df2016, ignore_index=True)
chargers_df = chargers_df.append(chargers_df2015, ignore_index=True)
chargers_df = chargers_df.append(chargers_df2014, ignore_index=True)
chargers_df = chargers_df.append(chargers_df2013, ignore_index=True)
chargers_df = chargers_df.append(chargers_df2012, ignore_index=True)
chargers_df = chargers_df.append(chargers_df2011, ignore_index=True)
chargers_df = chargers_df.append(chargers_df2010, ignore_index=True)

chargers_df['Base_Team'] = 'Chargers'
chargers_df = chargers_df.sort_values(by=['year', 'Week'], ascending=[False, True])

chargers_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Sun,1.0,0.0,0.0,Cincinnati Bengals,2020,Chargers,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Anthony Lynn,Shane Steichen,Gus Bradley,Tyrod Taylor
1,2,Sun,0.0,1.0,1.0,Kansas City Chiefs,2020,Chargers,16.0,13.0,...,2.0,173.0,122.0,362.0,295.0,0,Anthony Lynn,Shane Steichen,Gus Bradley,Tyrod Taylor
2,3,Sun,0.0,1.0,1.0,Carolina Panthers,2020,Chargers,36.0,36.0,...,2.0,462.0,247.0,841.0,709.0,0,Anthony Lynn,Shane Steichen,Gus Bradley,Tyrod Taylor
3,4,Sun,0.0,1.0,0.0,Tampa Bay Buccaneers,2020,Chargers,52.0,57.0,...,2.0,683.0,328.0,1277.0,1011.0,0,Anthony Lynn,Shane Steichen,Gus Bradley,Tyrod Taylor
4,5,Mon,NaN,NaN,0.0,New Orleans Saints,2020,Chargers,83.0,95.0,...,3.0,1052.0,443.0,1601.0,1495.0,0,Anthony Lynn,Shane Steichen,Gus Bradley,Tyrod Taylor
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,13,Sun,0.0,6.0,1.0,Oakland Raiders,2010,Chargers,310.0,225.0,...,18.0,2118.0,892.0,4483.0,3010.0,0,Norv Turner,Clarence Shelmon,Ron Rivera,Philip Rivers
172,14,Sun,1.0,6.0,1.0,Kansas City Chiefs,2010,Chargers,323.0,253.0,...,19.0,2235.0,1143.0,4769.0,3378.0,0,Norv Turner,Clarence Shelmon,Ron Rivera,Philip Rivers
173,15,Thu,1.0,7.0,1.0,San Francisco 49ers,2010,Chargers,354.0,253.0,...,19.0,2254.0,1191.0,5195.0,3445.0,0,Norv Turner,Clarence Shelmon,Ron Rivera,Philip Rivers
174,16,Sun,0.0,8.0,0.0,Cincinnati Bengals,2010,Chargers,388.0,260.0,...,20.0,2385.0,1252.0,5569.0,3637.0,0,Norv Turner,Clarence Shelmon,Ron Rivera,Philip Rivers


In [64]:
chargers_df.to_csv('Resources/indv_team_csvs/chargers.csv')

### Seahawks Data, 2010-2019

In [65]:
# Data Source: https://www.pro-football-reference.com/teams/sea/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

seahawks_df2010 = pd.read_csv('Resources/Seahawks_Raw/seahawks_sportsref_download2010.csv')
seahawks_df2011 = pd.read_csv('Resources/Seahawks_Raw/seahawks_sportsref_download2011.csv')
seahawks_df2012 = pd.read_csv('Resources/Seahawks_Raw/seahawks_sportsref_download2012.csv')
seahawks_df2013 = pd.read_csv('Resources/Seahawks_Raw/seahawks_sportsref_download2013.csv')
seahawks_df2014 = pd.read_csv('Resources/Seahawks_Raw/seahawks_sportsref_download2014.csv')
seahawks_df2015 = pd.read_csv('Resources/Seahawks_Raw/seahawks_sportsref_download2015.csv')
seahawks_df2016 = pd.read_csv('Resources/Seahawks_Raw/seahawks_sportsref_download2016.csv')
seahawks_df2017 = pd.read_csv('Resources/Seahawks_Raw/seahawks_sportsref_download2017.csv')
seahawks_df2018 = pd.read_csv('Resources/Seahawks_Raw/seahawks_sportsref_download2018.csv')
seahawks_df2019 = pd.read_csv('Resources/Seahawks_Raw/seahawks_sportsref_download2019.csv')
seahawks_2020 = pd.read_csv('Resources/2020data/seahawks_sportsref_download2020.csv')

seahawks_df2010 = yearly_data(df=seahawks_df2010, year=2010, 
                        HC='Pete Carroll', 
                        OC='Jeremy Bates', 
                        DC='Gus Bradley', 
                        QB='Matt Hasselbeck', 
                        QB2='Charlie Whitehurst', QB2_range=range(9,18,8))

seahawks_df2011 = yearly_data(df=seahawks_df2011, year=2011, 
                        HC='Pete Carroll', 
                        OC='Darrell Bevell', 
                        DC='Gus Bradley', 
                        QB='Tarvaris Jackson', 
                        QB2='Charlie Whitehurst', QB2_range=range(7,9,1))

seahawks_df2012 = yearly_data(df=seahawks_df2012, year=2012, 
                        HC='Pete Carroll', 
                        OC='Darrell Bevell', 
                        DC='Gus Bradley',
                        QB='Russell Wilson')

seahawks_df2013 = yearly_data(df=seahawks_df2013, year=2013, 
                        HC='Pete Carroll', 
                        OC='Darrell Bevell', 
                        DC='Dan Quinn', 
                        QB='Russell Wilson')

seahawks_df2014 = yearly_data(df=seahawks_df2014, year=2014, 
                        HC='Pete Carroll', 
                        OC='Darrell Bevell', 
                        DC='Dan Quinn', 
                        QB='Russell Wilson')

seahawks_df2015 = yearly_data(df=seahawks_df2015, year=2015, 
                        HC='Pete Carroll', 
                        OC='Darrell Bevell', 
                        DC='Kris Richard', 
                        QB='Russell Wilson')

seahawks_df2016 = yearly_data(df=seahawks_df2016, year=2016, 
                        HC='Pete Carroll',
                        OC='Darrell Bevell', 
                        DC='Kris Richard', 
                        QB='Russell Wilson')

seahawks_df2017 = yearly_data(df=seahawks_df2017, year=2017, 
                        HC='Pete Carroll', 
                        OC='Darrell Bevell', 
                        DC='Kris Richard', 
                        QB='Russell Wilson')

seahawks_df2018 = yearly_data(df=seahawks_df2018, year=2018, 
                        HC='Pete Carroll', 
                        OC='Brian Schottenheimer', 
                        DC='Ken Norton', 
                        QB='Russell Wilson')

seahawks_df2019 = yearly_data(df=seahawks_df2019, year=2019, 
                        HC='Pete Carroll', 
                        OC='Brian Schottenheimer', 
                        DC='Ken Norton', 
                        QB='Russell Wilson')

seahawks_2020 = yearly_data(df=seahawks_2020, year=2020, 
                        HC='Pete Carroll', 
                        OC='Brian Schottenheimer', 
                        DC='Ken Norton', 
                        QB='Russell Wilson')

# Make master dataframe for team
seahawks_df = seahawks_2020.append(seahawks_df2019, ignore_index=True)
seahawks_df = seahawks_df.append(seahawks_df2018, ignore_index=True)
seahawks_df = seahawks_df.append(seahawks_df2017, ignore_index=True)
seahawks_df = seahawks_df.append(seahawks_df2016, ignore_index=True)
seahawks_df = seahawks_df.append(seahawks_df2015, ignore_index=True)
seahawks_df = seahawks_df.append(seahawks_df2014, ignore_index=True)
seahawks_df = seahawks_df.append(seahawks_df2013, ignore_index=True)
seahawks_df = seahawks_df.append(seahawks_df2012, ignore_index=True)
seahawks_df = seahawks_df.append(seahawks_df2011, ignore_index=True)
seahawks_df = seahawks_df.append(seahawks_df2010, ignore_index=True)

seahawks_df['Base_Team'] = 'Seahawks'
seahawks_df = seahawks_df.sort_values(by=['year', 'Week'], ascending=[False, True])

seahawks_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Sun,1.0,0.0,0.0,Atlanta Falcons,2020,Seahawks,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Pete Carroll,Brian Schottenheimer,Ken Norton,Russell Wilson
1,2,Sun,1.0,1.0,1.0,New England Patriots,2020,Seahawks,38.0,25.0,...,2.0,434.0,72.0,383.0,506.0,0,Pete Carroll,Brian Schottenheimer,Ken Norton,Russell Wilson
2,3,Sun,1.0,2.0,1.0,Dallas Cowboys,2020,Seahawks,73.0,55.0,...,3.0,831.0,139.0,812.0,970.0,0,Pete Carroll,Brian Schottenheimer,Ken Norton,Russell Wilson
3,4,Sun,1.0,3.0,0.0,Miami Dolphins,2020,Seahawks,111.0,86.0,...,6.0,1292.0,200.0,1224.0,1492.0,0,Pete Carroll,Brian Schottenheimer,Ken Norton,Russell Wilson
4,5,Sun,NaN,NaN,1.0,Minnesota Vikings,2020,Seahawks,142.0,109.0,...,8.0,1604.0,303.0,1665.0,1907.0,0,Pete Carroll,Brian Schottenheimer,Ken Norton,Russell Wilson
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,13,Sun,1.0,5.0,1.0,Carolina Panthers,2010,Seahawks,209.0,275.0,...,19.0,3065.0,1324.0,3292.0,4389.0,0,Pete Carroll,Jeremy Bates,Gus Bradley,Matt Hasselbeck
172,14,Sun,0.0,6.0,0.0,San Francisco 49ers,2010,Seahawks,240.0,289.0,...,20.0,3217.0,1455.0,3663.0,4672.0,0,Pete Carroll,Jeremy Bates,Gus Bradley,Matt Hasselbeck
173,15,Sun,0.0,6.0,1.0,Atlanta Falcons,2010,Seahawks,261.0,329.0,...,20.0,3458.0,1550.0,4024.0,5008.0,0,Pete Carroll,Jeremy Bates,Gus Bradley,Matt Hasselbeck
174,16,Sun,0.0,6.0,0.0,Tampa Bay Buccaneers,2010,Seahawks,279.0,363.0,...,21.0,3626.0,1648.0,4258.0,5274.0,0,Pete Carroll,Jeremy Bates,Gus Bradley,Matt Hasselbeck


In [66]:
seahawks_df.to_csv('Resources/indv_team_csvs/seahawks.csv')

### Cardinals Data, 2010-2019

In [67]:
# Data Source: https://www.pro-football-reference.com/teams/crd/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

cardinals_df2010 = pd.read_csv('Resources/Cardinals_Raw/cardinals_sportsref_download2010.csv')
cardinals_df2011 = pd.read_csv('Resources/Cardinals_Raw/cardinals_sportsref_download2011.csv')
cardinals_df2012 = pd.read_csv('Resources/Cardinals_Raw/cardinals_sportsref_download2012.csv')
cardinals_df2013 = pd.read_csv('Resources/Cardinals_Raw/cardinals_sportsref_download2013.csv')
cardinals_df2014 = pd.read_csv('Resources/Cardinals_Raw/cardinals_sportsref_download2014.csv')
cardinals_df2015 = pd.read_csv('Resources/Cardinals_Raw/cardinals_sportsref_download2015.csv')
cardinals_df2016 = pd.read_csv('Resources/Cardinals_Raw/cardinals_sportsref_download2016.csv')
cardinals_df2017 = pd.read_csv('Resources/Cardinals_Raw/cardinals_sportsref_download2017.csv')
cardinals_df2018 = pd.read_csv('Resources/Cardinals_Raw/cardinals_sportsref_download2018.csv')
cardinals_df2019 = pd.read_csv('Resources/Cardinals_Raw/cardinals_sportsref_download2019.csv')
cardinals_2020 = pd.read_csv('Resources/2020data/cardinals_sportsref_download2020.csv')

cardinals_df2010 = yearly_data(df=cardinals_df2010, year=2010, 
                        HC='Ken Whisenhunt', 
                        OC='Mike Miller', 
                        DC='Ray Horton', 
                        QB='Derek Anderson', 
                        QB2='John Skelton', QB2_range=range(14,18,1))
cardinals_df2010.loc[cardinals_df2010['Week'] == range(5,9,1), 'QB'] = "Max Hall"

cardinals_df2011 = yearly_data(df=cardinals_df2011, year=2011, 
                        HC='Ken Whisenhunt', 
                        OC='Mike Miller', 
                        DC='Ray Horton', 
                        QB='Kevin Kolb', 
                        QB2='John Skelton', QB2_range=range(9,12,1))
cardinals_df2011.loc[cardinals_df2011['Week'] == 17, 'QB'] = "John Skelton"

cardinals_df2012 = yearly_data(df=cardinals_df2012, year=2012, 
                        HC='Ken Whisenhunt', 
                        OC='Mike Miller', 
                        DC='Ray Horton',
                        QB='John Skelton', 
                        QB2='Kevin Kolb', QB2_range=range(2,7,1))
cardinals_df2012.loc[cardinals_df2012['Week'] == range(12,14,1), 'QB'] = "Ryan Lindley"
cardinals_df2012.loc[cardinals_df2012['Week'] == range(15,16,1), 'QB'] = "Ryan Lindley"
cardinals_df2012.loc[cardinals_df2012['Week'] == 17, 'QB'] = "Brian Hoyer"

cardinals_df2013 = yearly_data(df=cardinals_df2013, year=2013, 
                        HC='Bruce Arians', 
                        OC='Harold Goodwin', 
                        DC='Todd Bowles', 
                        QB='Carson Palmer')

cardinals_df2014 = yearly_data(df=cardinals_df2014, year=2014, 
                        HC='Bruce Arians', 
                        OC='Harold Goodwin', 
                        DC='Todd Bowles', 
                        QB='Drew Stanton', 
                        QB2='Carson Palmer', QB2_range=1)
cardinals_df2014.loc[cardinals_df2014['Week'] == range(6,11,1), 'QB'] = "Carson Palmer"
cardinals_df2014.loc[cardinals_df2014['Week'] == range(16,18,1), 'QB'] = "Ryan Lindley"

cardinals_df2015 = yearly_data(df=cardinals_df2015, year=2015, 
                        HC='Bruce Arians', 
                        OC='Harold Goodwin', 
                        DC='James Bettcher', 
                        QB='Carson Palmer')

cardinals_df2016 = yearly_data(df=cardinals_df2016, year=2016, 
                        HC='Bruce Arians', 
                        OC='Harold Goodwin', 
                        DC='James Bettcher', 
                        QB='Carson Palmer', 
                        QB2='Drew Stanton', QB2_range=5)

cardinals_df2017 = yearly_data(df=cardinals_df2017, year=2017, 
                        HC='Bruce Arians', 
                        OC='Harold Goodwin', 
                        DC='James Bettcher', 
                        QB='Carson Palmer',
                        QB2="Drew Stanton", QB2_range=range(9,18,1))
cardinals_df2017.loc[cardinals_df2017['Week'] == range(10,16,1), 'QB'] = 'Blaine Gabbert'

cardinals_df2018 = yearly_data(df=cardinals_df2018, year=2018, 
                        HC='Steve Wilks', 
                        OC='Mike McCoy', 
                        DC='Al Holcomb', 
                        QB='Josh Rosen', 
                        QB2='Sam Bradford', QB2_range=range(1,4,1))

cardinals_df2019 = yearly_data(df=cardinals_df2019, year=2019, 
                        HC='Kliff Kingsbury', 
                        OC='Kliff Kingsbury', 
                        DC='Vance Joseph', 
                        QB='Kyler Murray')

cardinals_2020 = yearly_data(df=cardinals_2020, year=2020, 
                        HC='Kliff Kingsbury', 
                        OC='Kliff Kingsbury', 
                        DC='Vance Joseph', 
                        QB='Kyler Murray')

# Make master dataframe for team
cardinals_df = cardinals_2020.append(cardinals_df2019, ignore_index=True)
cardinals_df = cardinals_df.append(cardinals_df2018, ignore_index=True)
cardinals_df = cardinals_df.append(cardinals_df2017, ignore_index=True)
cardinals_df = cardinals_df.append(cardinals_df2016, ignore_index=True)
cardinals_df = cardinals_df.append(cardinals_df2015, ignore_index=True)
cardinals_df = cardinals_df.append(cardinals_df2014, ignore_index=True)
cardinals_df = cardinals_df.append(cardinals_df2013, ignore_index=True)
cardinals_df = cardinals_df.append(cardinals_df2012, ignore_index=True)
cardinals_df = cardinals_df.append(cardinals_df2011, ignore_index=True)
cardinals_df = cardinals_df.append(cardinals_df2010, ignore_index=True)

cardinals_df['Base_Team'] = 'Cardinals'
cardinals_df = cardinals_df.sort_values(by=['year', 'Week'], ascending=[False, True])

cardinals_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Sun,1.0,0.0,0.0,San Francisco 49ers,2020,Cardinals,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Kliff Kingsbury,Kliff Kingsbury,Vance Joseph,Kyler Murray
1,2,Sun,1.0,1.0,1.0,Washington Football Team,2020,Cardinals,24.0,20.0,...,0.0,243.0,123.0,404.0,366.0,0,Kliff Kingsbury,Kliff Kingsbury,Vance Joseph,Kyler Murray
2,3,Sun,0.0,2.0,1.0,Detroit Lions,2020,Cardinals,54.0,35.0,...,2.0,442.0,240.0,842.0,682.0,0,Kliff Kingsbury,Kliff Kingsbury,Vance Joseph,Kyler Murray
3,4,Sun,0.0,2.0,0.0,Carolina Panthers,2020,Cardinals,77.0,61.0,...,2.0,674.0,330.0,1219.0,1004.0,0,Kliff Kingsbury,Kliff Kingsbury,Vance Joseph,Kyler Murray
4,5,Sun,NaN,NaN,0.0,New York Jets,2020,Cardinals,98.0,92.0,...,3.0,950.0,498.0,1481.0,1448.0,0,Kliff Kingsbury,Kliff Kingsbury,Vance Joseph,Kyler Murray
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,13,Sun,0.0,4.0,0.0,New York Jets,2010,Cardinals,180.0,227.0,...,23.0,2241.0,1366.0,3212.0,3607.0,0,Ken Whisenhunt,Mike Miller,Ray Horton,Derek Anderson
172,14,Sun,0.0,4.0,0.0,Seattle Seahawks,2010,Cardinals,186.0,234.0,...,27.0,2353.0,1543.0,3349.0,3896.0,0,Ken Whisenhunt,Mike Miller,Ray Horton,Derek Anderson
173,15,Sun,1.0,4.0,1.0,Detroit Lions,2010,Cardinals,186.0,292.0,...,28.0,2562.0,1827.0,3503.0,4389.0,0,Ken Whisenhunt,Mike Miller,Ray Horton,Derek Anderson
174,16,Sun,0.0,5.0,1.0,Chicago Bears,2010,Cardinals,224.0,302.0,...,32.0,2790.0,1911.0,3699.0,4701.0,0,Ken Whisenhunt,Mike Miller,Ray Horton,Derek Anderson


In [68]:
cardinals_df.to_csv('Resources/indv_team_csvs/cardinals.csv')

### Packers Data, 2010-2019

In [69]:
# Data Source: https://www.pro-football-reference.com/teams/gnb/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

packers_df2010 = pd.read_csv('Resources/Packers_Raw/packers_sportsref_download2010.csv')
packers_df2011 = pd.read_csv('Resources/Packers_Raw/packers_sportsref_download2011.csv')
packers_df2012 = pd.read_csv('Resources/Packers_Raw/packers_sportsref_download2012.csv')
packers_df2013 = pd.read_csv('Resources/Packers_Raw/packers_sportsref_download2013.csv')
packers_df2014 = pd.read_csv('Resources/Packers_Raw/packers_sportsref_download2014.csv')
packers_df2015 = pd.read_csv('Resources/Packers_Raw/packers_sportsref_download2015.csv')
packers_df2016 = pd.read_csv('Resources/Packers_Raw/packers_sportsref_download2016.csv')
packers_df2017 = pd.read_csv('Resources/Packers_Raw/packers_sportsref_download2017.csv')
packers_df2018 = pd.read_csv('Resources/Packers_Raw/packers_sportsref_download2018.csv')
packers_df2019 = pd.read_csv('Resources/Packers_Raw/packers_sportsref_download2019.csv')
packers_2020 = pd.read_csv('Resources/2020data/packers_sportsref_download2020.csv')

packers_df2010 = yearly_data(df=packers_df2010, year=2010, 
                        HC='Mike McCarthy', 
                        OC='Joe Philbin', 
                        DC='Dom Capers', 
                        QB='Aaron Rodgers', 
                        QB2='Matt Flynn', QB2_range=15)

packers_df2011 = yearly_data(df=packers_df2011, year=2011, 
                        HC='Mike McCarthy', 
                        OC='Joe Philbin', 
                        DC='Dom Capers', 
                        QB='Aaron Rodgers', 
                        QB2='Matt Flynn', QB2_range=17)

packers_df2012 = yearly_data(df=packers_df2012, year=2012, 
                        HC='Mike McCarthy', 
                        OC='Tom Clements', 
                        DC='Dom Capers',
                        QB='Aaron Rodgers')

packers_df2013 = yearly_data(df=packers_df2013, year=2013, 
                        HC='Mike McCarthy', 
                        OC='Tom Clements', 
                        DC='Dom Capers', 
                        QB='Aaron Rodgers', 
                        QB2='Matt Flynn', QB2_range=range(13,17,1))
packers_df2013.loc[packers_df2013['Week'] == range(11,13,1), 'QB'] = "Scott Tolzien"
packers_df2013.loc[packers_df2013['Week'] == 10, 'QB'] = "Seneca Wallace"

packers_df2014 = yearly_data(df=packers_df2014, year=2014, 
                        HC='Mike McCarthy', 
                        OC='Tom Clements', 
                        DC='Dom Capers', 
                        QB='Aaron Rodgers')

packers_df2015 = yearly_data(df=packers_df2015, year=2015, 
                        HC='Mike McCarthy', 
                        OC='Edgar Bennett', 
                        DC='Dom Capers', 
                        QB='Aaron Rodgers')

packers_df2016 = yearly_data(df=packers_df2016, year=2016, 
                        HC='Mike McCarthy',
                        OC='Edgar Bennett', 
                        DC="Dom Capers", 
                        QB='Aaron Rodgers')

packers_df2017 = yearly_data(df=packers_df2017, year=2017, 
                        HC='Mike McCarthy', 
                        OC='Edgar Bennett', 
                        DC="Dom Capers", 
                        QB='Brett Hundley', 
                        QB2='Aaron Rodgers', QB2_range=range(1,7,1))
packers_df2017.loc[packers_df2017['Week'] == 15, 'QB'] = "Aaron Rodgers"

packers_df2018 = yearly_data(df=packers_df2018, year=2018, 
                        HC='Mike McCarthy', HC2='Joe Philbin', HC2_range=range(14,18,1), 
                        OC='Joe Philbin', 
                        DC='Mike Pettine', 
                        QB='Aaron Rodgers')

packers_df2019 = yearly_data(df=packers_df2019, year=2019, 
                        HC='Matt LaFleur', 
                        OC='Nathaniel Hackett', 
                        DC='Mike Pettine', 
                        QB='Aaron Rodgers')

packers_2020 = yearly_data(df=packers_2020, year=2020, 
                        HC='Matt LaFleur', 
                        OC='Nathaniel Hackett', 
                        DC='Mike Pettine', 
                        QB='Aaron Rodgers')

# Make master dataframe for team
packers_df = packers_2020.append(packers_df2019, ignore_index=True)
packers_df = packers_df.append(packers_df2018, ignore_index=True)
packers_df = packers_df.append(packers_df2017, ignore_index=True)
packers_df = packers_df.append(packers_df2016, ignore_index=True)
packers_df = packers_df.append(packers_df2015, ignore_index=True)
packers_df = packers_df.append(packers_df2014, ignore_index=True)
packers_df = packers_df.append(packers_df2013, ignore_index=True)
packers_df = packers_df.append(packers_df2012, ignore_index=True)
packers_df = packers_df.append(packers_df2011, ignore_index=True)
packers_df = packers_df.append(packers_df2010, ignore_index=True)

packers_df['Base_Team'] = 'Packers'
packers_df = packers_df.sort_values(by=['year', 'Week'], ascending=[False, True])

packers_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Sun,1.0,0.0,0.0,Minnesota Vikings,2020,Packers,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Matt LaFleur,Nathaniel Hackett,Mike Pettine,Aaron Rodgers
1,2,Sun,1.0,1.0,1.0,Detroit Lions,2020,Packers,43.0,34.0,...,1.0,248.0,134.0,522.0,382.0,0,Matt LaFleur,Nathaniel Hackett,Mike Pettine,Aaron Rodgers
2,3,Sun,1.0,2.0,0.0,New Orleans Saints,2020,Packers,85.0,55.0,...,2.0,466.0,223.0,1010.0,689.0,0,Matt LaFleur,Nathaniel Hackett,Mike Pettine,Aaron Rodgers
3,4,Mon,1.0,3.0,1.0,Atlanta Falcons,2020,Packers,122.0,85.0,...,3.0,741.0,345.0,1379.0,1086.0,0,Matt LaFleur,Nathaniel Hackett,Mike Pettine,Aaron Rodgers
4,6,Sun,NaN,NaN,0.0,Tampa Bay Buccaneers,2020,Packers,152.0,101.0,...,3.0,990.0,423.0,1782.0,1413.0,1,Matt LaFleur,Nathaniel Hackett,Mike Pettine,Aaron Rodgers
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,13,Sun,1.0,7.0,1.0,San Francisco 49ers,2010,Packers,269.0,166.0,...,21.0,2290.0,1238.0,3894.0,3528.0,0,Mike McCarthy,Joe Philbin,Dom Capers,Aaron Rodgers
172,14,Sun,0.0,8.0,0.0,Detroit Lions,2010,Packers,303.0,182.0,...,22.0,2462.0,1335.0,4304.0,3797.0,0,Mike McCarthy,Joe Philbin,Dom Capers,Aaron Rodgers
173,15,Sun,0.0,8.0,0.0,New England Patriots,2010,Packers,306.0,189.0,...,24.0,2558.0,1525.0,4562.0,4083.0,0,Mike McCarthy,Joe Philbin,Dom Capers,Matt Flynn
174,16,Sun,1.0,8.0,1.0,New York Giants,2010,Packers,333.0,220.0,...,24.0,2694.0,1638.0,4931.0,4332.0,0,Mike McCarthy,Joe Philbin,Dom Capers,Aaron Rodgers


In [70]:
packers_df.to_csv('Resources/indv_team_csvs/packers.csv')

### Saints Data, 2010-2019

In [71]:
# Data Source: https://www.pro-football-reference.com/teams/nor/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

saints_df2010 = pd.read_csv('Resources/Saints_Raw/saints_sportsref_download2010.csv')
saints_df2011 = pd.read_csv('Resources/Saints_Raw/saints_sportsref_download2011.csv')
saints_df2012 = pd.read_csv('Resources/Saints_Raw/saints_sportsref_download2012.csv')
saints_df2013 = pd.read_csv('Resources/Saints_Raw/saints_sportsref_download2013.csv')
saints_df2014 = pd.read_csv('Resources/Saints_Raw/saints_sportsref_download2014.csv')
saints_df2015 = pd.read_csv('Resources/Saints_Raw/saints_sportsref_download2015.csv')
saints_df2016 = pd.read_csv('Resources/Saints_Raw/saints_sportsref_download2016.csv')
saints_df2017 = pd.read_csv('Resources/Saints_Raw/saints_sportsref_download2017.csv')
saints_df2018 = pd.read_csv('Resources/Saints_Raw/saints_sportsref_download2018.csv')
saints_df2019 = pd.read_csv('Resources/Saints_Raw/saints_sportsref_download2019.csv')
saints_2020 = pd.read_csv('Resources/2020data/saints_sportsref_download2020.csv')

saints_df2010 = yearly_data(df=saints_df2010, year=2010, 
                        HC='Sean Payton', 
                        OC='Pete Carmichael', 
                        DC='Gregg Williams', 
                        QB='Drew Brees')

saints_df2011 = yearly_data(df=saints_df2011, year=2011, 
                        HC='Sean Payton', 
                        OC='Pete Carmichael', 
                        DC='Gregg Williams', 
                        QB='Drew Brees')

saints_df2012 = yearly_data(df=saints_df2012, year=2012, 
                        HC='Aaron Kromer', HC2='Joe Vitt', HC2_range=range(8,18,1), 
                        OC='Pete Carmichael', 
                        DC='Steve Spagnuolo',
                        QB='Drew Brees')

saints_df2013 = yearly_data(df=saints_df2013, year=2013, 
                        HC='Sean Payton', 
                        OC='Pete Carmichael', 
                        DC='Rob Ryan', 
                        QB='Drew Brees')

saints_df2014 = yearly_data(df=saints_df2014, year=2014, 
                        HC='Sean Payton', 
                        OC='Pete Carmichael', 
                        DC='Rob Ryan', 
                        QB='Drew Brees')

saints_df2015 = yearly_data(df=saints_df2015, year=2015, 
                        HC='Sean Payton', 
                        OC='Pete Carmichael', 
                        DC='Rob Ryan', 
                        QB='Drew Brees', 
                        QB2='Luke McCown', QB2_range=3)
saints_df2015.loc[saints_df2015['Week'] == range(12,18,1), 'DC'] = "Dennis Allen"

saints_df2016 = yearly_data(df=saints_df2016, year=2016, 
                        HC='Sean Payton',
                        OC='Pete Carmichael', 
                        DC="Dennis Allen", 
                        QB='Drew Brees')

saints_df2017 = yearly_data(df=saints_df2017, year=2017, 
                        HC='Sean Payton', 
                        OC='Pete Carmichael', 
                        DC="Dennis Allen", 
                        QB='Drew Brees')

saints_df2018 = yearly_data(df=saints_df2018, year=2018, 
                        HC='Sean Payton', 
                        OC='Pete Carmichael', 
                        DC="Dennis Allen", 
                        QB='Drew Brees', 
                        QB2='Teddy Bridgewater', QB2_range=17)

saints_df2019 = yearly_data(df=saints_df2019, year=2019, 
                        HC='Sean Payton', 
                        OC='Pete Carmichael', 
                        DC="Dennis Allen", 
                        QB='Drew Brees', 
                        QB2='Teddy Bridgewater', QB2_range=range(3,8,1))

saints_2020 = yearly_data(df=saints_2020, year=2020, 
                        HC='Sean Payton', 
                        OC='Pete Carmichael', 
                        DC='Dennis Allen', 
                        QB='Drew Brees')

# Make master dataframe for team
saints_df = saints_2020.append(saints_df2019, ignore_index=True)
saints_df = saints_df.append(saints_df2018, ignore_index=True)
saints_df = saints_df.append(saints_df2017, ignore_index=True)
saints_df = saints_df.append(saints_df2016, ignore_index=True)
saints_df = saints_df.append(saints_df2015, ignore_index=True)
saints_df = saints_df.append(saints_df2014, ignore_index=True)
saints_df = saints_df.append(saints_df2013, ignore_index=True)
saints_df = saints_df.append(saints_df2012, ignore_index=True)
saints_df = saints_df.append(saints_df2011, ignore_index=True)
saints_df = saints_df.append(saints_df2010, ignore_index=True)

saints_df['Base_Team'] = 'Saints'
saints_df = saints_df.sort_values(by=['year', 'Week'], ascending=[False, True])

saints_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Sun,1.0,0.0,1.0,Tampa Bay Buccaneers,2020,Saints,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Sean Payton,Pete Carmichael,Dennis Allen,Drew Brees
1,2,Mon,0.0,1.0,0.0,Las Vegas Raiders,2020,Saints,34.0,23.0,...,3.0,224.0,86.0,271.0,310.0,0,Sean Payton,Pete Carmichael,Dennis Allen,Drew Brees
2,3,Sun,0.0,1.0,1.0,Green Bay Packers,2020,Saints,58.0,57.0,...,4.0,485.0,202.0,695.0,687.0,0,Sean Payton,Pete Carmichael,Dennis Allen,Drew Brees
3,4,Sun,1.0,1.0,0.0,Detroit Lions,2020,Saints,88.0,94.0,...,4.0,756.0,300.0,1092.0,1056.0,0,Sean Payton,Pete Carmichael,Dennis Allen,Drew Brees
4,5,Mon,NaN,NaN,1.0,Los Angeles Chargers,2020,Saints,123.0,123.0,...,5.0,947.0,390.0,1484.0,1337.0,0,Sean Payton,Pete Carmichael,Dennis Allen,Drew Brees
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,13,Sun,1.0,8.0,0.0,Cincinnati Bengals,2010,Saints,265.0,197.0,...,18.0,2176.0,1198.0,4237.0,3374.0,0,Sean Payton,Pete Carmichael,Gregg Williams,Drew Brees
172,14,Sun,1.0,9.0,1.0,St. Louis Rams,2010,Saints,299.0,227.0,...,18.0,2391.0,1294.0,4673.0,3685.0,0,Sean Payton,Pete Carmichael,Gregg Williams,Drew Brees
173,15,Sun,0.0,10.0,0.0,Baltimore Ravens,2010,Saints,330.0,240.0,...,21.0,2582.0,1430.0,5018.0,4012.0,0,Sean Payton,Pete Carmichael,Gregg Williams,Drew Brees
174,16,Mon,1.0,10.0,0.0,Atlanta Falcons,2010,Saints,354.0,270.0,...,21.0,2730.0,1638.0,5287.0,4368.0,0,Sean Payton,Pete Carmichael,Gregg Williams,Drew Brees


In [72]:
saints_df.to_csv('Resources/indv_team_csvs/saints.csv')

### Giants Data, 2010-2019

In [73]:
# Data Source: https://www.pro-football-reference.com/teams/nyg/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

giants_df2010 = pd.read_csv('Resources/Giants_Raw/giants_sportsref_download2010.csv')
giants_df2011 = pd.read_csv('Resources/Giants_Raw/giants_sportsref_download2011.csv')
giants_df2012 = pd.read_csv('Resources/Giants_Raw/giants_sportsref_download2012.csv')
giants_df2013 = pd.read_csv('Resources/Giants_Raw/giants_sportsref_download2013.csv')
giants_df2014 = pd.read_csv('Resources/Giants_Raw/giants_sportsref_download2014.csv')
giants_df2015 = pd.read_csv('Resources/Giants_Raw/giants_sportsref_download2015.csv')
giants_df2016 = pd.read_csv('Resources/Giants_Raw/giants_sportsref_download2016.csv')
giants_df2017 = pd.read_csv('Resources/Giants_Raw/giants_sportsref_download2017.csv')
giants_df2018 = pd.read_csv('Resources/Giants_Raw/giants_sportsref_download2018.csv')
giants_df2019 = pd.read_csv('Resources/Giants_Raw/giants_sportsref_download2019.csv')
giants_2020 = pd.read_csv('Resources/2020data/giants_sportsref_download2020.csv')

giants_df2010 = yearly_data(df=giants_df2010, year=2010, 
                        HC='Tom Coughlin', 
                        OC='Kevin Gilbride', 
                        DC='Perry Fewell', 
                        QB='Eli Manning')

giants_df2011 = yearly_data(df=giants_df2011, year=2011, 
                        HC='Tom Coughlin', 
                        OC='Kevin Gilbride', 
                        DC='Perry Fewell', 
                        QB='Eli Manning')

giants_df2012 = yearly_data(df=giants_df2012, year=2012, 
                        HC='Tom Coughlin', 
                        OC='Kevin Gilbride', 
                        DC='Perry Fewell',
                        QB='Eli Manning')

giants_df2013 = yearly_data(df=giants_df2013, year=2013, 
                        HC='Tom Coughlin', 
                        OC='Kevin Gilbride', 
                        DC='Perry Fewell', 
                        QB='Eli Manning')

giants_df2014 = yearly_data(df=giants_df2014, year=2014, 
                        HC='Tom Coughlin', 
                        OC='Ben McAdoo', 
                        DC='Perry Fewell', 
                        QB='Eli Manning')

giants_df2015 = yearly_data(df=giants_df2015, year=2015, 
                        HC='Tom Coughlin', 
                        OC='Ben McAdoo', 
                        DC='Steve Spagnuolo', 
                        QB='Eli Manning')

giants_df2016 = yearly_data(df=giants_df2016, year=2016, 
                        HC='Ben McAdoo',
                        OC='Mike Sullivan', 
                        DC="Steve Spagnuolo", 
                        QB='Eli Manning')

giants_df2017 = yearly_data(df=giants_df2017, year=2017, 
                        HC='Ben McAdoo', HC2='Steve Spagnuolo', HC2_range=range(14,18,1), 
                        OC='Mike Sullivan', 
                        DC="Steve Spagnuolo", 
                        QB='Eli Manning', 
                        QB2='Geno Smith', QB2_range=13)

giants_df2018 = yearly_data(df=giants_df2018, year=2018, 
                        HC='Pat Shurmur', 
                        OC='Mike Shula', 
                        DC='James Bettcher', 
                        QB='Eli Manning')

giants_df2019 = yearly_data(df=giants_df2019, year=2019, 
                        HC='Pat Shurmur', 
                        OC='Mike Shula', 
                        DC='James Bettcher', 
                        QB='Daniel Jones', 
                        QB2='Eli Manning', QB2_range=range(1,3,1))
giants_df2019.loc[giants_df2019['Week'] == range(14,16,1), 'QB'] = "Eli Manning"

giants_2020 = yearly_data(df=giants_2020, year=2020, 
                        HC='Joe Judge', 
                        OC='Jason Garrett', 
                        DC='Patrick Graham', 
                        QB='Daniel Jones')

# Make master dataframe for team
giants_df = giants_2020.append(giants_df2019, ignore_index=True)
giants_df = giants_df.append(giants_df2018, ignore_index=True)
giants_df = giants_df.append(giants_df2017, ignore_index=True)
giants_df = giants_df.append(giants_df2016, ignore_index=True)
giants_df = giants_df.append(giants_df2015, ignore_index=True)
giants_df = giants_df.append(giants_df2014, ignore_index=True)
giants_df = giants_df.append(giants_df2013, ignore_index=True)
giants_df = giants_df.append(giants_df2012, ignore_index=True)
giants_df = giants_df.append(giants_df2011, ignore_index=True)
giants_df = giants_df.append(giants_df2010, ignore_index=True)

giants_df['Base_Team'] = 'Giants'
giants_df = giants_df.sort_values(by=['year', 'Week'], ascending=[False, True])

giants_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Mon,0.0,0.0,1.0,Pittsburgh Steelers,2020,Giants,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Joe Judge,Jason Garrett,Patrick Graham,Daniel Jones
1,2,Sun,0.0,0.0,0.0,Chicago Bears,2020,Giants,16.0,26.0,...,1.0,208.0,141.0,291.0,349.0,0,Joe Judge,Jason Garrett,Patrick Graham,Daniel Jones
2,3,Sun,0.0,0.0,1.0,San Francisco 49ers,2020,Giants,29.0,43.0,...,3.0,377.0,276.0,586.0,653.0,0,Joe Judge,Jason Garrett,Patrick Graham,Daniel Jones
3,4,Sun,0.0,0.0,0.0,Los Angeles Rams,2020,Giants,38.0,79.0,...,3.0,704.0,369.0,817.0,1073.0,0,Joe Judge,Jason Garrett,Patrick Graham,Daniel Jones
4,5,Sun,NaN,NaN,0.0,Dallas Cowboys,2020,Giants,47.0,96.0,...,4.0,886.0,427.0,1112.0,1313.0,0,Joe Judge,Jason Garrett,Patrick Graham,Daniel Jones
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,13,Sun,1.0,7.0,1.0,Washington Redskins,2010,Giants,277.0,240.0,...,24.0,2050.0,1102.0,4257.0,3152.0,0,Tom Coughlin,Kevin Gilbride,Perry Fewell,Eli Manning
172,14,Mon,1.0,8.0,0.0,Minnesota Vikings,2010,Giants,308.0,247.0,...,30.0,2314.0,1176.0,4615.0,3490.0,0,Tom Coughlin,Kevin Gilbride,Perry Fewell,Eli Manning
173,15,Sun,0.0,9.0,1.0,Philadelphia Eagles,2010,Giants,329.0,250.0,...,31.0,2417.0,1237.0,5010.0,3654.0,0,Tom Coughlin,Kevin Gilbride,Perry Fewell,Eli Manning
174,16,Sun,0.0,9.0,0.0,Green Bay Packers,2010,Giants,360.0,288.0,...,34.0,2638.0,1434.0,5374.0,4072.0,0,Tom Coughlin,Kevin Gilbride,Perry Fewell,Eli Manning


In [74]:
giants_df.to_csv('Resources/indv_team_csvs/giants.csv')

### Bears Data, 2010-2019

In [75]:
# Data Source: https://www.pro-football-reference.com/teams/chi/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

bears_df2010 = pd.read_csv('Resources/Bears_Raw/bears_sportsref_download2010.csv')
bears_df2011 = pd.read_csv('Resources/Bears_Raw/bears_sportsref_download2011.csv')
bears_df2012 = pd.read_csv('Resources/Bears_Raw/bears_sportsref_download2012.csv')
bears_df2013 = pd.read_csv('Resources/Bears_Raw/bears_sportsref_download2013.csv')
bears_df2014 = pd.read_csv('Resources/Bears_Raw/bears_sportsref_download2014.csv')
bears_df2015 = pd.read_csv('Resources/Bears_Raw/bears_sportsref_download2015.csv')
bears_df2016 = pd.read_csv('Resources/Bears_Raw/bears_sportsref_download2016.csv')
bears_df2017 = pd.read_csv('Resources/Bears_Raw/bears_sportsref_download2017.csv')
bears_df2018 = pd.read_csv('Resources/Bears_Raw/bears_sportsref_download2018.csv')
bears_df2019 = pd.read_csv('Resources/Bears_Raw/bears_sportsref_download2019.csv')
bears_2020 = pd.read_csv('Resources/2020data/bears_sportsref_download2020.csv')

bears_df2010 = yearly_data(df=bears_df2010, year=2010, 
                        HC='Lovie Smith', 
                        OC='Mike Martz', 
                        DC='Rod Marinelli', 
                        QB='Jay Cutler', 
                        QB2='Todd Collins', QB2_range=5)

bears_df2011 = yearly_data(df=bears_df2011, year=2011, 
                        HC='Lovie Smith', 
                        OC='Mike Martz', 
                        DC='Rod Marinelli', 
                        QB='Jay Cutler', 
                        QB2='Caleb Hanie', QB2_range=range(12,16,1))
bears_df2011.loc[bears_df2011['Week'] == range(16,18,1), 'QB'] = "Josh McCown"

bears_df2012 = yearly_data(df=bears_df2012, year=2012, 
                        HC='Lovie Smith', 
                        OC='Mike Tice', 
                        DC='Rod Marinelli',
                        QB='Jay Cutler', 
                        QB2='Jason Campbell', QB2_range=11)

bears_df2013 = yearly_data(df=bears_df2013, year=2013, 
                        HC='Marc Trestman', 
                        OC='Aaron Kromer', 
                        DC='Mel Tucker', 
                        QB='Jay Cutler', 
                        QB2='Josh McCown', QB2_range=9)
bears_df2013.loc[bears_df2013['Week'] == range(11,15,1), 'QB'] = "Josh McCown"

bears_df2014 = yearly_data(df=bears_df2014, year=2014, 
                        HC='Marc Trestman', 
                        OC='Aaron Kromer', 
                        DC='Mel Tucker', 
                        QB='Jay Cutler', 
                        QB2='Jimmy Clausen', QB2_range=16)

bears_df2015 = yearly_data(df=bears_df2015, year=2015, 
                        HC='John Fox', 
                        OC='Adam Gase', 
                        DC='Vic Fangio', 
                        QB='Jay Cutler', 
                        QB2='Jimmy Clausen', QB2_range=3)

bears_df2016 = yearly_data(df=bears_df2016, year=2016, 
                        HC='John Fox',
                        OC='Dowell Loggains', 
                        DC="Vic Fangio", 
                        QB='Jay Cutler', 
                        QB2='Matt Barkley', QB2_range=range(12,18,1))
bears_df2016.loc[bears_df2016['Week'] == range(3,8,1), 'QB'] = "Brian Hoyer"

bears_df2017 = yearly_data(df=bears_df2017, year=2017, 
                        HC='John Fox',
                        OC='Dowell Loggains', 
                        DC="Vic Fangio", 
                        QB='Mitchell Trubisky', 
                        QB2='Mike Glennon', QB2_range=range(1,5,1))

bears_df2018 = yearly_data(df=bears_df2018, year=2018, 
                        HC='Matt Nagy', 
                        OC='Mark Helfrich', 
                        DC='Vic Fangio', 
                        QB='Mitchell Trubisky', 
                        QB2='Chase Daniel', QB2_range=range(12,14,1))

bears_df2019 = yearly_data(df=bears_df2019, year=2019, 
                        HC='Matt Nagy', 
                        OC='Mark Helfrich', 
                        DC='Chuck Pagano', 
                        QB='Mitchell Trubisky', 
                        QB2='Chase Daniel', QB2_range=5)

bears_2020 = yearly_data(df=bears_2020, year=2020, 
                        HC='Matt Nagy', 
                        OC='Bill Lazor', 
                        DC='Chuck Pagano', 
                        QB='Mitchell Trubisky', 
                        QB2='Nick Foles', QB2_range=range(4,18,1))

# Make master dataframe for team
bears_df = bears_2020.append(bears_df2019, ignore_index=True)
bears_df = bears_df.append(bears_df2018, ignore_index=True)
bears_df = bears_df.append(bears_df2017, ignore_index=True)
bears_df = bears_df.append(bears_df2016, ignore_index=True)
bears_df = bears_df.append(bears_df2015, ignore_index=True)
bears_df = bears_df.append(bears_df2014, ignore_index=True)
bears_df = bears_df.append(bears_df2013, ignore_index=True)
bears_df = bears_df.append(bears_df2012, ignore_index=True)
bears_df = bears_df.append(bears_df2011, ignore_index=True)
bears_df = bears_df.append(bears_df2010, ignore_index=True)

bears_df['Base_Team'] = 'Bears'
bears_df = bears_df.sort_values(by=['year', 'Week'], ascending=[False, True])

bears_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Sun,1.0,0.0,0.0,Detroit Lions,2020,Bears,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Matt Nagy,Bill Lazor,Chuck Pagano,Mitchell Trubisky
1,2,Sun,1.0,1.0,1.0,New York Giants,2020,Bears,27.0,23.0,...,1.0,288.0,138.0,363.0,426.0,0,Matt Nagy,Bill Lazor,Chuck Pagano,Mitchell Trubisky
2,3,Sun,1.0,2.0,0.0,Atlanta Falcons,2020,Bears,44.0,36.0,...,3.0,508.0,213.0,667.0,721.0,0,Matt Nagy,Bill Lazor,Chuck Pagano,Mitchell Trubisky
3,4,Sun,0.0,3.0,1.0,Indianapolis Colts,2020,Bears,74.0,62.0,...,4.0,735.0,357.0,1104.0,1092.0,0,Matt Nagy,Bill Lazor,Chuck Pagano,Mitchell Trubisky
4,5,Thu,NaN,NaN,1.0,Tampa Bay Buccaneers,2020,Bears,85.0,81.0,...,4.0,921.0,460.0,1373.0,1381.0,0,Matt Nagy,Bill Lazor,Chuck Pagano,Mitchell Trubisky
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,13,Sun,1.0,8.0,0.0,Detroit Lions,2010,Bears,222.0,172.0,...,26.0,2417.0,885.0,3293.0,3302.0,0,Lovie Smith,Mike Martz,Rod Marinelli,Jay Cutler
172,14,Sun,0.0,9.0,1.0,New England Patriots,2010,Bears,246.0,192.0,...,26.0,2585.0,1019.0,3604.0,3604.0,0,Lovie Smith,Mike Martz,Rod Marinelli,Jay Cutler
173,15,Mon,1.0,9.0,0.0,Minnesota Vikings,2010,Bears,253.0,228.0,...,26.0,2936.0,1143.0,3789.0,4079.0,0,Lovie Smith,Mike Martz,Rod Marinelli,Jay Cutler
174,16,Sun,1.0,10.0,1.0,New York Jets,2010,Bears,293.0,242.0,...,31.0,3095.0,1257.0,4082.0,4352.0,0,Lovie Smith,Mike Martz,Rod Marinelli,Jay Cutler


In [76]:
bears_df.to_csv('Resources/indv_team_csvs/bears.csv')

### Panthers Data, 2010-2019

In [77]:
# Data Source: https://www.pro-football-reference.com/teams/car/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

panthers_df2010 = pd.read_csv('Resources/Panthers_Raw/panthers_sportsref_download2010.csv')
panthers_df2011 = pd.read_csv('Resources/Panthers_Raw/panthers_sportsref_download2011.csv')
panthers_df2012 = pd.read_csv('Resources/Panthers_Raw/panthers_sportsref_download2012.csv')
panthers_df2013 = pd.read_csv('Resources/Panthers_Raw/panthers_sportsref_download2013.csv')
panthers_df2014 = pd.read_csv('Resources/Panthers_Raw/panthers_sportsref_download2014.csv')
panthers_df2015 = pd.read_csv('Resources/Panthers_Raw/panthers_sportsref_download2015.csv')
panthers_df2016 = pd.read_csv('Resources/Panthers_Raw/panthers_sportsref_download2016.csv')
panthers_df2017 = pd.read_csv('Resources/Panthers_Raw/panthers_sportsref_download2017.csv')
panthers_df2018 = pd.read_csv('Resources/Panthers_Raw/panthers_sportsref_download2018.csv')
panthers_df2019 = pd.read_csv('Resources/Panthers_Raw/panthers_sportsref_download2019.csv')
panthers_2020 = pd.read_csv('Resources/2020data/panthers_sportsref_download2020.csv')

panthers_df2010 = yearly_data(df=panthers_df2010, year=2010, 
                        HC='John Fox', 
                        OC='Jeff Davidson', 
                        DC='Ron Meeks', 
                        QB='Jimmy Clausen', 
                        QB2='Matt Moore', QB2_range=range(1,3,1))
panthers_df2010.loc[panthers_df2010['Week'] == range(7,10,1), 'QB'] = "Matt Moore"
panthers_df2010.loc[panthers_df2010['Week'] == 11, 'QB'] = "Brian St. Pierre"

panthers_df2011 = yearly_data(df=panthers_df2011, year=2011, 
                        HC='Ron Rivera', 
                        OC='Rob Chudzinski', 
                        DC='Sean McDermott', 
                        QB='Cam Newton')

panthers_df2012 = yearly_data(df=panthers_df2012, year=2012, 
                        HC='Ron Rivera', 
                        OC='Rob Chudzinski', 
                        DC='Sean McDermott',
                        QB='Cam Newton')

panthers_df2013 = yearly_data(df=panthers_df2013, year=2013, 
                        HC='Ron Rivera', 
                        OC='Mike Shula', 
                        DC='Sean McDermott', 
                        QB='Cam Newton')

panthers_df2014 = yearly_data(df=panthers_df2014, year=2014, 
                        HC='Ron Rivera', 
                        OC='Mike Shula', 
                        DC='Sean McDermott', 
                        QB='Cam Newton', 
                        QB2='Derek Anderson', QB2_range=1)
panthers_df2014.loc[panthers_df2014['Week'] == 15, 'QB'] = "Derek Anderson"

panthers_df2015 = yearly_data(df=panthers_df2015, year=2015, 
                        HC='Ron Rivera', 
                        OC='Mike Shula', 
                        DC='Sean McDermott', 
                        QB='Cam Newton')

panthers_df2016 = yearly_data(df=panthers_df2016, year=2016, 
                        HC='Ron Rivera',
                        OC='Mike Shula', 
                        DC='Sean McDermott', 
                        QB='Cam Newton', 
                        QB2='Derek Anderson', QB2_range=5)
panthers_df2016.loc[panthers_df2016['Week'] == 13, 'QB'] = "Derek Anderson"

panthers_df2017 = yearly_data(df=panthers_df2017, year=2017, 
                        HC='Ron Rivera', 
                        OC='Mike Shula', 
                        DC='Steve Wilks', 
                        QB='Cam Newton')

panthers_df2018 = yearly_data(df=panthers_df2018, year=2018, 
                        HC='Ron Rivera', 
                        OC='Norv Turner', 
                        DC='Eric Washington', 
                        QB='Cam Newton',
                        QB2='Taylor Heinicke', QB2_range=16)
panthers_df2018.loc[panthers_df2018['Week'] == 17, 'QB'] = "Kyle Allen"

panthers_df2019 = yearly_data(df=panthers_df2019, year=2019, 
                        HC='Ron Rivera', HC2='Perry Fewell', HC2_range=range(14,18,1), 
                        OC='Norv Turner', 
                        DC='Eric Washington', 
                        QB='Kyle Allen',
                        QB2='Cam Newton', QB2_range=range(1,3,1))
panthers_df2019.loc[panthers_df2019['Week'] == 17, 'QB'] = "Will Grier"
panthers_df2019.loc[panthers_df2019['Week'] == range(14,18,1), 'OC'] = "Scott Turner"

panthers_2020 = yearly_data(df=panthers_2020, year=2020, 
                        HC='Matt Rhule', 
                        OC='Joe Brady', 
                        DC='Phil Snow', 
                        QB='Teddy Bridgewater')

# Make master dataframe for team
panthers_df = panthers_2020.append(panthers_df2019, ignore_index=True)
panthers_df = panthers_df.append(panthers_df2018, ignore_index=True)
panthers_df = panthers_df.append(panthers_df2017, ignore_index=True)
panthers_df = panthers_df.append(panthers_df2016, ignore_index=True)
panthers_df = panthers_df.append(panthers_df2015, ignore_index=True)
panthers_df = panthers_df.append(panthers_df2014, ignore_index=True)
panthers_df = panthers_df.append(panthers_df2013, ignore_index=True)
panthers_df = panthers_df.append(panthers_df2012, ignore_index=True)
panthers_df = panthers_df.append(panthers_df2011, ignore_index=True)
panthers_df = panthers_df.append(panthers_df2010, ignore_index=True)

panthers_df['Base_Team'] = 'Panthers'
panthers_df = panthers_df.sort_values(by=['year', 'Week'], ascending=[False, True])

panthers_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Sun,0.0,0.0,1.0,Las Vegas Raiders,2020,Panthers,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Matt Rhule,Joe Brady,Phil Snow,Teddy Bridgewater
1,2,Sun,0.0,0.0,0.0,Tampa Bay Buccaneers,2020,Panthers,30.0,34.0,...,0.0,239.0,133.0,388.0,372.0,0,Matt Rhule,Joe Brady,Phil Snow,Teddy Bridgewater
2,3,Sun,1.0,0.0,0.0,Los Angeles Chargers,2020,Panthers,47.0,65.0,...,2.0,456.0,255.0,815.0,711.0,0,Matt Rhule,Joe Brady,Phil Snow,Teddy Bridgewater
3,4,Sun,1.0,1.0,1.0,Arizona Cardinals,2020,Panthers,68.0,81.0,...,6.0,775.0,372.0,1117.0,1147.0,0,Matt Rhule,Joe Brady,Phil Snow,Teddy Bridgewater
4,5,Sun,NaN,NaN,0.0,Atlanta Falcons,2020,Panthers,99.0,102.0,...,7.0,908.0,501.0,1561.0,1409.0,0,Matt Rhule,Joe Brady,Phil Snow,Teddy Bridgewater
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,13,Sun,0.0,1.0,0.0,Seattle Seahawks,2010,Panthers,140.0,276.0,...,21.0,2262.0,1436.0,2851.0,3698.0,0,John Fox,Jeff Davidson,Ron Meeks,Jimmy Clausen
172,14,Sun,0.0,1.0,1.0,Atlanta Falcons,2010,Panthers,154.0,307.0,...,23.0,2472.0,1597.0,3134.0,4069.0,0,John Fox,Jeff Davidson,Ron Meeks,Jimmy Clausen
173,15,Sun,1.0,1.0,1.0,Arizona Cardinals,2010,Panthers,164.0,338.0,...,24.0,2672.0,1724.0,3422.0,4396.0,0,John Fox,Jeff Davidson,Ron Meeks,Jimmy Clausen
174,16,Thu,0.0,2.0,0.0,Pittsburgh Steelers,2010,Panthers,183.0,350.0,...,26.0,2847.0,1767.0,3725.0,4614.0,0,John Fox,Jeff Davidson,Ron Meeks,Jimmy Clausen


In [78]:
panthers_df.to_csv('Resources/indv_team_csvs/panthers.csv')

### Bucs Data, 2010-2019

In [79]:
# Data Source: https://www.pro-football-reference.com/teams/tam/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

bucs_df2010 = pd.read_csv('Resources/Bucs_Raw/bucs_sportsref_download2010.csv')
bucs_df2011 = pd.read_csv('Resources/Bucs_Raw/bucs_sportsref_download2011.csv')
bucs_df2012 = pd.read_csv('Resources/Bucs_Raw/bucs_sportsref_download2012.csv')
bucs_df2013 = pd.read_csv('Resources/Bucs_Raw/bucs_sportsref_download2013.csv')
bucs_df2014 = pd.read_csv('Resources/Bucs_Raw/bucs_sportsref_download2014.csv')
bucs_df2015 = pd.read_csv('Resources/Bucs_Raw/bucs_sportsref_download2015.csv')
bucs_df2016 = pd.read_csv('Resources/Bucs_Raw/bucs_sportsref_download2016.csv')
bucs_df2017 = pd.read_csv('Resources/Bucs_Raw/bucs_sportsref_download2017.csv')
bucs_df2018 = pd.read_csv('Resources/Bucs_Raw/bucs_sportsref_download2018.csv')
bucs_df2019 = pd.read_csv('Resources/Bucs_Raw/bucs_sportsref_download2019.csv')
bucs_2020 = pd.read_csv('Resources/2020data/bucs_sportsref_download2020.csv')

bucs_df2010 = yearly_data(df=bucs_df2010, year=2010, 
                        HC='Raheem Morris', 
                        OC='Greg Olson', 
                        DC='Raheem Morris', 
                        QB='Josh Freeman')

bucs_df2011 = yearly_data(df=bucs_df2011, year=2011, 
                        HC='Raheem Morris', 
                        OC='Greg Olson', 
                        DC='Keith Millard', 
                        QB='Josh Freeman', 
                        QB2='Josh Johnson', QB2_range=13)

bucs_df2012 = yearly_data(df=bucs_df2012, year=2012, 
                        HC='Greg Schiano', 
                        OC='Mike Sullivan', 
                        DC='Bill Sheridan',
                        QB='Josh Freeman')

bucs_df2013 = yearly_data(df=bucs_df2013, year=2013, 
                        HC='Greg Schiano', 
                        OC='Mike Sullivan', 
                        DC='Bill Sheridan', 
                        QB='Mike Glennon', 
                        QB2='Josh Freeman', QB2_range=range(1,3,1))

bucs_df2014 = yearly_data(df=bucs_df2014, year=2014, 
                        HC='Lovie Smith', 
                        OC='Jeff Tedford', 
                        DC='Leslie Frazier', 
                        QB='Josh McCown', 
                        QB2='Mike Glennon', QB2_range=range(4,10,1))

bucs_df2015 = yearly_data(df=bucs_df2015, year=2015, 
                        HC='Lovie Smith', 
                        OC='Dirk Koetter', 
                        DC='Leslie Frazier', 
                        QB='Jameis Winston')

bucs_df2016 = yearly_data(df=bucs_df2016, year=2016, 
                        HC='Dirk Koetter',
                        OC='Todd Monken', 
                        DC="Mike Smith", 
                        QB='Jameis Winston')

bucs_df2017 = yearly_data(df=bucs_df2017, year=2017, 
                        HC='Dirk Koetter',
                        OC='Todd Monken', 
                        DC="Mike Smith", 
                        QB='Jameis Winston', 
                        QB2='Ryan Fitzpatrick', QB2_range=range(10,13,1))

bucs_df2018 = yearly_data(df=bucs_df2018, year=2018, 
                        HC='Dirk Koetter',
                        OC='Todd Monken', 
                        DC="Mike Smith", 
                        QB='Jameis Winston', 
                        QB2='Ryan Fitzpatrick', QB2_range=range(1,5,1))
bucs_df2018.loc[bucs_df2018['Week'] == range(9,12,1), 'QB'] = "Ryan Fitzpatrick"
bucs_df2018.loc[bucs_df2018['Week'] == range(7,18,1), 'OC'] = "Mark Duffner"

bucs_df2019 = yearly_data(df=bucs_df2019, year=2019, 
                        HC='Bruce Arians', 
                        OC='Byron Leftwich', 
                        DC='Todd Bowles', 
                        QB='Jameis Winston')

bucs_2020 = yearly_data(df=bucs_2020, year=2020, 
                        HC='Bruce Arians', 
                        OC='Byron Leftwich', 
                        DC='Todd Bowles', 
                        QB='Tom Brady')

# Make master dataframe for team
bucs_df = bucs_2020.append(bucs_df2019, ignore_index=True)
bucs_df = bucs_df.append(bucs_df2018, ignore_index=True)
bucs_df = bucs_df.append(bucs_df2017, ignore_index=True)
bucs_df = bucs_df.append(bucs_df2016, ignore_index=True)
bucs_df = bucs_df.append(bucs_df2015, ignore_index=True)
bucs_df = bucs_df.append(bucs_df2014, ignore_index=True)
bucs_df = bucs_df.append(bucs_df2013, ignore_index=True)
bucs_df = bucs_df.append(bucs_df2012, ignore_index=True)
bucs_df = bucs_df.append(bucs_df2011, ignore_index=True)
bucs_df = bucs_df.append(bucs_df2010, ignore_index=True)

bucs_df['Base_Team'] = 'Buccaneers'
bucs_df = bucs_df.sort_values(by=['year', 'Week'], ascending=[False, True])

bucs_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Sun,0.0,0.0,0.0,New Orleans Saints,2020,Buccaneers,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Bruce Arians,Byron Leftwich,Todd Bowles,Tom Brady
1,2,Sun,1.0,0.0,1.0,Carolina Panthers,2020,Buccaneers,23.0,34.0,...,0.0,189.0,82.0,310.0,271.0,0,Bruce Arians,Byron Leftwich,Todd Bowles,Tom Brady
2,3,Sun,1.0,1.0,0.0,Denver Broncos,2020,Buccaneers,54.0,51.0,...,4.0,529.0,169.0,649.0,698.0,0,Bruce Arians,Byron Leftwich,Todd Bowles,Tom Brady
3,4,Sun,1.0,2.0,1.0,Los Angeles Chargers,2020,Buccaneers,82.0,61.0,...,6.0,713.0,211.0,1002.0,924.0,0,Bruce Arians,Byron Leftwich,Todd Bowles,Tom Brady
4,5,Thu,NaN,NaN,0.0,Chicago Bears,2020,Buccaneers,120.0,92.0,...,8.0,991.0,257.0,1486.0,1248.0,0,Bruce Arians,Byron Leftwich,Todd Bowles,Tom Brady
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,13,Sun,0.0,7.0,1.0,Atlanta Falcons,2010,Buccaneers,219.0,223.0,...,22.0,2265.0,1457.0,3513.0,3722.0,0,Raheem Morris,Greg Olson,Raheem Morris,Josh Freeman
172,14,Sun,1.0,7.0,0.0,Washington Redskins,2010,Buccaneers,243.0,251.0,...,24.0,2470.0,1542.0,3838.0,4012.0,0,Raheem Morris,Greg Olson,Raheem Morris,Josh Freeman
173,15,Sun,0.0,8.0,1.0,Detroit Lions,2010,Buccaneers,260.0,267.0,...,25.0,2681.0,1730.0,4203.0,4411.0,0,Raheem Morris,Greg Olson,Raheem Morris,Josh Freeman
174,16,Sun,1.0,8.0,1.0,Seattle Seahawks,2010,Buccaneers,280.0,290.0,...,25.0,2933.0,1911.0,4606.0,4844.0,0,Raheem Morris,Greg Olson,Raheem Morris,Josh Freeman


In [80]:
bucs_df.to_csv('Resources/indv_team_csvs/bucs.csv')

### Falcons Data, 2010-2019

In [81]:
# Data Source: https://www.pro-football-reference.com/teams/atl/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

falcons_df2010 = pd.read_csv('Resources/Falcons_Raw/falcons_sportsref_download2010.csv')
falcons_df2011 = pd.read_csv('Resources/Falcons_Raw/falcons_sportsref_download2011.csv')
falcons_df2012 = pd.read_csv('Resources/Falcons_Raw/falcons_sportsref_download2012.csv')
falcons_df2013 = pd.read_csv('Resources/Falcons_Raw/falcons_sportsref_download2013.csv')
falcons_df2014 = pd.read_csv('Resources/Falcons_Raw/falcons_sportsref_download2014.csv')
falcons_df2015 = pd.read_csv('Resources/Falcons_Raw/falcons_sportsref_download2015.csv')
falcons_df2016 = pd.read_csv('Resources/Falcons_Raw/falcons_sportsref_download2016.csv')
falcons_df2017 = pd.read_csv('Resources/Falcons_Raw/falcons_sportsref_download2017.csv')
falcons_df2018 = pd.read_csv('Resources/Falcons_Raw/falcons_sportsref_download2018.csv')
falcons_df2019 = pd.read_csv('Resources/Falcons_Raw/falcons_sportsref_download2019.csv')
falcons_2020 = pd.read_csv('Resources/2020data/falcons_sportsref_download2020.csv')

falcons_df2010 = yearly_data(df=falcons_df2010, year=2010, 
                        HC='Mike Smith', 
                        OC='Mike Mularkey', 
                        DC='Brian VanGorder', 
                        QB='Matt Ryan')

falcons_df2011 = yearly_data(df=falcons_df2011, year=2011, 
                          HC='Mike Smith', 
                          OC='Mike Mularkey', 
                          DC='Brian VanGorder', 
                          QB='Matt Ryan')

falcons_df2012 = yearly_data(df=falcons_df2012, year=2012, 
                        HC='Mike Smith', 
                        OC='Dirk Koetter', 
                        DC='Mike Nolan',
                        QB='Matt Ryan')

falcons_df2013 = yearly_data(df=falcons_df2013, year=2013, 
                        HC='Mike Smith', 
                        OC='Dirk Koetter', 
                        DC='Mike Nolan', 
                        QB='Matt Ryan')

falcons_df2014 = yearly_data(df=falcons_df2014, year=2014, 
                        HC='Mike Smith', 
                        OC='Dirk Koetter', 
                        DC='Mike Nolan', 
                        QB='Matt Ryan')

falcons_df2015 = yearly_data(df=falcons_df2015, year=2015, 
                        HC='Dan Quinn', 
                        OC='Kyle Shanahan', 
                        DC='Richard Smith', 
                        QB='Matt Ryan')

falcons_df2016 = yearly_data(df=falcons_df2016, year=2016, 
                        HC='Dan Quinn',
                        OC='Kyle Shanahan', 
                        DC="Richard Smith", 
                        QB='Matt Ryan')

falcons_df2017 = yearly_data(df=falcons_df2017, year=2017, 
                        HC='Dan Quinn', 
                        OC='Steve Sarkisian', 
                        DC='Marquand Manuel', 
                        QB='Matt Ryan')

falcons_df2018 = yearly_data(df=falcons_df2018, year=2018, 
                        HC='Dan Quinn', 
                        OC='Steve Sarkisian', 
                        DC='Marquand Manuel', 
                        QB='Matt Ryan')

falcons_df2019 = yearly_data(df=falcons_df2019, year=2019, 
                        HC='Dan Quinn', 
                        OC='Dirk Koetter', 
                        DC='Dan Quinn', 
                        QB='Matt Ryan', 
                        QB2='Matt Schaub', QB2_range=8)

falcons_2020 = yearly_data(df=falcons_2020, year=2020, 
                        HC='Dan Quinn', 
                        OC='Dirk Koetter', 
                        DC='Raheem Morris', 
                        QB='Matt Ryan')

# Make master dataframe for team
falcons_df = falcons_2020.append(falcons_df2019, ignore_index=True)
falcons_df = falcons_df.append(falcons_df2018, ignore_index=True)
falcons_df = falcons_df.append(falcons_df2017, ignore_index=True)
falcons_df = falcons_df.append(falcons_df2016, ignore_index=True)
falcons_df = falcons_df.append(falcons_df2015, ignore_index=True)
falcons_df = falcons_df.append(falcons_df2014, ignore_index=True)
falcons_df = falcons_df.append(falcons_df2013, ignore_index=True)
falcons_df = falcons_df.append(falcons_df2012, ignore_index=True)
falcons_df = falcons_df.append(falcons_df2011, ignore_index=True)
falcons_df = falcons_df.append(falcons_df2010, ignore_index=True)

falcons_df['Base_Team'] = 'Falcons'
falcons_df = falcons_df.sort_values(by=['year', 'Week'], ascending=[False, True])

falcons_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Sun,0.0,0.0,1.0,Seattle Seahawks,2020,Falcons,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Dan Quinn,Dirk Koetter,Raheem Morris,Matt Ryan
1,2,Sun,0.0,0.0,0.0,Dallas Cowboys,2020,Falcons,25.0,38.0,...,0.0,299.0,84.0,506.0,383.0,0,Dan Quinn,Dirk Koetter,Raheem Morris,Matt Ryan
2,3,Sun,0.0,0.0,1.0,Chicago Bears,2020,Falcons,64.0,78.0,...,3.0,744.0,209.0,886.0,953.0,0,Dan Quinn,Dirk Koetter,Raheem Morris,Matt Ryan
3,4,Mon,0.0,0.0,0.0,Green Bay Packers,2020,Falcons,90.0,108.0,...,5.0,1051.0,339.0,1257.0,1390.0,0,Dan Quinn,Dirk Koetter,Raheem Morris,Matt Ryan
4,5,Sun,NaN,NaN,1.0,Carolina Panthers,2020,Falcons,106.0,138.0,...,5.0,1366.0,427.0,1584.0,1793.0,0,Dan Quinn,Dirk Koetter,Raheem Morris,Matt Ryan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,13,Sun,1.0,9.0,0.0,Tampa Bay Buccaneers,2010,Falcons,276.0,209.0,...,21.0,2782.0,1031.0,4008.0,3813.0,0,Mike Smith,Mike Mularkey,Brian VanGorder,Matt Ryan
172,14,Sun,1.0,10.0,0.0,Carolina Panthers,2010,Falcons,304.0,233.0,...,22.0,2956.0,1182.0,4298.0,4138.0,0,Mike Smith,Mike Mularkey,Brian VanGorder,Matt Ryan
173,15,Sun,1.0,11.0,0.0,Seattle Seahawks,2010,Falcons,335.0,243.0,...,24.0,3032.0,1394.0,4625.0,4426.0,0,Mike Smith,Mike Mularkey,Brian VanGorder,Matt Ryan
174,16,Mon,0.0,12.0,1.0,New Orleans Saints,2010,Falcons,369.0,261.0,...,27.0,3175.0,1485.0,4891.0,4660.0,0,Mike Smith,Mike Mularkey,Brian VanGorder,Matt Ryan


In [82]:
falcons_df.to_csv('Resources/indv_team_csvs/falcons.csv')

### (13) Washington Data, 2010-2019

In [83]:
# Data Source: https://www.pro-football-reference.com/teams/was/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

wash_df2010 = pd.read_csv('Resources/Wash_Raw/wash_sportsref_download2010.csv')
wash_df2011 = pd.read_csv('Resources/Wash_Raw/wash_sportsref_download2011.csv')
wash_df2012 = pd.read_csv('Resources/Wash_Raw/wash_sportsref_download2012.csv')
wash_df2013 = pd.read_csv('Resources/Wash_Raw/wash_sportsref_download2013.csv')
wash_df2014 = pd.read_csv('Resources/Wash_Raw/wash_sportsref_download2014.csv')
wash_df2015 = pd.read_csv('Resources/Wash_Raw/wash_sportsref_download2015.csv')
wash_df2016 = pd.read_csv('Resources/Wash_Raw/wash_sportsref_download2016.csv')
wash_df2017 = pd.read_csv('Resources/Wash_Raw/wash_sportsref_download2017.csv')
wash_df2018 = pd.read_csv('Resources/Wash_Raw/wash_sportsref_download2018.csv')
wash_df2019 = pd.read_csv('Resources/Wash_Raw/wash_sportsref_download2019.csv')
wash_2020 = pd.read_csv('Resources/2020data/wash_sportsref_download2020.csv')

wash_df2010 = yearly_data(df=wash_df2010, year=2010, 
                        HC='Mike Shanahan', 
                        OC='Kyle Shanahan', 
                        DC='Jim Haslett', 
                        QB='Donovan McNabb', 
                        QB2='Rex Grossman', QB2_range=range(15,18,1))

wash_df2011 = yearly_data(df=wash_df2011, year=2011, 
                        HC='Mike Shanahan', 
                        OC='Kyle Shanahan', 
                        DC='Jim Haslett', 
                        QB='Rex Grossman', 
                        QB2='John Beck', QB2_range=range(7,10,1))

wash_df2012 = yearly_data(df=wash_df2012, year=2012, 
                        HC='Mike Shanahan', 
                        OC='Kyle Shanahan', 
                        DC='Jim Haslett',
                        QB='Robert Griffin III', 
                        QB2='Kirk Cousins', QB2_range=15)

wash_df2013 = yearly_data(df=wash_df2013, year=2013, 
                        HC='Mike Shanahan', 
                        OC='Kyle Shanahan', 
                        DC='Jim Haslett', 
                        QB='Robert Griffin III', 
                        QB2='Kirk Cousins', QB2_range=range(15,18,1))

wash_df2014 = yearly_data(df=wash_df2014, year=2014, 
                        HC='Jay Gruden', 
                        OC='Sean McVay', 
                        DC='Jim Haslett', 
                        QB='Robert Griffin III', 
                        QB2='Kirk Cousins', QB2_range=range(3,8,1))
wash_df2014.loc[wash_df2014['Week'] == 8, 'QB'] = "Colt McCoy"
wash_df2014.loc[wash_df2014['Week'] == range(13,16,1), 'QB'] = "Colt McCoy"

wash_df2015 = yearly_data(df=wash_df2015, year=2015, 
                        HC='Jay Gruden', 
                        OC='Sean McVay', 
                        DC='Joe Barry', 
                        QB='Kirk Cousins')

wash_df2016 = yearly_data(df=wash_df2016, year=2016, 
                        HC='Jay Gruden', 
                        OC='Sean McVay', 
                        DC='Joe Barry', 
                        QB='Kirk Cousins')

wash_df2017 = yearly_data(df=wash_df2017, year=2017, 
                        HC='Jay Gruden', 
                        OC='Matt Cavanaugh', 
                        DC='Greg Manusky', 
                        QB='Kirk Cousins')

wash_df2018 = yearly_data(df=wash_df2018, year=2018, 
                        HC='Jay Gruden', 
                        OC='Matt Cavanaugh', 
                        DC='Greg Manusky', 
                        QB='Alex Smith', 
                        QB2='Colt McCoy', QB2_range=range(12,14,1))
wash_df2018.loc[wash_df2018['Week'] == range(15,18,1), 'QB'] = "Josh Johnson"
wash_df2018.loc[wash_df2018['Week'] == 14, 'QB'] = "Mark Sanchez"

wash_df2019 = yearly_data(df=wash_df2019, year=2019, 
                        HC='Jay Gruden', HC2='Bill Callahan', HC2_range=range(6,18,1), 
                        OC="Kevin O'Connell", 
                        DC='Greg Manusky', 
                        QB='Case Keenum', 
                        QB2='Dwayne Haskins', QB2_range=range(9,17,1))
wash_df2019.loc[wash_df2019['Week'] == 5, 'QB'] = "Colt McCoy"

wash_2020 = yearly_data(df=wash_2020, year=2020, 
                        HC='Ron Rivera', 
                        OC='Scott Turner', 
                        DC='Jack Del Rio', 
                        QB='Dwayne Haskins', 
                        QB2='Kyle Allen', QB2_range=5)

# Make master dataframe for team
wash_df = wash_2020.append(wash_df2019, ignore_index=True)
wash_df = wash_df.append(wash_df2018, ignore_index=True)
wash_df = wash_df.append(wash_df2017, ignore_index=True)
wash_df = wash_df.append(wash_df2016, ignore_index=True)
wash_df = wash_df.append(wash_df2015, ignore_index=True)
wash_df = wash_df.append(wash_df2014, ignore_index=True)
wash_df = wash_df.append(wash_df2013, ignore_index=True)
wash_df = wash_df.append(wash_df2012, ignore_index=True)
wash_df = wash_df.append(wash_df2011, ignore_index=True)
wash_df = wash_df.append(wash_df2010, ignore_index=True)

wash_df['Base_Team'] = 'Washington'
wash_df = wash_df.sort_values(by=['year', 'Week'], ascending=[False, True])

wash_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Sun,1.0,0.0,1.0,Philadelphia Eagles,2020,Washington,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Ron Rivera,Scott Turner,Jack Del Rio,Dwayne Haskins
1,2,Sun,0.0,1.0,0.0,Arizona Cardinals,2020,Washington,27.0,17.0,...,3.0,208.0,57.0,239.0,265.0,0,Ron Rivera,Scott Turner,Jack Del Rio,Dwayne Haskins
2,3,Sun,0.0,1.0,0.0,Cleveland Browns,2020,Washington,42.0,47.0,...,4.0,486.0,217.0,555.0,703.0,0,Ron Rivera,Scott Turner,Jack Del Rio,Dwayne Haskins
3,4,Sun,0.0,1.0,1.0,Baltimore Ravens,2020,Washington,62.0,81.0,...,4.0,628.0,375.0,864.0,1003.0,0,Ron Rivera,Scott Turner,Jack Del Rio,Dwayne Haskins
4,5,Sun,NaN,NaN,1.0,Los Angeles Rams,2020,Washington,79.0,112.0,...,6.0,834.0,519.0,1207.0,1353.0,0,Ron Rivera,Scott Turner,Jack Del Rio,Kyle Allen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,13,Sun,0.0,5.0,0.0,New York Giants,2010,Washington,215.0,262.0,...,21.0,2963.0,1447.0,3660.0,4410.0,0,Mike Shanahan,Kyle Shanahan,Jim Haslett,Donovan McNabb
172,14,Sun,0.0,5.0,1.0,Tampa Bay Buccaneers,2010,Washington,222.0,293.0,...,22.0,3124.0,1644.0,3998.0,4768.0,0,Mike Shanahan,Kyle Shanahan,Jim Haslett,Donovan McNabb
173,15,Sun,0.0,5.0,0.0,Dallas Cowboys,2010,Washington,238.0,310.0,...,24.0,3386.0,1747.0,4397.0,5133.0,0,Mike Shanahan,Kyle Shanahan,Jim Haslett,Donovan McNabb
174,16,Sun,1.0,5.0,0.0,Jacksonville Jaguars,2010,Washington,268.0,343.0,...,24.0,3686.0,1881.0,4738.0,5567.0,0,Mike Shanahan,Kyle Shanahan,Jim Haslett,Donovan McNabb


In [84]:
wash_df.to_csv('Resources/indv_team_csvs/wash.csv')

### (14) Ravens Data, 2010-2019

In [85]:
# Data Source: https://www.pro-football-reference.com/teams/rav/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

ravens_df2010 = pd.read_csv('Resources/Ravens_Raw/ravens_sportsref_download2010.csv')
ravens_df2011 = pd.read_csv('Resources/Ravens_Raw/ravens_sportsref_download2011.csv')
ravens_df2012 = pd.read_csv('Resources/Ravens_Raw/ravens_sportsref_download2012.csv')
ravens_df2013 = pd.read_csv('Resources/Ravens_Raw/ravens_sportsref_download2013.csv')
ravens_df2014 = pd.read_csv('Resources/Ravens_Raw/ravens_sportsref_download2014.csv')
ravens_df2015 = pd.read_csv('Resources/Ravens_Raw/ravens_sportsref_download2015.csv')
ravens_df2016 = pd.read_csv('Resources/Ravens_Raw/ravens_sportsref_download2016.csv')
ravens_df2017 = pd.read_csv('Resources/Ravens_Raw/ravens_sportsref_download2017.csv')
ravens_df2018 = pd.read_csv('Resources/Ravens_Raw/ravens_sportsref_download2018.csv')
ravens_df2019 = pd.read_csv('Resources/Ravens_Raw/ravens_sportsref_download2019.csv')
ravens_2020 = pd.read_csv('Resources/2020data/ravens_sportsref_download2020.csv')

ravens_df2010 = yearly_data(df=ravens_df2010, year=2010, 
                        HC='John Harbaugh', 
                        OC='Cam Cameron', 
                        DC='Greg Mattison', 
                        QB='Joe Flacco')

ravens_df2011 = yearly_data(df=ravens_df2011, year=2011, 
                          HC='John Harbaugh', 
                          OC='Cam Cameron', 
                          DC='Chuck Pagano', 
                          QB='Joe Flacco')

ravens_df2012 = yearly_data(df=ravens_df2012, year=2012, 
                        HC='John Harbaugh', 
                        OC='Cam Cameron', 
                        DC='Dean Pees',
                        QB='Joe Flacco')
ravens_df2012.loc[ravens_df2012['Week'] == range(15,18,1), 'OC'] = "Jim Caldwell"

ravens_df2013 = yearly_data(df=ravens_df2013, year=2013, 
                        HC='John Harbaugh', 
                        OC='Jim Caldwell', 
                        DC='Dean Pees', 
                        QB='Joe Flacco')

ravens_df2014 = yearly_data(df=ravens_df2014, year=2014, 
                        HC='John Harbaugh', 
                        OC='Gary Kubiak', 
                        DC='Dean Pees', 
                        QB='Joe Flacco')

ravens_df2015 = yearly_data(df=ravens_df2015, year=2015, 
                        HC='John Harbaugh', 
                        OC='Marc Trestman', 
                        DC='Dean Pees', 
                        QB='Joe Flacco', 
                        QB2='Matt Schaub', QB2_range=range(12,14,1))
ravens_df2015.loc[ravens_df2015['Week'] == range(14,16,1), 'QB'] = "Jimmy Clausen"
ravens_df2015.loc[ravens_df2015['Week'] == range(16,18,1), 'QB'] = "Ryan Mallett"

ravens_df2016 = yearly_data(df=ravens_df2016, year=2016, 
                        HC='John Harbaugh',
                        OC='Marc Trestman', 
                        DC="Dean Pees", 
                        QB='Joe Flacco')
ravens_df2016.loc[ravens_df2016['Week'] == range(6,18,1), 'OC'] = "Marty Mornhinweg"

ravens_df2017 = yearly_data(df=ravens_df2017, year=2017, 
                        HC='John Harbaugh', 
                        OC='Marty Mornhinweg', 
                        DC='Dean Pees', 
                        QB='Joe Flacco')

ravens_df2018 = yearly_data(df=ravens_df2018, year=2018, 
                        HC='John Harbaugh', 
                        OC='Marty Mornhinweg', 
                        DC='Don Martindale', 
                        QB='Joe Flacco', 
                        QB2='Lamar Jackson', QB2_range=range(11,18,1))

ravens_df2019 = yearly_data(df=ravens_df2019, year=2019, 
                        HC='John Harbaugh', 
                        OC='Greg Roman', 
                        DC='Don Martindale', 
                        QB='Lamar Jackson', 
                        QB2='Robert Griffin III', QB2_range=17)

ravens_2020 = yearly_data(df=ravens_2020, year=2020, 
                        HC='John Harbaugh', 
                        OC='Greg Roman', 
                        DC='Don Martindale', 
                        QB='Lamar Jackson')

# Make master dataframe for team
ravens_df = ravens_2020.append(ravens_df2019, ignore_index=True)
ravens_df = ravens_df.append(ravens_df2018, ignore_index=True)
ravens_df = ravens_df.append(ravens_df2017, ignore_index=True)
ravens_df = ravens_df.append(ravens_df2016, ignore_index=True)
ravens_df = ravens_df.append(ravens_df2015, ignore_index=True)
ravens_df = ravens_df.append(ravens_df2014, ignore_index=True)
ravens_df = ravens_df.append(ravens_df2013, ignore_index=True)
ravens_df = ravens_df.append(ravens_df2012, ignore_index=True)
ravens_df = ravens_df.append(ravens_df2011, ignore_index=True)
ravens_df = ravens_df.append(ravens_df2010, ignore_index=True)

ravens_df['Base_Team'] = 'Ravens'
ravens_df = ravens_df.sort_values(by=['year', 'Week'], ascending=[False, True])

ravens_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Sun,1.0,0.0,1.0,Cleveland Browns,2020,Ravens,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,John Harbaugh,Greg Roman,Don Martindale,Lamar Jackson
1,2,Sun,1.0,1.0,0.0,Houston Texans,2020,Ravens,38.0,6.0,...,3.0,168.0,138.0,381.0,306.0,0,John Harbaugh,Greg Roman,Don Martindale,Lamar Jackson
2,3,Mon,0.0,2.0,1.0,Kansas City Chiefs,2020,Ravens,71.0,22.0,...,5.0,421.0,189.0,788.0,610.0,0,John Harbaugh,Greg Roman,Don Martindale,Lamar Jackson
3,4,Sun,1.0,2.0,0.0,Washington Football Team,2020,Ravens,91.0,56.0,...,6.0,806.0,321.0,1016.0,1127.0,0,John Harbaugh,Greg Roman,Don Martindale,Lamar Jackson
4,5,Sun,NaN,NaN,1.0,Cincinnati Bengals,2020,Ravens,122.0,73.0,...,7.0,1080.0,390.0,1366.0,1470.0,0,John Harbaugh,Greg Roman,Don Martindale,Lamar Jackson
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,13,Sun,0.0,8.0,1.0,Pittsburgh Steelers,2010,Ravens,250.0,188.0,...,14.0,2262.0,1119.0,3831.0,3381.0,0,John Harbaugh,Cam Cameron,Greg Mattison,Joe Flacco
172,14,Mon,1.0,8.0,0.0,Houston Texans,2010,Ravens,260.0,201.0,...,15.0,2496.0,1173.0,4100.0,3669.0,0,John Harbaugh,Cam Cameron,Greg Mattison,Joe Flacco
173,15,Sun,1.0,9.0,1.0,New Orleans Saints,2010,Ravens,294.0,229.0,...,17.0,2874.0,1284.0,4353.0,4158.0,0,John Harbaugh,Cam Cameron,Greg Mattison,Joe Flacco
174,16,Sun,1.0,10.0,0.0,Cleveland Browns,2010,Ravens,324.0,253.0,...,18.0,3116.0,1311.0,4709.0,4427.0,0,John Harbaugh,Cam Cameron,Greg Mattison,Joe Flacco


In [86]:
ravens_df.to_csv('Resources/indv_team_csvs/ravens.csv')

### (15) Eagles Data, 2010-2019

In [87]:
# Data Source: https://www.pro-football-reference.com/teams/phi/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

eagles_df2010 = pd.read_csv('Resources/Eagles_Raw/eagles_sportsref_download2010.csv')
eagles_df2011 = pd.read_csv('Resources/Eagles_Raw/eagles_sportsref_download2011.csv')
eagles_df2012 = pd.read_csv('Resources/Eagles_Raw/eagles_sportsref_download2012.csv')
eagles_df2013 = pd.read_csv('Resources/Eagles_Raw/eagles_sportsref_download2013.csv')
eagles_df2014 = pd.read_csv('Resources/Eagles_Raw/eagles_sportsref_download2014.csv')
eagles_df2015 = pd.read_csv('Resources/Eagles_Raw/eagles_sportsref_download2015.csv')
eagles_df2016 = pd.read_csv('Resources/Eagles_Raw/eagles_sportsref_download2016.csv')
eagles_df2017 = pd.read_csv('Resources/Eagles_Raw/eagles_sportsref_download2017.csv')
eagles_df2018 = pd.read_csv('Resources/Eagles_Raw/eagles_sportsref_download2018.csv')
eagles_df2019 = pd.read_csv('Resources/Eagles_Raw/eagles_sportsref_download2019.csv')
eagles_2020 = pd.read_csv('Resources/2020data/eagles_sportsref_download2020.csv')

eagles_df2010 = yearly_data(df=eagles_df2010, year=2010, 
                        HC='Andy Reid', 
                        OC='Marty Mornhinweg', 
                        DC='Sean McDermott', 
                        QB='Michael Vick', 
                        QB2='Kevin Kolb', QB2_range=range(4,7,1))
eagles_df2010.loc[eagles_df2010['Week'] == range(1,18,16), 'QB'] = "Kevin Kolb"

eagles_df2011 = yearly_data(df=eagles_df2011, year=2011, 
                        HC='Andy Reid', 
                        OC='Marty Mornhinweg', 
                        DC='Juan Castillo', 
                        QB='Michael Vick', 
                        QB2='Vince Young', QB2_range=range(11,14,1))

eagles_df2012 = yearly_data(df=eagles_df2012, year=2012, 
                        HC='Andy Reid', 
                        OC='Marty Mornhinweg', 
                        DC='Juan Castillo',
                        QB='Michael Vick', 
                        QB2='Nick Foles', QB2_range=range(11,17,1))
eagles_df2012.loc[eagles_df2012['Week'] == range(8,18,1), 'DC'] = "Todd Bowles"

eagles_df2013 = yearly_data(df=eagles_df2013, year=2013, 
                        HC='Chip Kelly', 
                        OC='Pat Shurmur', 
                        DC='Bill Davis', 
                        QB='Nick Foles', 
                        QB2='Michael Vick', QB2_range=range(1,6,1))
eagles_df2013.loc[eagles_df2013['Week'] == 8, 'QB'] = "Michael Vick"

eagles_df2014 = yearly_data(df=eagles_df2014, year=2014, 
                        HC='Chip Kelly', 
                        OC='Pat Shurmur', 
                        DC='Bill Davis', 
                        QB='Nick Foles', 
                        QB2='Mark Sanchez', QB2_range=range(10,18,1))

eagles_df2015 = yearly_data(df=eagles_df2015, year=2015, 
                        HC='Chip Kelly', HC2='Pat Shurmur', HC2_range=17, 
                        OC='Pat Shurmur', 
                        DC='Bill Davis', 
                        QB='Sam Bradford', 
                        QB2='Mark Sanchez', QB2_range=range(11,13,1))

eagles_df2016 = yearly_data(df=eagles_df2016, year=2016, 
                        HC='Doug Pederson',
                        OC='Frank Reich', 
                        DC="Jim Schwartz", 
                        QB='Carson Wentz')

eagles_df2017 = yearly_data(df=eagles_df2017, year=2017, 
                        HC='Doug Pederson',
                        OC='Frank Reich', 
                        DC="Jim Schwartz", 
                        QB='Carson Wentz', 
                        QB2='Nick Foles', QB2_range=range(15,18,1))

eagles_df2018 = yearly_data(df=eagles_df2018, year=2018, 
                        HC='Doug Pederson',
                        OC='Mike Groh', 
                        DC="Jim Schwartz", 
                        QB='Carson Wentz', 
                        QB2='Nick Foles', QB2_range=range(15,18,1))
eagles_df2018.loc[eagles_df2018['Week'] == range(1,3,1), 'QB'] = "Nick Foles"

eagles_df2019 = yearly_data(df=eagles_df2019, year=2019, 
                        HC='Doug Pederson',
                        OC='Mike Groh', 
                        DC="Jim Schwartz", 
                        QB='Carson Wentz')

eagles_2020 = yearly_data(df=eagles_2020, year=2020, 
                        HC='Doug Pederson', 
                        OC='Doug Pederson', 
                        DC='Jim Schwartz', 
                        QB='Carson Wentz')

# Make master dataframe for team
eagles_df = eagles_2020.append(eagles_df2019, ignore_index=True)
eagles_df = eagles_df.append(eagles_df2018, ignore_index=True)
eagles_df = eagles_df.append(eagles_df2017, ignore_index=True)
eagles_df = eagles_df.append(eagles_df2016, ignore_index=True)
eagles_df = eagles_df.append(eagles_df2015, ignore_index=True)
eagles_df = eagles_df.append(eagles_df2014, ignore_index=True)
eagles_df = eagles_df.append(eagles_df2013, ignore_index=True)
eagles_df = eagles_df.append(eagles_df2012, ignore_index=True)
eagles_df = eagles_df.append(eagles_df2011, ignore_index=True)
eagles_df = eagles_df.append(eagles_df2010, ignore_index=True)

eagles_df['Base_Team'] = 'Eagles'
eagles_df = eagles_df.sort_values(by=['year', 'Week'], ascending=[False, True])

eagles_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Sun,0.0,0.0,0.0,Washington Football Team,2020,Eagles,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Doug Pederson,Doug Pederson,Jim Schwartz,Carson Wentz
1,2,Sun,0.0,0.0,1.0,Los Angeles Rams,2020,Eagles,17.0,27.0,...,0.0,159.0,80.0,265.0,239.0,0,Doug Pederson,Doug Pederson,Jim Schwartz,Carson Wentz
2,3,Sun,0.0,0.0,1.0,Cincinnati Bengals,2020,Eagles,36.0,64.0,...,1.0,417.0,271.0,628.0,688.0,0,Doug Pederson,Doug Pederson,Jim Schwartz,Carson Wentz
3,4,Sun,1.0,0.0,0.0,San Francisco 49ers,2020,Eagles,59.0,87.0,...,1.0,673.0,319.0,1009.0,992.0,0,Doug Pederson,Doug Pederson,Jim Schwartz,Carson Wentz
4,5,Sun,NaN,NaN,0.0,Pittsburgh Steelers,2020,Eagles,84.0,107.0,...,4.0,974.0,435.0,1276.0,1409.0,0,Doug Pederson,Doug Pederson,Jim Schwartz,Carson Wentz
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,13,Thu,1.0,7.0,1.0,Houston Texans,2010,Eagles,310.0,257.0,...,26.0,2354.0,1128.0,4392.0,3482.0,0,Andy Reid,Marty Mornhinweg,Sean McDermott,Michael Vick
172,14,Sun,1.0,8.0,0.0,Dallas Cowboys,2010,Eagles,344.0,281.0,...,28.0,2677.0,1236.0,4808.0,3913.0,0,Andy Reid,Marty Mornhinweg,Sean McDermott,Michael Vick
173,15,Sun,1.0,9.0,0.0,New York Giants,2010,Eagles,374.0,308.0,...,30.0,2916.0,1346.0,5237.0,4262.0,0,Andy Reid,Marty Mornhinweg,Sean McDermott,Michael Vick
174,16,Tue,0.0,10.0,1.0,Minnesota Vikings,2010,Eagles,412.0,339.0,...,32.0,3180.0,1446.0,5655.0,4626.0,0,Andy Reid,Marty Mornhinweg,Sean McDermott,Michael Vick


In [88]:
eagles_df.to_csv('Resources/indv_team_csvs/eagles.csv')

### (16) Cowboys Data, 2010-2019

In [89]:
# Data Source: https://www.pro-football-reference.com/teams/dal/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

cowboys_df2010 = pd.read_csv('Resources/Cowboys_Raw/cowboys_sportsref_download2010.csv')
cowboys_df2011 = pd.read_csv('Resources/Cowboys_Raw/cowboys_sportsref_download2011.csv')
cowboys_df2012 = pd.read_csv('Resources/Cowboys_Raw/cowboys_sportsref_download2012.csv')
cowboys_df2013 = pd.read_csv('Resources/Cowboys_Raw/cowboys_sportsref_download2013.csv')
cowboys_df2014 = pd.read_csv('Resources/Cowboys_Raw/cowboys_sportsref_download2014.csv')
cowboys_df2015 = pd.read_csv('Resources/Cowboys_Raw/cowboys_sportsref_download2015.csv')
cowboys_df2016 = pd.read_csv('Resources/Cowboys_Raw/cowboys_sportsref_download2016.csv')
cowboys_df2017 = pd.read_csv('Resources/Cowboys_Raw/cowboys_sportsref_download2017.csv')
cowboys_df2018 = pd.read_csv('Resources/Cowboys_Raw/cowboys_sportsref_download2018.csv')
cowboys_df2019 = pd.read_csv('Resources/Cowboys_Raw/cowboys_sportsref_download2019.csv')
cowboys_2020 = pd.read_csv('Resources/2020data/cowboys_sportsref_download2020.csv')

cowboys_df2010 = yearly_data(df=cowboys_df2010, year=2010, 
                        HC='Jason Garrett', HC2='Wade Phillips', HC2_range=range(1,10,1), 
                        OC='Jason Garrett', 
                        DC='Paul Pasqualoni', 
                        QB='Jon Kitna', 
                        QB2='Tony Romo', QB2_range=range(1,8,1))
cowboys_df2010.loc[cowboys_df2010['Week'] == 17, 'QB'] = "Stephen McGee"

cowboys_df2011 = yearly_data(df=cowboys_df2011, year=2011, 
                          HC='Jason Garrett', 
                          OC='Jason Garrett', 
                          DC='Rob Ryan', 
                          QB='Tony Romo')

cowboys_df2012 = yearly_data(df=cowboys_df2012, year=2012, 
                        HC='Jason Garrett', 
                        OC='Bill Callahan', 
                        DC='Rob Ryan',
                        QB='Tony Romo')

cowboys_df2013 = yearly_data(df=cowboys_df2013, year=2013, 
                        HC='Jason Garrett', 
                        OC='Bill Callahan', 
                        DC='Monte Kiffin', 
                        QB='Tony Romo',
                        QB2='Kyle Orton', QB2_range=17)

cowboys_df2014 = yearly_data(df=cowboys_df2014, year=2014, 
                        HC='Jason Garrett', 
                        OC='Bill Callahan', 
                        DC='Rod Marinelli', 
                        QB='Tony Romo', 
                        QB2='Brandon Weeden', QB2_range=9)

cowboys_df2015 = yearly_data(df=cowboys_df2015, year=2015, 
                        HC='Jason Garrett', 
                        OC='Scott Linehan', 
                        DC='Rod Marinelli', 
                        QB='Matt Cassel', 
                        QB2='Tony Romo', QB2_range=range(1,3,1))
cowboys_df2015.loc[cowboys_df2015['Week'] == range(3,6,1), 'QB'] = "Brandon Weeden"
cowboys_df2015.loc[cowboys_df2015['Week'] == range(16,18,1), 'QB'] = "Kellen Moore"
cowboys_df2015.loc[cowboys_df2015['Week'] == range(11,13,1), 'QB'] = "Tony Romo"

cowboys_df2016 = yearly_data(df=cowboys_df2016, year=2016, 
                        HC='Jason Garrett',
                        OC='Scott Linehan', 
                        DC="Rod Marinelli", 
                        QB='Dak Prescott')

cowboys_df2017 = yearly_data(df=cowboys_df2017, year=2017, 
                        HC='Jason Garrett', 
                        OC='Scott Linehan', 
                        DC="Rod Marinelli", 
                        QB='Dak Prescott')

cowboys_df2018 = yearly_data(df=cowboys_df2018, year=2018, 
                        HC='Jason Garrett', 
                        OC='Scott Linehan', 
                        DC="Rod Marinelli", 
                        QB='Dak Prescott')

cowboys_df2019 = yearly_data(df=cowboys_df2019, year=2019, 
                        HC='Jason Garrett', 
                        OC='Kellen Moore', 
                        DC='Rod Marinelli', 
                        QB='Dak Prescott')

cowboys_2020 = yearly_data(df=cowboys_2020, year=2020, 
                        HC='Mike McCarthy',
                        OC='Kellen Moore', 
                        DC='Mike Nolan', 
                        QB='Dak Prescott')

# Make master dataframe for team
cowboys_df = cowboys_2020.append(cowboys_df2019, ignore_index=True)
cowboys_df = cowboys_df.append(cowboys_df2018, ignore_index=True)
cowboys_df = cowboys_df.append(cowboys_df2017, ignore_index=True)
cowboys_df = cowboys_df.append(cowboys_df2016, ignore_index=True)
cowboys_df = cowboys_df.append(cowboys_df2015, ignore_index=True)
cowboys_df = cowboys_df.append(cowboys_df2014, ignore_index=True)
cowboys_df = cowboys_df.append(cowboys_df2013, ignore_index=True)
cowboys_df = cowboys_df.append(cowboys_df2012, ignore_index=True)
cowboys_df = cowboys_df.append(cowboys_df2011, ignore_index=True)
cowboys_df = cowboys_df.append(cowboys_df2010, ignore_index=True)

cowboys_df['Base_Team'] = 'Cowboys'
cowboys_df = cowboys_df.sort_values(by=['year', 'Week'], ascending=[False, True])

cowboys_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Sun,0.0,0.0,0.0,Los Angeles Rams,2020,Cowboys,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Mike McCarthy,Kellen Moore,Mike Nolan,Dak Prescott
1,2,Sun,1.0,0.0,1.0,Atlanta Falcons,2020,Cowboys,17.0,20.0,...,1.0,269.0,153.0,380.0,422.0,0,Mike McCarthy,Kellen Moore,Mike Nolan,Dak Prescott
2,3,Sun,0.0,1.0,0.0,Seattle Seahawks,2020,Cowboys,57.0,59.0,...,1.0,536.0,266.0,950.0,802.0,0,Mike McCarthy,Kellen Moore,Mike Nolan,Dak Prescott
3,4,Sun,0.0,1.0,1.0,Cleveland Browns,2020,Cowboys,88.0,97.0,...,2.0,831.0,383.0,1472.0,1214.0,0,Mike McCarthy,Kellen Moore,Mike Nolan,Dak Prescott
4,5,Sun,NaN,NaN,1.0,New York Giants,2020,Cowboys,126.0,146.0,...,2.0,1032.0,690.0,2038.0,1722.0,0,Mike McCarthy,Kellen Moore,Mike Nolan,Dak Prescott
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,13,Sun,1.0,3.0,0.0,Indianapolis Colts,2010,Cowboys,256.0,301.0,...,17.0,2684.0,1254.0,4023.0,3938.0,0,Jason Garrett,Jason Garrett,Paul Pasqualoni,Jon Kitna
172,14,Sun,0.0,4.0,1.0,Philadelphia Eagles,2010,Cowboys,294.0,336.0,...,21.0,3049.0,1294.0,4391.0,4343.0,0,Jason Garrett,Jason Garrett,Paul Pasqualoni,Jon Kitna
173,15,Sun,1.0,4.0,1.0,Washington Redskins,2010,Cowboys,321.0,366.0,...,23.0,3307.0,1465.0,4740.0,4772.0,0,Jason Garrett,Jason Garrett,Paul Pasqualoni,Jon Kitna
174,16,Sat,0.0,5.0,0.0,Arizona Cardinals,2010,Cowboys,354.0,396.0,...,26.0,3593.0,1520.0,5174.0,5113.0,0,Jason Garrett,Jason Garrett,Paul Pasqualoni,Jon Kitna


In [90]:
cowboys_df.to_csv('Resources/indv_team_csvs/cowboys.csv')

### (17) Chiefs Data, 2010-2019

In [91]:
# Data Source: https://www.pro-football-reference.com/teams/kan/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

chiefs_df2010 = pd.read_csv('Resources/Chiefs_Raw/chiefs_sportsref_download2010.csv')
chiefs_df2011 = pd.read_csv('Resources/Chiefs_Raw/chiefs_sportsref_download2011.csv')
chiefs_df2012 = pd.read_csv('Resources/Chiefs_Raw/chiefs_sportsref_download2012.csv')
chiefs_df2013 = pd.read_csv('Resources/Chiefs_Raw/chiefs_sportsref_download2013.csv')
chiefs_df2014 = pd.read_csv('Resources/Chiefs_Raw/chiefs_sportsref_download2014.csv')
chiefs_df2015 = pd.read_csv('Resources/Chiefs_Raw/chiefs_sportsref_download2015.csv')
chiefs_df2016 = pd.read_csv('Resources/Chiefs_Raw/chiefs_sportsref_download2016.csv')
chiefs_df2017 = pd.read_csv('Resources/Chiefs_Raw/chiefs_sportsref_download2017.csv')
chiefs_df2018 = pd.read_csv('Resources/Chiefs_Raw/chiefs_sportsref_download2018.csv')
chiefs_df2019 = pd.read_csv('Resources/Chiefs_Raw/chiefs_sportsref_download2019.csv')
chiefs_2020 = pd.read_csv('Resources/2020data/chiefs_sportsref_download2020.csv')

chiefs_df2010 = yearly_data(df=chiefs_df2010, year=2010, 
                        HC='Todd Haley', 
                        OC='Charlie Weis', 
                        DC='Romeo Crennel', 
                        QB='Matt Cassel', 
                        QB2='Brodie Croyle', QB2_range=14)

chiefs_df2011 = yearly_data(df=chiefs_df2011, year=2011, 
                        HC='Todd Haley', HC2='Romeo Crennel', HC2_range=range(15,18,1), 
                        OC='Bill Muir', 
                        DC='Romeo Crennel', 
                        QB='Matt Cassel', 
                        QB2='Tyler Palko', QB2_range=range(11,15,1))
chiefs_df2011.loc[chiefs_df2011['Week'] == range(15,18,1), 'QB'] = "Kyle Orton"

chiefs_df2012 = yearly_data(df=chiefs_df2012, year=2012, 
                        HC='Romeo Crennel', 
                        OC='Brian Daboll', 
                        DC='Romeo Crennel',
                        QB='Matt Cassel', 
                        QB2='Brady Quinn', QB2_range=range(6,9,1))
chiefs_df2012.loc[chiefs_df2012['Week'] == range(12,18,1), 'QB'] = "Brady Quinn"

chiefs_df2013 = yearly_data(df=chiefs_df2013, year=2013, 
                        HC='Andy Reid', 
                        OC='Doug Pederson', 
                        DC='Bob Sutton', 
                        QB='Alex Smith', 
                        QB2='Chase Daniel', QB2_range=17)

chiefs_df2014 = yearly_data(df=chiefs_df2014, year=2014, 
                        HC='Andy Reid', 
                        OC='Doug Pederson', 
                        DC='Bob Sutton', 
                        QB='Alex Smith', 
                        QB2='Chase Daniel', QB2_range=17)

chiefs_df2015 = yearly_data(df=chiefs_df2015, year=2015, 
                        HC='Andy Reid', 
                        OC='Doug Pederson', 
                        DC='Bob Sutton', 
                        QB='Alex Smith')

chiefs_df2016 = yearly_data(df=chiefs_df2016, year=2016, 
                        HC='Andy Reid', 
                        OC='Brad Childress', 
                        DC='Bob Sutton', 
                        QB='Alex Smith', 
                        QB2='Nick Foles', QB2_range=9)

chiefs_df2017 = yearly_data(df=chiefs_df2017, year=2017, 
                        HC='Andy Reid', 
                        OC='Matt Nagy', 
                        DC='Bob Sutton', 
                        QB='Alex Smith', 
                        QB2='Patrick Mahomes', QB2_range=17)

chiefs_df2018 = yearly_data(df=chiefs_df2018, year=2018, 
                        HC='Andy Reid', 
                        OC='Eric Bieniemy', 
                        DC='Bob Sutton', 
                        QB='Patrick Mahomes')

chiefs_df2019 = yearly_data(df=chiefs_df2019, year=2019, 
                        HC='Andy Reid', 
                        OC='Eric Bieniemy', 
                        DC='Steve Spagnuolo', 
                        QB='Patrick Mahomes', 
                        QB2='Matt Moore', QB2_range=range(8,10,1))

chiefs_2020 = yearly_data(df=chiefs_2020, year=2020, 
                        HC='Andy Reid', 
                        OC='Eric Bieniemy', 
                        DC='Steve Spagnuolo', 
                        QB='Patrick Mahomes')

# Make master dataframe for team
chiefs_df = chiefs_2020.append(chiefs_df2019, ignore_index=True)
chiefs_df = chiefs_df.append(chiefs_df2018, ignore_index=True)
chiefs_df = chiefs_df.append(chiefs_df2017, ignore_index=True)
chiefs_df = chiefs_df.append(chiefs_df2016, ignore_index=True)
chiefs_df = chiefs_df.append(chiefs_df2015, ignore_index=True)
chiefs_df = chiefs_df.append(chiefs_df2014, ignore_index=True)
chiefs_df = chiefs_df.append(chiefs_df2013, ignore_index=True)
chiefs_df = chiefs_df.append(chiefs_df2012, ignore_index=True)
chiefs_df = chiefs_df.append(chiefs_df2011, ignore_index=True)
chiefs_df = chiefs_df.append(chiefs_df2010, ignore_index=True)

chiefs_df['Base_Team'] = 'Chiefs'
chiefs_df = chiefs_df.sort_values(by=['year', 'Week'], ascending=[False, True])

chiefs_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Thu,1.0,0.0,1.0,Houston Texans,2020,Chiefs,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Andy Reid,Eric Bieniemy,Steve Spagnuolo,Patrick Mahomes
1,2,Sun,1.0,1.0,0.0,Los Angeles Chargers,2020,Chiefs,34.0,20.0,...,1.0,242.0,118.0,369.0,360.0,0,Andy Reid,Eric Bieniemy,Steve Spagnuolo,Patrick Mahomes
2,3,Mon,1.0,2.0,0.0,Baltimore Ravens,2020,Chiefs,57.0,40.0,...,2.0,538.0,301.0,783.0,839.0,0,Andy Reid,Eric Bieniemy,Steve Spagnuolo,Patrick Mahomes
3,4,Mon,1.0,3.0,1.0,New England Patriots,2020,Chiefs,91.0,60.0,...,3.0,608.0,459.0,1300.0,1067.0,0,Andy Reid,Eric Bieniemy,Steve Spagnuolo,Patrick Mahomes
4,5,Sun,NaN,NaN,1.0,Las Vegas Raiders,2020,Chiefs,117.0,70.0,...,7.0,780.0,644.0,1623.0,1424.0,0,Andy Reid,Eric Bieniemy,Steve Spagnuolo,Patrick Mahomes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,13,Sun,1.0,7.0,1.0,Denver Broncos,2010,Chiefs,285.0,231.0,...,15.0,2711.0,1061.0,4127.0,3772.0,0,Todd Haley,Charlie Weis,Romeo Crennel,Matt Cassel
172,14,Sun,0.0,8.0,0.0,San Diego Chargers,2010,Chiefs,295.0,237.0,...,16.0,2797.0,1222.0,4486.0,4019.0,0,Todd Haley,Charlie Weis,Romeo Crennel,Brodie Croyle
173,15,Sun,1.0,8.0,0.0,St. Louis Rams,2010,Chiefs,295.0,268.0,...,18.0,3016.0,1429.0,4553.0,4445.0,0,Todd Haley,Charlie Weis,Romeo Crennel,Matt Cassel
174,16,Sun,1.0,9.0,1.0,Tennessee Titans,2010,Chiefs,322.0,281.0,...,20.0,3171.0,1498.0,4936.0,4669.0,0,Todd Haley,Charlie Weis,Romeo Crennel,Matt Cassel


In [92]:
chiefs_df.to_csv('Resources/indv_team_csvs/chiefs.csv')

### Lions Data, 2010-2019

In [93]:
# Data Source: https://www.pro-football-reference.com/teams/det/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

lions_df2010 = pd.read_csv('Resources/Lions_Raw/lions_sportsref_download2010.csv')
lions_df2011 = pd.read_csv('Resources/Lions_Raw/lions_sportsref_download2011.csv')
lions_df2012 = pd.read_csv('Resources/Lions_Raw/lions_sportsref_download2012.csv')
lions_df2013 = pd.read_csv('Resources/Lions_Raw/lions_sportsref_download2013.csv')
lions_df2014 = pd.read_csv('Resources/Lions_Raw/lions_sportsref_download2014.csv')
lions_df2015 = pd.read_csv('Resources/Lions_Raw/lions_sportsref_download2015.csv')
lions_df2016 = pd.read_csv('Resources/Lions_Raw/lions_sportsref_download2016.csv')
lions_df2017 = pd.read_csv('Resources/Lions_Raw/lions_sportsref_download2017.csv')
lions_df2018 = pd.read_csv('Resources/Lions_Raw/lions_sportsref_download2018.csv')
lions_df2019 = pd.read_csv('Resources/Lions_Raw/lions_sportsref_download2019.csv')
lions_2020 = pd.read_csv('Resources/2020data/lions_sportsref_download2020.csv')

lions_df2010 = yearly_data(df=lions_df2010, year=2010, 
                        HC='Jim Schwartz', 
                        OC='Scott Linehan', 
                        DC='Gunther Cunningham', 
                        QB='Shaun Hill', 
                        QB2='Matthew Stafford', QB2_range=range(8,10,1))
lions_df2010.loc[lions_df2010['Week'] == range(13,16,1), 'QB'] = "Drew Stanton"
lions_df2010.loc[lions_df2010['Week'] == 1, 'QB'] = "Matthew Stafford"

lions_df2011 = yearly_data(df=lions_df2011, year=2011, 
                        HC='Jim Schwartz', 
                        OC='Scott Linehan', 
                        DC='Gunther Cunningham', 
                        QB='Matthew Stafford')

lions_df2012 = yearly_data(df=lions_df2012, year=2012, 
                        HC='Jim Schwartz', 
                        OC='Scott Linehan', 
                        DC='Gunther Cunningham',
                        QB='Matthew Stafford')

lions_df2013 = yearly_data(df=lions_df2013, year=2013, 
                        HC='Jim Schwartz', 
                        OC='Scott Linehan', 
                        DC='Gunther Cunningham', 
                        QB='Matthew Stafford')

lions_df2014 = yearly_data(df=lions_df2014, year=2014, 
                        HC='Jim Caldwell', 
                        OC='Joe Lombardi', 
                        DC='Jim Bob Cooter', 
                        QB='Matthew Stafford')

lions_df2015 = yearly_data(df=lions_df2015, year=2015, 
                        HC='Jim Caldwell', 
                        OC='Joe Lombardi', 
                        DC='Teryl Austin', 
                        QB='Matthew Stafford')
lions_df2015.loc[lions_df2015['Week'] == range(7,18,1), 'OC'] = 'Jim Bob Cooter'

lions_df2016 = yearly_data(df=lions_df2016, year=2016, 
                        HC='Jim Caldwell',
                        OC='Jim Bob Cooter', 
                        DC="Teryl Austin", 
                        QB='Matthew Stafford')

lions_df2017 = yearly_data(df=lions_df2017, year=2017, 
                        HC='Jim Caldwell', 
                        OC='Jim Bob Cooter', 
                        DC='Teryl Austin', 
                        QB='Matthew Stafford')

lions_df2018 = yearly_data(df=lions_df2018, year=2018, 
                        HC='Matt Patricia', 
                        OC='Jim Bob Cooter', 
                        DC='Paul Pasqualoni', 
                        QB='Matthew Stafford')

lions_df2019 = yearly_data(df=lions_df2019, year=2019, 
                        HC='Matt Patricia', 
                        OC='Darrell Bevell', 
                        DC='Paul Pasqualoni', 
                        QB='Matthew Stafford',
                        QB2='David Blough', QB2_range=range(13,18,1))
lions_df2019.loc[lions_df2019['Week'] == range(10,13,1), 'QB'] = "Jeff Driskel"

lions_2020 = yearly_data(df=lions_2020, year=2020, 
                        HC='Matt Patricia', 
                        OC='Darrell Bevell', 
                        DC='Cory Undlin', 
                        QB='Matthew Stafford')

# Make master dataframe for team
lions_df = lions_2020.append(lions_df2019, ignore_index=True)
lions_df = lions_df.append(lions_df2018, ignore_index=True)
lions_df = lions_df.append(lions_df2017, ignore_index=True)
lions_df = lions_df.append(lions_df2016, ignore_index=True)
lions_df = lions_df.append(lions_df2015, ignore_index=True)
lions_df = lions_df.append(lions_df2014, ignore_index=True)
lions_df = lions_df.append(lions_df2013, ignore_index=True)
lions_df = lions_df.append(lions_df2012, ignore_index=True)
lions_df = lions_df.append(lions_df2011, ignore_index=True)
lions_df = lions_df.append(lions_df2010, ignore_index=True)

lions_df['Base_Team'] = 'Lions'
lions_df = lions_df.sort_values(by=['year', 'Week'], ascending=[False, True])

lions_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Sun,0.0,0.0,1.0,Chicago Bears,2020,Lions,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Matt Patricia,Darrell Bevell,Cory Undlin,Matthew Stafford
1,2,Sun,0.0,0.0,0.0,Green Bay Packers,2020,Lions,23.0,27.0,...,0.0,214.0,149.0,426.0,363.0,0,Matt Patricia,Darrell Bevell,Cory Undlin,Matthew Stafford
2,3,Sun,1.0,0.0,0.0,Arizona Cardinals,2020,Lions,44.0,69.0,...,0.0,443.0,408.0,733.0,851.0,0,Matt Patricia,Darrell Bevell,Cory Undlin,Matthew Stafford
3,4,Sun,0.0,1.0,1.0,New Orleans Saints,2020,Lions,70.0,92.0,...,3.0,711.0,517.0,1055.0,1228.0,0,Matt Patricia,Darrell Bevell,Cory Undlin,Matthew Stafford
4,6,Sun,NaN,NaN,0.0,Jacksonville Jaguars,2020,Lions,99.0,127.0,...,4.0,939.0,681.0,1336.0,1620.0,1,Matt Patricia,Darrell Bevell,Cory Undlin,Matthew Stafford
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,13,Thu,1.0,6.0,1.0,Green Bay Packers,2010,Lions,286.0,277.0,...,14.0,3045.0,968.0,4533.0,4013.0,0,Jim Schwartz,Scott Linehan,Gunther Cunningham,Shaun Hill
172,14,Sun,0.0,7.0,0.0,Philadelphia Eagles,2010,Lions,326.0,287.0,...,17.0,3147.0,992.0,5094.0,4139.0,0,Jim Schwartz,Scott Linehan,Gunther Cunningham,Shaun Hill
173,15,Mon,0.0,7.0,1.0,Baltimore Ravens,2010,Lions,346.0,321.0,...,18.0,3326.0,1291.0,5325.0,4617.0,0,Jim Schwartz,Scott Linehan,Gunther Cunningham,Shaun Hill
174,16,Sun,0.0,7.0,1.0,New York Giants,2010,Lions,362.0,339.0,...,18.0,3541.0,1381.0,5674.0,4922.0,0,Jim Schwartz,Scott Linehan,Gunther Cunningham,Shaun Hill


In [94]:
lions_df.to_csv('Resources/indv_team_csvs/lions.csv')

### Vikings Data, 2010-2019

In [95]:
# Data Source: https://www.pro-football-reference.com/teams/min/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

vikings_df2010 = pd.read_csv('Resources/Vikings_Raw/vikings_sportsref_download2010.csv')
vikings_df2011 = pd.read_csv('Resources/Vikings_Raw/vikings_sportsref_download2011.csv')
vikings_df2012 = pd.read_csv('Resources/Vikings_Raw/vikings_sportsref_download2012.csv')
vikings_df2013 = pd.read_csv('Resources/Vikings_Raw/vikings_sportsref_download2013.csv')
vikings_df2014 = pd.read_csv('Resources/Vikings_Raw/vikings_sportsref_download2014.csv')
vikings_df2015 = pd.read_csv('Resources/Vikings_Raw/vikings_sportsref_download2015.csv')
vikings_df2016 = pd.read_csv('Resources/Vikings_Raw/vikings_sportsref_download2016.csv')
vikings_df2017 = pd.read_csv('Resources/Vikings_Raw/vikings_sportsref_download2017.csv')
vikings_df2018 = pd.read_csv('Resources/Vikings_Raw/vikings_sportsref_download2018.csv')
vikings_df2019 = pd.read_csv('Resources/Vikings_Raw/vikings_sportsref_download2019.csv')
vikings_2020 = pd.read_csv('Resources/2020data/vikings_sportsref_download2020.csv')

vikings_df2010 = yearly_data(df=vikings_df2010, year=2010, 
                        HC='Brad Childress', HC2='Leslie Frazier', HC2_range=range(13,18,1),
                        OC='Darrell Bevell', 
                        DC='Leslie Frazier', 
                        QB='Brett Favre',
                        QB2='Joe Webb', QB2_range=range(16,18,1))
vikings_df2010.loc[vikings_df2010['Week'] == range(13,18,1), 'DC'] = "Fred Pagac"
vikings_df2010.loc[vikings_df2010['Week'] == 14, 'QB'] = "Tarvaris Jackson"

vikings_df2011 = yearly_data(df=vikings_df2011, year=2011, 
                        HC='Leslie Frazier', 
                        OC='Bill Musgrave', 
                        DC='Fred Pagac', 
                        QB='Christian Ponder',
                        QB2='Donovan McNabb', QB2_range=range(1,7,1))

vikings_df2012 = yearly_data(df=vikings_df2012, year=2012, 
                        HC='Leslie Frazier', 
                        OC='Bill Musgrave', 
                        DC='Alan Williams',
                        QB='Christian Ponder')

vikings_df2013 = yearly_data(df=vikings_df2013, year=2013, 
                        HC='Leslie Frazier', 
                        OC='Bill Musgrave', 
                        DC='Alan Williams', 
                        QB='Christian Ponder',
                        QB2='Matt Cassel', QB2_range=range(14,18,1))
vikings_df2013.loc[vikings_df2013['Week'] == range(4,7,1), 'QB'] = "Matt Cassel"
vikings_df2013.loc[vikings_df2013['Week'] == 7, 'QB'] = "Josh Freeman"

vikings_df2014 = yearly_data(df=vikings_df2014, year=2014, 
                        HC='Mike Zimmer', 
                        OC='Norv Turner', 
                        DC='George Edwards', 
                        QB='Teddy Bridgewater',
                        QB2='Matt Cassel', QB2_range=range(1,4,1))
vikings_df2014.loc[vikings_df2014['Week'] == 5, 'QB'] = "Christian Ponder"

vikings_df2015 = yearly_data(df=vikings_df2015, year=2015, 
                        HC='Mike Zimmer', 
                        OC='Norv Turner', 
                        DC='George Edwards', 
                        QB='Teddy Bridgewater')

vikings_df2016 = yearly_data(df=vikings_df2016, year=2016, 
                        HC='Mike Zimmer', HC2='Mike Priefer', HC2_range=13, 
                        OC='Norv Turner', 
                        DC='George Edwards', 
                        QB='Sam Bradford',
                        QB2='Shaun Hill', QB2_range=1)
vikings_df2016.loc[vikings_df2016['Week'] == range(9,18,1), 'OC'] = "Pat Shurmur"

vikings_df2017 = yearly_data(df=vikings_df2017, year=2017, 
                        HC='Mike Zimmer', 
                        OC='Pat Shurmur', 
                        DC='George Edwards', 
                        QB='Case Keenum',
                        QB2='Sam Bradford', QB2_range=range(1,3,1))

vikings_df2018 = yearly_data(df=vikings_df2018, year=2018, 
                        HC='Mike Zimmer', 
                        OC='John DeFilippo', 
                        DC='George Edwards', 
                        QB='Kirk Cousins')

vikings_df2019 = yearly_data(df=vikings_df2019, year=2019, 
                        HC='Mike Zimmer', 
                        OC='Kevin Stefanski', 
                        DC='George Edwards', 
                        QB='Kirk Cousins',
                        QB2='Sean Mannion', QB2_range=17)

vikings_2020 = yearly_data(df=vikings_2020, year=2020, 
                        HC='Mike Zimmer', 
                        OC='Gary Kubiak', 
                        DC='Andre Patterson', 
                        QB='Kirk Cousins')

# Make master dataframe for team
vikings_df = vikings_2020.append(vikings_df2019, ignore_index=True)
vikings_df = vikings_df.append(vikings_df2018, ignore_index=True)
vikings_df = vikings_df.append(vikings_df2017, ignore_index=True)
vikings_df = vikings_df.append(vikings_df2016, ignore_index=True)
vikings_df = vikings_df.append(vikings_df2015, ignore_index=True)
vikings_df = vikings_df.append(vikings_df2014, ignore_index=True)
vikings_df = vikings_df.append(vikings_df2013, ignore_index=True)
vikings_df = vikings_df.append(vikings_df2012, ignore_index=True)
vikings_df = vikings_df.append(vikings_df2011, ignore_index=True)
vikings_df = vikings_df.append(vikings_df2010, ignore_index=True)

vikings_df['Base_Team'] = 'Vikings'
vikings_df = vikings_df.sort_values(by=['year', 'Week'], ascending=[False, True])

vikings_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Sun,0.0,0.0,1.0,Green Bay Packers,2020,Vikings,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Mike Zimmer,Gary Kubiak,Andre Patterson,Kirk Cousins
1,2,Sun,0.0,0.0,0.0,Indianapolis Colts,2020,Vikings,34.0,43.0,...,0.0,364.0,158.0,382.0,522.0,0,Mike Zimmer,Gary Kubiak,Andre Patterson,Kirk Cousins
2,3,Sun,0.0,0.0,1.0,Tennessee Titans,2020,Vikings,45.0,71.0,...,1.0,567.0,309.0,557.0,876.0,0,Mike Zimmer,Gary Kubiak,Andre Patterson,Kirk Cousins
3,4,Sun,1.0,0.0,0.0,Houston Texans,2020,Vikings,75.0,102.0,...,2.0,877.0,443.0,1021.0,1320.0,0,Mike Zimmer,Gary Kubiak,Andre Patterson,Kirk Cousins
4,5,Sun,NaN,NaN,0.0,Seattle Seahawks,2020,Vikings,106.0,125.0,...,3.0,1167.0,539.0,1431.0,1706.0,0,Mike Zimmer,Gary Kubiak,Andre Patterson,Kirk Cousins
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,13,Sun,1.0,4.0,1.0,Buffalo Bills,2010,Vikings,189.0,239.0,...,13.0,2367.0,1020.0,3667.0,3387.0,0,Brad Childress,Darrell Bevell,Leslie Frazier,Brett Favre
172,14,Mon,0.0,5.0,1.0,New York Giants,2010,Vikings,227.0,253.0,...,18.0,2522.0,1104.0,4054.0,3626.0,0,Brad Childress,Darrell Bevell,Leslie Frazier,Tarvaris Jackson
173,15,Mon,0.0,5.0,1.0,Chicago Bears,2010,Vikings,230.0,274.0,...,20.0,2704.0,1317.0,4218.0,4021.0,0,Brad Childress,Darrell Bevell,Leslie Frazier,Brett Favre
174,16,Tue,1.0,5.0,0.0,Philadelphia Eagles,2010,Vikings,244.0,314.0,...,21.0,2893.0,1421.0,4491.0,4314.0,0,Brad Childress,Darrell Bevell,Leslie Frazier,Brett Favre


In [96]:
vikings_df.to_csv('Resources/indv_team_csvs/vikings.csv')

### (20) Bengals Data, 2010-2019

In [97]:
# Data Source: https://www.pro-football-reference.com/teams/cin/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

bengals_df2010 = pd.read_csv('Resources/Bengals_Raw/bengals_sportsref_download2010.csv')
bengals_df2011 = pd.read_csv('Resources/Bengals_Raw/bengals_sportsref_download2011.csv')
bengals_df2012 = pd.read_csv('Resources/Bengals_Raw/bengals_sportsref_download2012.csv')
bengals_df2013 = pd.read_csv('Resources/Bengals_Raw/bengals_sportsref_download2013.csv')
bengals_df2014 = pd.read_csv('Resources/Bengals_Raw/bengals_sportsref_download2014.csv')
bengals_df2015 = pd.read_csv('Resources/Bengals_Raw/bengals_sportsref_download2015.csv')
bengals_df2016 = pd.read_csv('Resources/Bengals_Raw/bengals_sportsref_download2016.csv')
bengals_df2017 = pd.read_csv('Resources/Bengals_Raw/bengals_sportsref_download2017.csv')
bengals_df2018 = pd.read_csv('Resources/Bengals_Raw/bengals_sportsref_download2018.csv')
bengals_df2019 = pd.read_csv('Resources/Bengals_Raw/bengals_sportsref_download2019.csv')
bengals_2020 = pd.read_csv('Resources/2020data/bengals_sportsref_download2020.csv')

bengals_df2010 = yearly_data(df=bengals_df2010, year=2010, 
                        HC='Marvin Lewis', 
                        OC='Bob Bratkowski', 
                        DC='Mike Zimmer', 
                        QB='Carson Palmer')

bengals_df2011 = yearly_data(df=bengals_df2011, year=2011, 
                        HC='Marvin Lewis', 
                        OC='Jay Gruden', 
                        DC='Mike Zimmer', 
                        QB='Andy Dalton')

bengals_df2012 = yearly_data(df=bengals_df2012, year=2012, 
                        HC='Marvin Lewis', 
                        OC='Jay Gruden', 
                        DC='Mike Zimmer',
                        QB='Andy Dalton')

bengals_df2013 = yearly_data(df=bengals_df2013, year=2013, 
                        HC='Marvin Lewis', 
                        OC='Jay Gruden', 
                        DC='Mike Zimmer', 
                        QB='Andy Dalton')

bengals_df2014 = yearly_data(df=bengals_df2014, year=2014, 
                        HC='Marvin Lewis', 
                        OC='Hue Jackson', 
                        DC='Paul Guenther', 
                        QB='Andy Dalton')

bengals_df2015 = yearly_data(df=bengals_df2015, year=2015, 
                        HC='Marvin Lewis', 
                        OC='Hue Jackson', 
                        DC='Paul Guenther', 
                        QB='Andy Dalton',
                        QB2='AJ McCarron', QB2_range=range(15,18,1))

bengals_df2016 = yearly_data(df=bengals_df2016, year=2016, 
                        HC='Marvin Lewis',
                        OC='Ken Zampese', 
                        DC="Paul Guenther", 
                        QB='Andy Dalton')

bengals_df2017 = yearly_data(df=bengals_df2017, year=2017, 
                        HC='Marvin Lewis', 
                        OC='Ken Zampese', 
                        DC='Paul Guenther', 
                        QB='Andy Dalton')
bengals_df2017.loc[bengals_df2017['Week'] == range(3,18,1), 'OC'] = "Bill Lazor"

bengals_df2018 = yearly_data(df=bengals_df2018, year=2018, 
                        HC='Marvin Lewis', 
                        OC='Bill Lazor', 
                        DC='Teryl Austin', 
                        QB='Andy Dalton',
                        QB2='Jeff Driskel', QB2_range=range(13,18,1))

bengals_df2019 = yearly_data(df=bengals_df2019, year=2019, 
                        HC='Zac Taylor', 
                        OC='Brian Callahan', 
                        DC='Lou Anarumo', 
                        QB='Andy Dalton',
                        QB2='Ryan Finley', QB2_range=range(10,13,1))

bengals_2020 = yearly_data(df=bengals_2020, year=2020, 
                        HC='Zac Taylor', 
                        OC='Brian Callahan', 
                        DC='Lou Anarumo', 
                        QB='Joe Burrow')

# Make master dataframe for team
bengals_df = bengals_2020.append(bengals_df2019, ignore_index=True)
bengals_df = bengals_df.append(bengals_df2018, ignore_index=True)
bengals_df = bengals_df.append(bengals_df2017, ignore_index=True)
bengals_df = bengals_df.append(bengals_df2016, ignore_index=True)
bengals_df = bengals_df.append(bengals_df2015, ignore_index=True)
bengals_df = bengals_df.append(bengals_df2014, ignore_index=True)
bengals_df = bengals_df.append(bengals_df2013, ignore_index=True)
bengals_df = bengals_df.append(bengals_df2012, ignore_index=True)
bengals_df = bengals_df.append(bengals_df2011, ignore_index=True)
bengals_df = bengals_df.append(bengals_df2010, ignore_index=True)

bengals_df['Base_Team'] = 'Bengals'
bengals_df = bengals_df.sort_values(by=['year', 'Week'], ascending=[False, True])

bengals_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Sun,0.0,0.0,1.0,Los Angeles Chargers,2020,Bengals,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Zac Taylor,Brian Callahan,Lou Anarumo,Joe Burrow
1,2,Thu,0.0,0.0,0.0,Cleveland Browns,2020,Bengals,13.0,16.0,...,0.0,207.0,155.0,295.0,362.0,0,Zac Taylor,Brian Callahan,Lou Anarumo,Joe Burrow
2,3,Sun,0.0,0.0,0.0,Philadelphia Eagles,2020,Bengals,43.0,51.0,...,1.0,426.0,370.0,648.0,796.0,0,Zac Taylor,Brian Callahan,Lou Anarumo,Joe Burrow
3,4,Sun,1.0,0.0,1.0,Jacksonville Jaguars,2020,Bengals,66.0,74.0,...,3.0,632.0,545.0,952.0,1177.0,0,Zac Taylor,Brian Callahan,Lou Anarumo,Joe Burrow
4,5,Sun,NaN,NaN,0.0,Baltimore Ravens,2020,Bengals,99.0,99.0,...,4.0,972.0,634.0,1457.0,1606.0,0,Zac Taylor,Brian Callahan,Lou Anarumo,Joe Burrow
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,13,Sun,0.0,2.0,1.0,New Orleans Saints,2010,Bengals,225.0,288.0,...,21.0,2379.0,1353.0,3625.0,3732.0,0,Marvin Lewis,Bob Bratkowski,Mike Zimmer,Carson Palmer
172,14,Sun,0.0,2.0,0.0,Pittsburgh Steelers,2010,Bengals,255.0,322.0,...,22.0,2669.0,1499.0,3936.0,4168.0,0,Marvin Lewis,Bob Bratkowski,Mike Zimmer,Carson Palmer
173,15,Sun,1.0,2.0,1.0,Cleveland Browns,2010,Bengals,262.0,345.0,...,22.0,2900.0,1622.0,4126.0,4522.0,0,Marvin Lewis,Bob Bratkowski,Mike Zimmer,Carson Palmer
174,16,Sun,1.0,3.0,1.0,San Diego Chargers,2010,Bengals,281.0,362.0,...,22.0,3119.0,1681.0,4523.0,4800.0,0,Marvin Lewis,Bob Bratkowski,Mike Zimmer,Carson Palmer


In [98]:
bengals_df.to_csv('Resources/indv_team_csvs/bengals.csv')

### Raiders Data, 2010-2019

In [99]:
# Data Source: https://www.pro-football-reference.com/teams/rai/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

raiders_df2010 = pd.read_csv('Resources/Raiders_Raw/raiders_sportsref_download2010.csv')
raiders_df2011 = pd.read_csv('Resources/Raiders_Raw/raiders_sportsref_download2011.csv')
raiders_df2012 = pd.read_csv('Resources/Raiders_Raw/raiders_sportsref_download2012.csv')
raiders_df2013 = pd.read_csv('Resources/Raiders_Raw/raiders_sportsref_download2013.csv')
raiders_df2014 = pd.read_csv('Resources/Raiders_Raw/raiders_sportsref_download2014.csv')
raiders_df2015 = pd.read_csv('Resources/Raiders_Raw/raiders_sportsref_download2015.csv')
raiders_df2016 = pd.read_csv('Resources/Raiders_Raw/raiders_sportsref_download2016.csv')
raiders_df2017 = pd.read_csv('Resources/Raiders_Raw/raiders_sportsref_download2017.csv')
raiders_df2018 = pd.read_csv('Resources/Raiders_Raw/raiders_sportsref_download2018.csv')
raiders_df2019 = pd.read_csv('Resources/Raiders_Raw/raiders_sportsref_download2019.csv')
raiders_2020 = pd.read_csv('Resources/2020data/raiders_sportsref_download2020.csv')

raiders_df2010 = yearly_data(df=raiders_df2010, year=2010, 
                        HC='Tom Cable', 
                        OC='Hue Jackson', 
                        DC='John Marshall', 
                        QB='Jason Campbell', 
                        QB2='Bruce Gradkowski', QB2_range=range(3,6,1))
raiders_df2010.loc[raiders_df2010['Week'] == 12, 'QB'] = "Bruce Gradkowski"

raiders_df2011 = yearly_data(df=raiders_df2011, year=2011, 
                        HC='Hue Jackson', 
                        OC='Al Saunders', 
                        DC='Chuck Bresnahan', 
                        QB='Carson Palmer', 
                        QB2='Jason Campbell', QB2_range=range(1,7,1))
raiders_df2011.loc[raiders_df2011['Week'] == 7, 'QB'] = "Kyle Boller"

raiders_df2012 = yearly_data(df=raiders_df2012, year=2012, 
                        HC='Dennis Allen', 
                        OC='Greg Knapp', 
                        DC='Jason Tarver',
                        QB='Carson Palmer', 
                        QB2='Terrelle Pryor', QB2_range=17)

raiders_df2013 = yearly_data(df=raiders_df2013, year=2013, 
                        HC='Dennis Allen', 
                        OC='Greg Olson', 
                        DC='Jason Tarver', 
                        QB='Terrelle Pryor', 
                        QB2='Matt McGloin', QB2_range=range(11,17,1))
raiders_df2013.loc[raiders_df2013['Week'] == 4, 'QB'] = "Matt Flynn"

raiders_df2014 = yearly_data(df=raiders_df2014, year=2014,  
                        HC='Dennis Allen', HC2='Tony Sparano', HC2_range=range(5,18,1),
                        OC='Greg Olson', 
                        DC='Jason Tarver', 
                        QB='Derek Carr')

raiders_df2015 = yearly_data(df=raiders_df2015, year=2015, 
                        HC='Jack Del Rio', 
                        OC='Bill Musgrave', 
                        DC='Ken Norton', 
                        QB='Derek Carr')

raiders_df2016 = yearly_data(df=raiders_df2016, year=2016, 
                        HC='Jack Del Rio', 
                        OC='Bill Musgrave', 
                        DC='Ken Norton', 
                        QB='Derek Carr', 
                        QB2='Matt McGloin', QB2_range=17)

raiders_df2017 = yearly_data(df=raiders_df2017, year=2017, 
                        HC='Jack Del Rio', 
                        OC='Todd Downing', 
                        DC='Ken Norton', 
                        QB='Derek Carr', 
                        QB2='EJ Manuel', QB2_range=5)
raiders_df2017.loc[raiders_df2017['Week'] == range(12,18,1), 'DC'] = "John Pagano"

raiders_df2018 = yearly_data(df=raiders_df2018, year=2018, 
                        HC='Jon Gruden', 
                        OC='Greg Olson', 
                        DC='Paul Guenther', 
                        QB='Derek Carr')

raiders_df2019 = yearly_data(df=raiders_df2019, year=2019, 
                        HC='Jon Gruden', 
                        OC='Greg Olson', 
                        DC='Paul Guenther', 
                        QB='Derek Carr')

raiders_2020 = yearly_data(df=raiders_2020, year=2020, 
                        HC='Jon Gruden', 
                        OC='Greg Olson', 
                        DC='Paul Guenther', 
                        QB='Derek Carr')

# Make master dataframe for team
raiders_df = raiders_2020.append(raiders_df2019, ignore_index=True)
raiders_df = raiders_df.append(raiders_df2018, ignore_index=True)
raiders_df = raiders_df.append(raiders_df2017, ignore_index=True)
raiders_df = raiders_df.append(raiders_df2016, ignore_index=True)
raiders_df = raiders_df.append(raiders_df2015, ignore_index=True)
raiders_df = raiders_df.append(raiders_df2014, ignore_index=True)
raiders_df = raiders_df.append(raiders_df2013, ignore_index=True)
raiders_df = raiders_df.append(raiders_df2012, ignore_index=True)
raiders_df = raiders_df.append(raiders_df2011, ignore_index=True)
raiders_df = raiders_df.append(raiders_df2010, ignore_index=True)

raiders_df['Base_Team'] = 'Raiders'
raiders_df = raiders_df.sort_values(by=['year', 'Week'], ascending=[False, True])

raiders_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Sun,1.0,0.0,0.0,Carolina Panthers,2020,Raiders,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Jon Gruden,Greg Olson,Paul Guenther,Derek Carr
1,2,Mon,1.0,1.0,1.0,New Orleans Saints,2020,Raiders,34.0,30.0,...,0.0,259.0,129.0,372.0,388.0,0,Jon Gruden,Greg Olson,Paul Guenther,Derek Carr
2,3,Sun,0.0,2.0,0.0,New England Patriots,2020,Raiders,68.0,54.0,...,1.0,571.0,241.0,749.0,812.0,0,Jon Gruden,Greg Olson,Paul Guenther,Derek Carr
3,4,Sun,0.0,2.0,1.0,Buffalo Bills,2020,Raiders,88.0,90.0,...,2.0,727.0,491.0,1124.0,1218.0,0,Jon Gruden,Greg Olson,Paul Guenther,Derek Carr
4,5,Sun,NaN,NaN,0.0,Kansas City Chiefs,2020,Raiders,111.0,120.0,...,2.0,1001.0,553.0,1507.0,1554.0,0,Jon Gruden,Greg Olson,Paul Guenther,Derek Carr
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,13,Sun,1.0,5.0,0.0,San Diego Chargers,2010,Raiders,255.0,256.0,...,16.0,2217.0,1471.0,3694.0,3688.0,0,Tom Cable,Hue Jackson,John Marshall,Jason Campbell
172,14,Sun,0.0,6.0,0.0,Jacksonville Jaguars,2010,Raiders,283.0,269.0,...,18.0,2482.0,1492.0,4062.0,3974.0,0,Tom Cable,Hue Jackson,John Marshall,Jason Campbell
173,15,Sun,1.0,6.0,1.0,Denver Broncos,2010,Raiders,314.0,307.0,...,19.0,2633.0,1726.0,4538.0,4359.0,0,Tom Cable,Hue Jackson,John Marshall,Jason Campbell
174,16,Sun,0.0,7.0,1.0,Indianapolis Colts,2010,Raiders,353.0,330.0,...,20.0,2762.0,1832.0,5040.0,4594.0,0,Tom Cable,Hue Jackson,John Marshall,Jason Campbell


In [100]:
raiders_df.to_csv('Resources/indv_team_csvs/raiders.csv')

### Steelers Data, 2010-2019

In [101]:
# Data Source: https://www.pro-football-reference.com/teams/pit/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

steelers_df2010 = pd.read_csv('Resources/Steelers_Raw/steelers_sportsref_download2010.csv')
steelers_df2011 = pd.read_csv('Resources/Steelers_Raw/steelers_sportsref_download2011.csv')
steelers_df2012 = pd.read_csv('Resources/Steelers_Raw/steelers_sportsref_download2012.csv')
steelers_df2013 = pd.read_csv('Resources/Steelers_Raw/steelers_sportsref_download2013.csv')
steelers_df2014 = pd.read_csv('Resources/Steelers_Raw/steelers_sportsref_download2014.csv')
steelers_df2015 = pd.read_csv('Resources/Steelers_Raw/steelers_sportsref_download2015.csv')
steelers_df2016 = pd.read_csv('Resources/Steelers_Raw/steelers_sportsref_download2016.csv')
steelers_df2017 = pd.read_csv('Resources/Steelers_Raw/steelers_sportsref_download2017.csv')
steelers_df2018 = pd.read_csv('Resources/Steelers_Raw/steelers_sportsref_download2018.csv')
steelers_df2019 = pd.read_csv('Resources/Steelers_Raw/steelers_sportsref_download2019.csv')
steelers_2020 = pd.read_csv('Resources/2020data/steelers_sportsref_download2020.csv')

steelers_df2010 = yearly_data(df=steelers_df2010, year=2010, 
                        HC='Mike Tomlin', 
                        OC='Bruce Arians', 
                        DC='Dick LeBeau', 
                        QB='Ben Roethlisberger',
                        QB2='Charlie Batch', QB2_range=range(3,5,1))
steelers_df2010.loc[steelers_df2010['Week'] == range(1,3,1), 'QB'] = "Dennis Dixon"

steelers_df2011 = yearly_data(df=steelers_df2011, year=2011, 
                        HC='Mike Tomlin', 
                        OC='Bruce Arians', 
                        DC='Dick LeBeau', 
                        QB='Ben Roethlisberger',
                        QB2='Charlie Batch', QB2_range=16)

steelers_df2012 = yearly_data(df=steelers_df2012, year=2012, 
                        HC='Mike Tomlin', 
                        OC='Todd Haley', 
                        DC='Dick LeBeau',
                        QB='Ben Roethlisberger',
                        QB2='Charlie Batch', QB2_range=range(12,14,1))
steelers_df2012.loc[steelers_df2012['Week'] == 11, 'QB'] = "Byron Leftwich"

steelers_df2013 = yearly_data(df=steelers_df2013, year=2013, 
                        HC='Mike Tomlin', 
                        OC='Todd Haley', 
                        DC='Dick LeBeau', 
                        QB='Ben Roethlisberger')

steelers_df2014 = yearly_data(df=steelers_df2014, year=2014, 
                        HC='Mike Tomlin', 
                        OC='Todd Haley', 
                        DC='Dick LeBeau', 
                        QB='Ben Roethlisberger')

steelers_df2015 = yearly_data(df=steelers_df2015, year=2015, 
                        HC='Mike Tomlin', 
                        OC='Todd Haley', 
                        DC='Keith Butler', 
                        QB='Ben Roethlisberger',
                        QB2='Michael Vick', QB2_range=range(4,7,1))
steelers_df2015.loc[steelers_df2015['Week'] == range(7,11,3), 'QB'] = "Landry Jones"

steelers_df2016 = yearly_data(df=steelers_df2016, year=2016, 
                        HC='Mike Tomlin', 
                        OC='Todd Haley', 
                        DC='Keith Butler', 
                        QB='Ben Roethlisberger',
                        QB2='Landry Jones', QB2_range=range(7,18,10))

steelers_df2017 = yearly_data(df=steelers_df2017, year=2017, 
                        HC='Mike Tomlin', 
                        OC='Todd Haley', 
                        DC='Keith Butler', 
                        QB='Ben Roethlisberger',
                        QB2='Landry Jones', QB2_range=17)

steelers_df2018 = yearly_data(df=steelers_df2018, year=2018, 
                        HC='Mike Tomlin', 
                        OC='Randy Fichtner', 
                        DC='Keith Butler', 
                        QB='Ben Roethlisberger')

steelers_df2019 = yearly_data(df=steelers_df2019, year=2019, 
                        HC='Mike Tomlin', 
                        OC='Randy Fichtner', 
                        DC='Keith Butler', 
                        QB='Mason Rudolph',
                        QB2='Devlin Hodges', QB2_range=6)
steelers_df2019.loc[steelers_df2019['Week'] == range(1,3,1), 'QB'] = "Ben Roethlisberger"
steelers_df2019.loc[steelers_df2019['Week'] == range(13,18,1), 'QB'] = "Devlin Hodges"

steelers_2020 = yearly_data(df=steelers_2020, year=2020, 
                        HC='Mike Tomlin', 
                        OC='Randy Fichtner', 
                        DC='Keith Butler', 
                        QB='Ben Roethlisberger')

# Make master dataframe for team
steelers_df = steelers_2020.append(steelers_df2019, ignore_index=True)
steelers_df = steelers_df.append(steelers_df2018, ignore_index=True)
steelers_df = steelers_df.append(steelers_df2017, ignore_index=True)
steelers_df = steelers_df.append(steelers_df2016, ignore_index=True)
steelers_df = steelers_df.append(steelers_df2015, ignore_index=True)
steelers_df = steelers_df.append(steelers_df2014, ignore_index=True)
steelers_df = steelers_df.append(steelers_df2013, ignore_index=True)
steelers_df = steelers_df.append(steelers_df2012, ignore_index=True)
steelers_df = steelers_df.append(steelers_df2011, ignore_index=True)
steelers_df = steelers_df.append(steelers_df2010, ignore_index=True)

steelers_df['Base_Team'] = 'Steelers'
steelers_df = steelers_df.sort_values(by=['year', 'Week'], ascending=[False, True])

steelers_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Mon,1.0,0.0,0.0,New York Giants,2020,Steelers,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Mike Tomlin,Randy Fichtner,Keith Butler,Ben Roethlisberger
1,2,Sun,1.0,1.0,1.0,Denver Broncos,2020,Steelers,26.0,16.0,...,2.0,262.0,29.0,349.0,291.0,0,Mike Tomlin,Randy Fichtner,Keith Butler,Ben Roethlisberger
2,3,Sun,1.0,2.0,1.0,Houston Texans,2020,Steelers,52.0,37.0,...,4.0,477.0,133.0,759.0,610.0,0,Mike Tomlin,Randy Fichtner,Keith Butler,Ben Roethlisberger
3,5,Sun,NaN,NaN,1.0,Philadelphia Eagles,2020,Steelers,80.0,58.0,...,5.0,708.0,162.0,1146.0,870.0,1,Mike Tomlin,Randy Fichtner,Keith Butler,Ben Roethlisberger
4,6,Sun,NaN,NaN,1.0,Cleveland Browns,2020,Steelers,80.0,58.0,...,5.0,708.0,162.0,1146.0,870.0,0,Mike Tomlin,Randy Fichtner,Keith Butler,Ben Roethlisberger
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,13,Sun,1.0,8.0,0.0,Baltimore Ravens,2010,Steelers,254.0,181.0,...,26.0,2646.0,704.0,3679.0,3350.0,0,Mike Tomlin,Bruce Arians,Dick LeBeau,Ben Roethlisberger
172,14,Sun,1.0,9.0,1.0,Cincinnati Bengals,2010,Steelers,267.0,191.0,...,27.0,2872.0,747.0,3967.0,3619.0,0,Mike Tomlin,Bruce Arians,Dick LeBeau,Ben Roethlisberger
173,15,Sun,0.0,10.0,1.0,New York Jets,2010,Steelers,290.0,198.0,...,30.0,3028.0,781.0,4321.0,3809.0,0,Mike Tomlin,Bruce Arians,Dick LeBeau,Ben Roethlisberger
174,16,Thu,1.0,10.0,1.0,Carolina Panthers,2010,Steelers,307.0,220.0,...,30.0,3198.0,887.0,4699.0,4085.0,0,Mike Tomlin,Bruce Arians,Dick LeBeau,Ben Roethlisberger


In [102]:
steelers_df.to_csv('Resources/indv_team_csvs/steelers.csv')

### Browns Data, 2010-2019

In [103]:
# Data Source: https://www.pro-football-reference.com/teams/cle/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

browns_df2010 = pd.read_csv('Resources/Browns_Raw/browns_sportsref_download2010.csv')
browns_df2011 = pd.read_csv('Resources/Browns_Raw/browns_sportsref_download2011.csv')
browns_df2012 = pd.read_csv('Resources/Browns_Raw/browns_sportsref_download2012.csv')
browns_df2013 = pd.read_csv('Resources/Browns_Raw/browns_sportsref_download2013.csv')
browns_df2014 = pd.read_csv('Resources/Browns_Raw/browns_sportsref_download2014.csv')
browns_df2015 = pd.read_csv('Resources/Browns_Raw/browns_sportsref_download2015.csv')
browns_df2016 = pd.read_csv('Resources/Browns_Raw/browns_sportsref_download2016.csv')
browns_df2017 = pd.read_csv('Resources/Browns_Raw/browns_sportsref_download2017.csv')
browns_df2018 = pd.read_csv('Resources/Browns_Raw/browns_sportsref_download2018.csv')
browns_df2019 = pd.read_csv('Resources/Browns_Raw/browns_sportsref_download2019.csv')
browns_2020 = pd.read_csv('Resources/2020data/browns_sportsref_download2020.csv')

browns_df2010 = yearly_data(df=browns_df2010, year=2010, 
                        HC='Eric Mangini', 
                        OC='Brian Daboll', 
                        DC='Rob Ryan', 
                        QB='Jake Delhomme', 
                        QB2='Colt McCoy', QB2_range=range(9,12,1))
browns_df2010.loc[browns_df2010['Week'] == range(2,6,1), 'QB'] = "Seneca Wallace"
browns_df2010.loc[browns_df2010['Week'] == range(15,18,1), 'QB'] = "Colt McCoy"

browns_df2011 = yearly_data(df=browns_df2011, year=2011, 
                        HC='Pat Shurmur', 
                        OC='Pat Shurmur', 
                        DC='Dick Jauron', 
                        QB='Colt McCoy', 
                        QB2='Seneca Wallace', QB2_range=range(15,18,1))

browns_df2012 = yearly_data(df=browns_df2012, year=2012, 
                        HC='Pat Shurmur', 
                        OC='Brad Childress', 
                        DC='Dick Jauron',
                        QB='Brandon Weeden', 
                        QB2='Thad Lewis', QB2_range=17)

browns_df2013 = yearly_data(df=browns_df2013, year=2013, 
                        HC='Rob Chudzinski', 
                        OC='Norv Turner', 
                        DC='Ray Horton', 
                        QB='Jason Campbell', 
                        QB2='Brandon Weeden', QB2_range=range(1,8,1))
browns_df2013.loc[browns_df2013['Week'] == range(3,6,1), 'QB'] = "Brian Hoyer"
browns_df2013.loc[browns_df2013['Week'] == 13, 'QB'] = "Brandon Weeden"

browns_df2014 = yearly_data(df=browns_df2014, year=2014, 
                        HC='Mike Pettine', 
                        OC='Kyle Shanahan', 
                        DC="Jim O'Neil", 
                        QB='Brian Hoyer', 
                        QB2='Johnny Manziel', QB2_range=range(15,17,1))
browns_df2014.loc[browns_df2014['Week'] == 17, 'QB'] = "Connor Shaw"

browns_df2015 = yearly_data(df=browns_df2015, year=2015, 
                        HC='Mike Pettine', 
                        OC='John DeFilippo', 
                        DC="Jim O'Neil", 
                        QB='Josh McCown', 
                        QB2='Johnny Manziel', QB2_range=2)
browns_df2015.loc[browns_df2015['Week'] == 13, 'QB'] = "Austin Davis"
browns_df2015.loc[browns_df2015['Week'] == 17, 'QB'] = "Austin Davis"
browns_df2015.loc[browns_df2015['Week'] == range(9,11,1), 'QB'] = "Johnny Manziel"
browns_df2015.loc[browns_df2015['Week'] == range(15,17,1), 'QB'] = "Johnny Manziel"

browns_df2016 = yearly_data(df=browns_df2016, year=2016, 
                        HC='Hue Jackson',
                        OC='Hue Jackson', 
                        DC="Ray Horton", 
                        QB='Cody Kessler', 
                        QB2='Robert Griffin III', QB2_range=range(14,18,1))
browns_df2016.loc[browns_df2016['Week'] == 2, 'QB'] = "Josh McCown"
browns_df2016.loc[browns_df2016['Week'] == 8, 'QB'] = "Josh McCown"
browns_df2016.loc[browns_df2016['Week'] == 12, 'QB'] = "Josh McCown"
browns_df2016.loc[browns_df2016['Week'] == 1, 'QB'] = "Robert Griffin III"

browns_df2017 = yearly_data(df=browns_df2017, year=2017, 
                        HC='Hue Jackson', 
                        OC='Hue Jackson', 
                        DC='Gregg Williams', 
                        QB='DeShone Kizer', 
                        QB2='Kevin Hogan', QB2_range=6)

browns_df2018 = yearly_data(df=browns_df2018, year=2018, 
                        HC='Hue Jackson', HC2='Gregg Williams', HC2_range=range(9,18,1), 
                        OC='Todd Haley', 
                        DC='Gregg Williams', 
                        QB='Baker Mayfield', 
                        QB2='Tyrod Taylor', QB2_range=range(1,4,1))
browns_df2018.loc[browns_df2018['Week'] == range(9,18,1), 'OC'] = "Freddie Kitchens"

browns_df2019 = yearly_data(df=browns_df2019, year=2019, 
                        HC='Freddie Kitchens', 
                        OC='Todd Monken', 
                        DC='Steve Wilks', 
                        QB='Baker Mayfield')

browns_2020 = yearly_data(df=browns_2020, year=2020, 
                        HC='Kevin Stefanski', 
                        OC='Alex Van Pelt', 
                        DC='Joe Woods', 
                        QB='Baker Mayfield')

# Make master dataframe for team
browns_df = browns_2020.append(browns_df2019, ignore_index=True)
browns_df = browns_df.append(browns_df2018, ignore_index=True)
browns_df = browns_df.append(browns_df2017, ignore_index=True)
browns_df = browns_df.append(browns_df2016, ignore_index=True)
browns_df = browns_df.append(browns_df2015, ignore_index=True)
browns_df = browns_df.append(browns_df2014, ignore_index=True)
browns_df = browns_df.append(browns_df2013, ignore_index=True)
browns_df = browns_df.append(browns_df2012, ignore_index=True)
browns_df = browns_df.append(browns_df2011, ignore_index=True)
browns_df = browns_df.append(browns_df2010, ignore_index=True)

browns_df['Base_Team'] = 'Browns'
browns_df = browns_df.sort_values(by=['year', 'Week'], ascending=[False, True])

browns_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Sun,0.0,0.0,0.0,Baltimore Ravens,2020,Browns,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Kevin Stefanski,Alex Van Pelt,Joe Woods,Baker Mayfield
1,2,Thu,1.0,0.0,1.0,Cincinnati Bengals,2020,Browns,6.0,38.0,...,1.0,270.0,111.0,306.0,381.0,0,Kevin Stefanski,Alex Van Pelt,Joe Woods,Baker Mayfield
2,3,Sun,1.0,1.0,1.0,Washington Football Team,2020,Browns,41.0,68.0,...,2.0,555.0,179.0,740.0,734.0,0,Kevin Stefanski,Alex Van Pelt,Joe Woods,Baker Mayfield
3,4,Sun,1.0,2.0,0.0,Dallas Cowboys,2020,Browns,75.0,88.0,...,7.0,761.0,282.0,1040.0,1043.0,0,Kevin Stefanski,Alex Van Pelt,Joe Woods,Baker Mayfield
4,5,Sun,NaN,NaN,1.0,Indianapolis Colts,2020,Browns,124.0,126.0,...,10.0,1242.0,367.0,1548.0,1609.0,0,Kevin Stefanski,Alex Van Pelt,Joe Woods,Baker Mayfield
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,13,Sun,1.0,4.0,0.0,Miami Dolphins,2010,Browns,216.0,229.0,...,23.0,2609.0,1315.0,3413.0,3924.0,0,Eric Mangini,Brian Daboll,Rob Ryan,Jake Delhomme
172,14,Sun,0.0,5.0,0.0,Buffalo Bills,2010,Browns,229.0,239.0,...,26.0,2776.0,1429.0,3665.0,4205.0,0,Eric Mangini,Brian Daboll,Rob Ryan,Jake Delhomme
173,15,Sun,0.0,5.0,0.0,Cincinnati Bengals,2010,Browns,235.0,252.0,...,27.0,2907.0,1621.0,3852.0,4528.0,0,Eric Mangini,Brian Daboll,Rob Ryan,Jake Delhomme
174,16,Sun,0.0,5.0,1.0,Baltimore Ravens,2010,Browns,252.0,271.0,...,27.0,3116.0,1809.0,4130.0,4925.0,0,Eric Mangini,Brian Daboll,Rob Ryan,Jake Delhomme


In [104]:
browns_df.to_csv('Resources/indv_team_csvs/browns.csv')

### (25) Colts Data, 2010-2019

In [105]:
# Data Source: https://www.pro-football-reference.com/teams/clt/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

colts_df2010 = pd.read_csv('Resources/Colts_Raw/colts_sportsref_download2010.csv')
colts_df2011 = pd.read_csv('Resources/Colts_Raw/colts_sportsref_download2011.csv')
colts_df2012 = pd.read_csv('Resources/Colts_Raw/colts_sportsref_download2012.csv')
colts_df2013 = pd.read_csv('Resources/Colts_Raw/colts_sportsref_download2013.csv')
colts_df2014 = pd.read_csv('Resources/Colts_Raw/colts_sportsref_download2014.csv')
colts_df2015 = pd.read_csv('Resources/Colts_Raw/colts_sportsref_download2015.csv')
colts_df2016 = pd.read_csv('Resources/Colts_Raw/colts_sportsref_download2016.csv')
colts_df2017 = pd.read_csv('Resources/Colts_Raw/colts_sportsref_download2017.csv')
colts_df2018 = pd.read_csv('Resources/Colts_Raw/colts_sportsref_download2018.csv')
colts_df2019 = pd.read_csv('Resources/Colts_Raw/colts_sportsref_download2019.csv')
colts_2020 = pd.read_csv('Resources/2020data/colts_sportsref_download2020.csv')

colts_df2010 = yearly_data(df=colts_df2010, year=2010, 
                        HC='Jim Caldwell', 
                        OC='Clyde Christensen', 
                        DC='Larry Coyer', 
                        QB='Peyton Manning')

colts_df2011 = yearly_data(df=colts_df2011, year=2011, 
                        HC='Jim Caldwell', 
                        OC='Clyde Christensen', 
                        DC='Larry Coyer', 
                        QB='Curtis Painter',
                        QB2='Dan Orlovsky', QB2_range=range(13,18,1))
colts_df2011.loc[colts_df2011['Week'] == range(1,4,1), 'QB'] = "Kerry Collins"

colts_df2012 = yearly_data(df=colts_df2012, year=2012, 
                        HC='Chuck Pagano', 
                        OC='Bruce Arians', 
                        DC='Greg Manusky',
                        QB='Andrew Luck')

colts_df2013 = yearly_data(df=colts_df2013, year=2013, 
                        HC='Chuck Pagano', 
                        OC='Pep Hamilton', 
                        DC='Greg Manusky', 
                        QB='Andrew Luck')

colts_df2014 = yearly_data(df=colts_df2014, year=2014, 
                        HC='Chuck Pagano', 
                        OC='Pep Hamilton', 
                        DC='Greg Manusky', 
                        QB='Andrew Luck')

colts_df2015 = yearly_data(df=colts_df2015, year=2015, 
                        HC='Chuck Pagano', 
                        OC='Pep Hamilton', 
                        DC='Greg Manusky', 
                        QB='Matt Hasselbeck',
                        QB2='Andrew Luck', QB2_range=range(1,4,1))
colts_df2015.loc[colts_df2015['Week'] == range(9,18,1), 'OC'] = "Rob Chudzinski"
colts_df2015.loc[colts_df2015['Week'] == 17, 'QB'] = "Josh Freeman"
colts_df2015.loc[colts_df2015['Week'] == range(6,10,1), 'QB'] = "Andrew Luck"

colts_df2016 = yearly_data(df=colts_df2016, year=2016, 
                        HC='Chuck Pagano',
                        OC='Rob Chudzinski', 
                        DC="Ted Monachino", 
                        QB='Andrew Luck', 
                        QB2='Scott Tolzien', QB2_range=12)

colts_df2017 = yearly_data(df=colts_df2017, year=2017, 
                        HC='Chuck Pagano',
                        OC='Rob Chudzinski', 
                        DC="Ted Monachino", 
                        QB='Jacoby Brissett', 
                        QB2='Scott Tolzien', QB2_range=1)

colts_df2018 = yearly_data(df=colts_df2018, year=2018, 
                        HC='Frank Reich', 
                        OC='Nick Sirianni', 
                        DC='Matt Eberflus', 
                        QB='Andrew Luck')

colts_df2019 = yearly_data(df=colts_df2019, year=2019, 
                        HC='Frank Reich', 
                        OC='Nick Sirianni', 
                        DC='Matt Eberflus', 
                        QB='Jacoby Brissett', 
                        QB2='Brian Hoyer', QB2_range=10)

colts_2020 = yearly_data(df=colts_2020, year=2020, 
                        HC='Frank Reich', 
                        OC='Nick Sirianni', 
                        DC='Matt Eberflus', 
                        QB='Philip Rivers')

# Make master dataframe for team
colts_df = colts_2020.append(colts_df2019, ignore_index=True)
colts_df = colts_df.append(colts_df2018, ignore_index=True)
colts_df = colts_df.append(colts_df2017, ignore_index=True)
colts_df = colts_df.append(colts_df2016, ignore_index=True)
colts_df = colts_df.append(colts_df2015, ignore_index=True)
colts_df = colts_df.append(colts_df2014, ignore_index=True)
colts_df = colts_df.append(colts_df2013, ignore_index=True)
colts_df = colts_df.append(colts_df2012, ignore_index=True)
colts_df = colts_df.append(colts_df2011, ignore_index=True)
colts_df = colts_df.append(colts_df2010, ignore_index=True)

colts_df['Base_Team'] = 'Colts'
colts_df = colts_df.sort_values(by=['year', 'Week'], ascending=[False, True])

colts_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Sun,0.0,0.0,0.0,Jacksonville Jaguars,2020,Colts,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Frank Reich,Nick Sirianni,Matt Eberflus,Philip Rivers
1,2,Sun,1.0,0.0,1.0,Minnesota Vikings,2020,Colts,20.0,27.0,...,0.0,150.0,91.0,445.0,241.0,0,Frank Reich,Nick Sirianni,Matt Eberflus,Philip Rivers
2,3,Sun,1.0,1.0,1.0,New York Jets,2020,Colts,48.0,38.0,...,3.0,245.0,171.0,799.0,416.0,0,Frank Reich,Nick Sirianni,Matt Eberflus,Philip Rivers
3,4,Sun,1.0,2.0,0.0,Chicago Bears,2020,Colts,84.0,45.0,...,6.0,396.0,280.0,1152.0,676.0,0,Frank Reich,Nick Sirianni,Matt Eberflus,Philip Rivers
4,5,Sun,NaN,NaN,0.0,Cleveland Browns,2020,Colts,103.0,56.0,...,7.0,637.0,308.0,1441.0,945.0,0,Frank Reich,Nick Sirianni,Matt Eberflus,Philip Rivers
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,13,Sun,0.0,6.0,1.0,Dallas Cowboys,2010,Colts,282.0,252.0,...,16.0,2249.0,1496.0,4184.0,3745.0,0,Jim Caldwell,Clyde Christensen,Larry Coyer,Peyton Manning
172,14,Thu,1.0,6.0,0.0,Tennessee Titans,2010,Colts,317.0,290.0,...,16.0,2400.0,1713.0,4589.0,4113.0,0,Jim Caldwell,Clyde Christensen,Larry Coyer,Peyton Manning
173,15,Sun,1.0,7.0,1.0,Jacksonville Jaguars,2010,Colts,347.0,318.0,...,18.0,2644.0,1834.0,4988.0,4478.0,0,Jim Caldwell,Clyde Christensen,Larry Coyer,Peyton Manning
174,16,Sun,1.0,8.0,0.0,Oakland Raiders,2010,Colts,381.0,342.0,...,20.0,2933.0,1901.0,5364.0,4834.0,0,Jim Caldwell,Clyde Christensen,Larry Coyer,Peyton Manning


In [106]:
colts_df.to_csv('Resources/indv_team_csvs/colts.csv')

### Broncos Data, 2010-2019

In [107]:
# Data Source: https://www.pro-football-reference.com/teams/den/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

broncos_df2010 = pd.read_csv('Resources/Broncos_Raw/broncos_sportsref_download2010.csv')
broncos_df2011 = pd.read_csv('Resources/Broncos_Raw/broncos_sportsref_download2011.csv')
broncos_df2012 = pd.read_csv('Resources/Broncos_Raw/broncos_sportsref_download2012.csv')
broncos_df2013 = pd.read_csv('Resources/Broncos_Raw/broncos_sportsref_download2013.csv')
broncos_df2014 = pd.read_csv('Resources/Broncos_Raw/broncos_sportsref_download2014.csv')
broncos_df2015 = pd.read_csv('Resources/Broncos_Raw/broncos_sportsref_download2015.csv')
broncos_df2016 = pd.read_csv('Resources/Broncos_Raw/broncos_sportsref_download2016.csv')
broncos_df2017 = pd.read_csv('Resources/Broncos_Raw/broncos_sportsref_download2017.csv')
broncos_df2018 = pd.read_csv('Resources/Broncos_Raw/broncos_sportsref_download2018.csv')
broncos_df2019 = pd.read_csv('Resources/Broncos_Raw/broncos_sportsref_download2019.csv')
broncos_2020 = pd.read_csv('Resources/2020data/broncos_sportsref_download2020.csv')

broncos_df2010 = yearly_data(df=broncos_df2010, year=2010, 
                        HC='Josh McDaniels', HC2='Eric Studesville', HC2_range=range(14,18,1), 
                        OC='Mike McCoy', 
                        DC='Don Martindale', 
                        QB='Kyle Orton', 
                        QB2='Tim Tebow', QB2_range=range(15,18,1))

broncos_df2011 = yearly_data(df=broncos_df2011, year=2011, 
                        HC='John Fox', 
                        OC='Mike McCoy', 
                        DC='Dennis Allen', 
                        QB='Tim Tebow', 
                        QB2='Kyle Orton', QB2_range=range(1,6,1))

broncos_df2012 = yearly_data(df=broncos_df2012, year=2012, 
                        HC='John Fox', 
                        OC='Mike McCoy', 
                        DC='Jack Del Rio',
                        QB='Peyton Manning')

broncos_df2013 = yearly_data(df=broncos_df2013, year=2013, 
                        HC='John Fox', 
                        OC='Adam Gase', 
                        DC='Jack Del Rio', 
                        QB='Peyton Manning')

broncos_df2014 = yearly_data(df=broncos_df2014, year=2014, 
                        HC='John Fox', 
                        OC='Adam Gase', 
                        DC='Jack Del Rio', 
                        QB='Peyton Manning')

broncos_df2015 = yearly_data(df=broncos_df2015, year=2015, 
                        HC='Gary Kubiak', 
                        OC='Rick Dennison', 
                        DC='Wade Phillips', 
                        QB='Peyton Manning',
                        QB2='Brock Osweiler', QB2_range=range(11,18,1))

broncos_df2016 = yearly_data(df=broncos_df2016, year=2016, 
                        HC='Gary Kubiak', 
                        OC='Rick Dennison', 
                        DC='Wade Phillips', 
                        QB='Trevor Siemian',
                        QB2='Paxton Lynch', QB2_range=5)
broncos_df2016.loc[broncos_df2016['Week'] == 13, 'QB'] = "Paxton Lynch"

broncos_df2017 = yearly_data(df=broncos_df2017, year=2017, 
                        HC='Vance Joseph', 
                        OC='Mike McCoy', 
                        DC='Joe Woods', 
                        QB='Trevor Siemian',
                        QB2='Brock Osweiler', QB2_range=range(9,12,1))
broncos_df2017.loc[broncos_df2017['Week'] == 12, 'QB'] = "Paxton Lynch"
broncos_df2017.loc[broncos_df2017['Week'] == 16, 'QB'] = "Brock Osweiler"
broncos_df2017.loc[broncos_df2017['Week'] == 17, 'QB'] = "Paxton Lynch"
broncos_df2017.loc[broncos_df2017['Week'] == range(12,18,1), 'OC'] = "Bill Musgrave"

broncos_df2018 = yearly_data(df=broncos_df2018, year=2018, 
                        HC='Vance Joseph', 
                        OC='Bill Musgrave', 
                        DC='Joe Woods', 
                        QB='Case Keenum')

broncos_df2019 = yearly_data(df=broncos_df2019, year=2019, 
                        HC='Vic Fangio', 
                        OC='Rich Scangarello', 
                        DC='Ed Donatell', 
                        QB='Joe Flacco',
                        QB2='Drew Lock', QB2_range=range(13,18,1))
broncos_df2019.loc[broncos_df2019['Week'] == range(9,13,1), 'QB'] = "Brandon Allen"

broncos_2020 = yearly_data(df=broncos_2020, year=2020, 
                        HC='Vic Fangio', 
                        OC='Pat Shurmur', 
                        DC='Ed Donatell', 
                        QB='Drew Lock', 
                        QB2='Jeff Driskel', QB2_range=3)
broncos_2020.loc[broncos_2020['Week'] == 4, 'QB'] = "Brett Rypien"

# Make master dataframe for team
broncos_df = broncos_2020.append(broncos_df2019, ignore_index=True)
broncos_df = broncos_df.append(broncos_df2018, ignore_index=True)
broncos_df = broncos_df.append(broncos_df2017, ignore_index=True)
broncos_df = broncos_df.append(broncos_df2016, ignore_index=True)
broncos_df = broncos_df.append(broncos_df2015, ignore_index=True)
broncos_df = broncos_df.append(broncos_df2014, ignore_index=True)
broncos_df = broncos_df.append(broncos_df2013, ignore_index=True)
broncos_df = broncos_df.append(broncos_df2012, ignore_index=True)
broncos_df = broncos_df.append(broncos_df2011, ignore_index=True)
broncos_df = broncos_df.append(broncos_df2010, ignore_index=True)

broncos_df['Base_Team'] = 'Broncos'
broncos_df = broncos_df.sort_values(by=['year', 'Week'], ascending=[False, True])

broncos_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Mon,0.0,0.0,1.0,Tennessee Titans,2020,Broncos,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Vic Fangio,Pat Shurmur,Ed Donatell,Drew Lock
1,2,Sun,0.0,0.0,0.0,Pittsburgh Steelers,2020,Broncos,14.0,16.0,...,0.0,247.0,130.0,323.0,377.0,0,Vic Fangio,Pat Shurmur,Ed Donatell,Drew Lock
2,3,Sun,0.0,0.0,1.0,Tampa Bay Buccaneers,2020,Broncos,35.0,42.0,...,2.0,548.0,239.0,642.0,787.0,0,Vic Fangio,Pat Shurmur,Ed Donatell,Jeff Driskel
3,4,Thu,1.0,0.0,0.0,New York Jets,2020,Broncos,45.0,70.0,...,2.0,833.0,307.0,868.0,1140.0,0,Vic Fangio,Pat Shurmur,Ed Donatell,Brett Rypien
4,5,Sun,NaN,NaN,0.0,New England Patriots,2020,Broncos,82.0,98.0,...,2.0,1025.0,436.0,1227.0,1461.0,0,Vic Fangio,Pat Shurmur,Ed Donatell,Drew Lock
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,13,Sun,0.0,3.0,0.0,Kansas City Chiefs,2010,Broncos,250.0,323.0,...,11.0,2629.0,1558.0,4044.0,4187.0,0,Josh McDaniels,Mike McCoy,Don Martindale,Kyle Orton
172,14,Sun,0.0,3.0,0.0,Arizona Cardinals,2010,Broncos,256.0,333.0,...,12.0,2803.0,1743.0,4291.0,4546.0,0,Josh McDaniels,Mike McCoy,Don Martindale,Kyle Orton
173,15,Sun,0.0,3.0,0.0,Oakland Raiders,2010,Broncos,269.0,376.0,...,13.0,2949.0,1954.0,4579.0,4903.0,0,Josh McDaniels,Mike McCoy,Don Martindale,Kyle Orton
174,16,Sun,1.0,3.0,1.0,Houston Texans,2010,Broncos,292.0,415.0,...,16.0,3187.0,2218.0,4814.0,5405.0,0,Josh McDaniels,Mike McCoy,Don Martindale,Kyle Orton


In [108]:
broncos_df.to_csv('Resources/indv_team_csvs/broncos.csv')

### Texans Data, 2010-2019

In [109]:
# Data Source: https://www.pro-football-reference.com/teams/htx/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

texans_df2010 = pd.read_csv('Resources/Texans_Raw/texans_sportsref_download2010.csv')
texans_df2011 = pd.read_csv('Resources/Texans_Raw/texans_sportsref_download2011.csv')
texans_df2012 = pd.read_csv('Resources/Texans_Raw/texans_sportsref_download2012.csv')
texans_df2013 = pd.read_csv('Resources/Texans_Raw/texans_sportsref_download2013.csv')
texans_df2014 = pd.read_csv('Resources/Texans_Raw/texans_sportsref_download2014.csv')
texans_df2015 = pd.read_csv('Resources/Texans_Raw/texans_sportsref_download2015.csv')
texans_df2016 = pd.read_csv('Resources/Texans_Raw/texans_sportsref_download2016.csv')
texans_df2017 = pd.read_csv('Resources/Texans_Raw/texans_sportsref_download2017.csv')
texans_df2018 = pd.read_csv('Resources/Texans_Raw/texans_sportsref_download2018.csv')
texans_df2019 = pd.read_csv('Resources/Texans_Raw/texans_sportsref_download2019.csv')
texans_2020 = pd.read_csv('Resources/2020data/texans_sportsref_download2020.csv')

texans_df2010 = yearly_data(df=texans_df2010, year=2010, 
                        HC='Gary Kubiak', 
                        OC='Rick Dennison', 
                        DC='Frank Bush', 
                        QB='Matt Schaub')

texans_df2011 = yearly_data(df=texans_df2011, year=2011, 
                        HC='Gary Kubiak', 
                        OC='Rick Dennison', 
                        DC='Wade Phillips', 
                        QB='Matt Schaub', 
                        QB2='T. J. Yates', QB2_range=range(13,18,1))
texans_df2011.loc[texans_df2011['Week'] == 1, 'QB'] = "Matt Leinart"

texans_df2012 = yearly_data(df=texans_df2012, year=2012, 
                        HC='Gary Kubiak', 
                        OC='Rick Dennison', 
                        DC='Wade Phillips',
                        QB='Matt Schaub')

texans_df2013 = yearly_data(df=texans_df2013, year=2013, 
                        HC='Gary Kubiak', HC2='Wade Phillips', HC2_range=range(15,18,1), 
                        OC='Rick Dennison', 
                        DC='Wade Phillips', 
                        QB='Matt Schaub', 
                        QB2='Case Keenum', QB2_range=range(7,17,1))

texans_df2014 = yearly_data(df=texans_df2014, year=2014, 
                        HC="Bill O'Brien", 
                        OC="Bill O'Brien", 
                        DC='Romeo Crennel', 
                        QB='Ryan Fitzpatrick', 
                        QB2='Case Keenum', QB2_range=range(16,18,1))
texans_df2014.loc[texans_df2014['Week'] == range(11,13,1), 'QB'] = "Ryan Mallett"

texans_df2015 = yearly_data(df=texans_df2015, year=2015, 
                        HC="Bill O'Brien", 
                        OC='George Godsey', 
                        DC='Romeo Crennel', 
                        QB='Brian Hoyer', 
                        QB2='Ryan Mallett', QB2_range=range(2,6,1))
texans_df2015.loc[texans_df2015['Week'] == 11, 'QB'] = "T. J. Yates"
texans_df2015.loc[texans_df2015['Week'] == 15, 'QB'] = "T. J. Yates"
texans_df2015.loc[texans_df2015['Week'] == 16, 'QB'] = "Brandon Weeden"

texans_df2016 = yearly_data(df=texans_df2016, year=2016, 
                        HC="Bill O'Brien",
                        OC='George Godsey', 
                        DC='Romeo Crennel', 
                        QB='Brock Osweiler', 
                        QB2='Tom Savage', QB2_range=range(16,18,1))

texans_df2017 = yearly_data(df=texans_df2017, year=2017, 
                        HC="Bill O'Brien", 
                        OC="Bill O'Brien", 
                        DC='Mike Vrabel', 
                        QB='Tom Savage', 
                        QB2='Deshaun Watson', QB2_range=range(2,9,1))
texans_df2017.loc[texans_df2017['Week'] == range(15,18,1), 'QB'] = "T. J. Yates"

texans_df2018 = yearly_data(df=texans_df2018, year=2018, 
                        HC="Bill O'Brien", 
                        OC="Bill O'Brien", 
                        DC='Romeo Crennel', 
                        QB='Deshaun Watson')

texans_df2019 = yearly_data(df=texans_df2019, year=2019, 
                        HC="Bill O'Brien", 
                        OC='Tim Kelly', 
                        DC='Romeo Crennel', 
                        QB='Deshaun Watson', 
                        QB2='A. J. McCarron', QB2_range=17)

texans_2020 = yearly_data(df=texans_2020, year=2020, 
                        HC="Bill O'Brien", HC2_range=range(5,18,1), HC2='Romeo Crennel', 
                        OC='Tim Kelly', 
                        DC='Anthony Weaver', 
                        QB='Deshaun Watson')

# Make master dataframe for team
texans_df = texans_2020.append(texans_df2019, ignore_index=True)
texans_df = texans_df.append(texans_df2018, ignore_index=True)
texans_df = texans_df.append(texans_df2017, ignore_index=True)
texans_df = texans_df.append(texans_df2016, ignore_index=True)
texans_df = texans_df.append(texans_df2015, ignore_index=True)
texans_df = texans_df.append(texans_df2014, ignore_index=True)
texans_df = texans_df.append(texans_df2013, ignore_index=True)
texans_df = texans_df.append(texans_df2012, ignore_index=True)
texans_df = texans_df.append(texans_df2011, ignore_index=True)
texans_df = texans_df.append(texans_df2010, ignore_index=True)

texans_df['Base_Team'] = 'Texans'
texans_df = texans_df.sort_values(by=['year', 'Week'], ascending=[False, True])

texans_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Thu,0.0,0.0,0.0,Kansas City Chiefs,2020,Texans,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Bill O'Brien,Tim Kelly,Anthony Weaver,Deshaun Watson
1,2,Sun,0.0,0.0,1.0,Baltimore Ravens,2020,Texans,20.0,34.0,...,0.0,203.0,166.0,360.0,369.0,0,Bill O'Brien,Tim Kelly,Anthony Weaver,Deshaun Watson
2,3,Sun,0.0,0.0,0.0,Pittsburgh Steelers,2020,Texans,36.0,67.0,...,0.0,380.0,396.0,664.0,776.0,0,Bill O'Brien,Tim Kelly,Anthony Weaver,Deshaun Watson
3,4,Sun,0.0,0.0,1.0,Minnesota Vikings,2020,Texans,57.0,95.0,...,0.0,598.0,565.0,924.0,1163.0,0,Bill O'Brien,Tim Kelly,Anthony Weaver,Deshaun Watson
4,5,Sun,NaN,NaN,1.0,Jacksonville Jaguars,2020,Texans,80.0,126.0,...,0.0,846.0,727.0,1310.0,1573.0,0,Bill O'Brien,Tim Kelly,Anthony Weaver,Deshaun Watson
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,13,Thu,0.0,5.0,0.0,Philadelphia Eagles,2010,Texans,264.0,287.0,...,13.0,3148.0,1102.0,4045.0,4250.0,0,Gary Kubiak,Rick Dennison,Frank Bush,Matt Schaub
172,14,Mon,0.0,5.0,1.0,Baltimore Ravens,2010,Texans,288.0,321.0,...,14.0,3449.0,1217.0,4476.0,4666.0,0,Gary Kubiak,Rick Dennison,Frank Bush,Matt Schaub
173,15,Sun,0.0,5.0,0.0,Tennessee Titans,2010,Texans,316.0,355.0,...,14.0,3639.0,1280.0,4965.0,4919.0,0,Gary Kubiak,Rick Dennison,Frank Bush,Matt Schaub
174,16,Sun,0.0,5.0,0.0,Denver Broncos,2010,Texans,333.0,386.0,...,15.0,3851.0,1427.0,5288.0,5278.0,0,Gary Kubiak,Rick Dennison,Frank Bush,Matt Schaub


In [110]:
texans_df.to_csv('Resources/indv_team_csvs/texans.csv')

### Jets Data, 2010-2019

In [123]:
# Data Source: https://www.pro-football-reference.com/teams/nyj/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

jets_df2010 = pd.read_csv('Resources/Jets_Raw/jets_sportsref_download2010.csv')
jets_df2011 = pd.read_csv('Resources/Jets_Raw/jets_sportsref_download2011.csv')
jets_df2012 = pd.read_csv('Resources/Jets_Raw/jets_sportsref_download2012.csv')
jets_df2013 = pd.read_csv('Resources/Jets_Raw/jets_sportsref_download2013.csv')
jets_df2014 = pd.read_csv('Resources/Jets_Raw/jets_sportsref_download2014.csv')
jets_df2015 = pd.read_csv('Resources/Jets_Raw/jets_sportsref_download2015.csv')
jets_df2016 = pd.read_csv('Resources/Jets_Raw/jets_sportsref_download2016.csv')
jets_df2017 = pd.read_csv('Resources/Jets_Raw/jets_sportsref_download2017.csv')
jets_df2018 = pd.read_csv('Resources/Jets_Raw/jets_sportsref_download2018.csv')
jets_df2019 = pd.read_csv('Resources/Jets_Raw/jets_sportsref_download2019.csv')
jets_2020 = pd.read_csv('Resources/2020data/jets_sportsref_download2020.csv')

jets_df2010 = yearly_data(df=jets_df2010, year=2010, 
                        HC='Rex Ryan', 
                        OC='Brian Schottenheimer', 
                        DC='Mike Pettine', 
                        QB='Mark Sanchez')

jets_df2011 = yearly_data(df=jets_df2011, year=2011, 
                        HC='Rex Ryan', 
                        OC='Brian Schottenheimer', 
                        DC='Mike Pettine', 
                        QB='Mark Sanchez')

jets_df2012 = yearly_data(df=jets_df2012, year=2012, 
                        HC='Rex Ryan', 
                        OC='Tony Sparano', 
                        DC='Mike Pettine',
                        QB='Mark Sanchez', 
                        QB2='Greg McElroy', QB2_range=16)

jets_df2013 = yearly_data(df=jets_df2013, year=2013, 
                        HC='Rex Ryan', 
                        OC='Marty Mornhinweg', 
                        DC='Dennis Thurman', 
                        QB='Geno Smith')

jets_df2014 = yearly_data(df=jets_df2014, year=2014, 
                        HC='Rex Ryan', 
                        OC='Marty Mornhinweg', 
                        DC='Dennis Thurman', 
                        QB='Geno Smith', 
                        QB2='Michael Vick', QB2_range=range(9,13,1))

jets_df2015 = yearly_data(df=jets_df2015, year=2015, 
                        HC='Todd Bowles', 
                        OC='Chan Gailey', 
                        DC='Kacy Rodgers', 
                        QB='Ryan Fitzpatrick')

jets_df2016 = yearly_data(df=jets_df2016, year=2016, 
                        HC='Todd Bowles', 
                        OC='Chan Gailey', 
                        DC='Kacy Rodgers', 
                        QB='Ryan Fitzpatrick', 
                        QB2='Bryce Petty', QB2_range=range(14,17,1))
jets_df2016.loc[jets_df2016['Week'] == 7, 'QB'] = "Geno Smith"
jets_df2016.loc[jets_df2016['Week'] == 10, 'QB'] = "Bryce Petty"

jets_df2017 = yearly_data(df=jets_df2017, year=2017, 
                        HC='Todd Bowles', 
                        OC='John Morton', 
                        DC='Kacy Rodgers', 
                        QB='Josh McCown', 
                        QB2='Bryce Petty', QB2_range=range(15,18,1))

jets_df2018 = yearly_data(df=jets_df2018, year=2018, 
                        HC='Todd Bowles', 
                        OC='Jeremy Bates', 
                        DC='Kacy Rodgers', 
                        QB='Sam Darnold', 
                        QB2='Josh McCown', QB2_range=range(10,14,1))

jets_df2019 = yearly_data(df=jets_df2019, year=2019, 
                        HC='Adam Gase', 
                        OC='Dowell Loggains', 
                        DC='Gregg Williams', 
                        QB='Sam Darnold', 
                        QB2='Luke Falk', QB2_range=range(3,6,2))
jets_df2019.loc[jets_df2019['Week'] == 1, 'QB'] = "Trevor Siemian"

jets_2020 = yearly_data(df=jets_2020, year=2020, 
                        HC='Adam Gase', 
                        OC='Dowell Loggains', 
                        DC='Gregg Williams', 
                        QB='Sam Darnold')

# Make master dataframe for team
jets_df = jets_2020.append(jets_df2019, ignore_index=True)
jets_df = jets_df.append(jets_df2018, ignore_index=True)
jets_df = jets_df.append(jets_df2017, ignore_index=True)
jets_df = jets_df.append(jets_df2016, ignore_index=True)
jets_df = jets_df.append(jets_df2015, ignore_index=True)
jets_df = jets_df.append(jets_df2014, ignore_index=True)
jets_df = jets_df.append(jets_df2013, ignore_index=True)
jets_df = jets_df.append(jets_df2012, ignore_index=True)
jets_df = jets_df.append(jets_df2011, ignore_index=True)
jets_df = jets_df.append(jets_df2010, ignore_index=True)

jets_df['Base_Team'] = 'Jets'
jets_df = jets_df.sort_values(by=['year', 'Week'], ascending=[False, True])

jets_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Sun,0.0,0.0,0.0,Buffalo Bills,2020,Jets,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Adam Gase,Dowell Loggains,Gregg Williams,Sam Darnold
1,2,Sun,0.0,0.0,1.0,San Francisco 49ers,2020,Jets,17.0,27.0,...,2.0,306.0,98.0,254.0,404.0,0,Adam Gase,Dowell Loggains,Gregg Williams,Sam Darnold
2,3,Sun,0.0,0.0,0.0,Indianapolis Colts,2020,Jets,30.0,58.0,...,3.0,483.0,280.0,531.0,763.0,0,Adam Gase,Dowell Loggains,Gregg Williams,Sam Darnold
3,4,Thu,0.0,0.0,1.0,Denver Broncos,2020,Jets,37.0,94.0,...,3.0,717.0,399.0,791.0,1116.0,0,Adam Gase,Dowell Loggains,Gregg Williams,Sam Darnold
4,5,Sun,NaN,NaN,1.0,Arizona Cardinals,2020,Jets,65.0,131.0,...,6.0,959.0,516.0,1112.0,1475.0,0,Adam Gase,Dowell Loggains,Gregg Williams,Sam Darnold
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,13,Mon,0.0,9.0,0.0,New England Patriots,2010,Jets,264.0,187.0,...,20.0,2317.0,949.0,3978.0,3266.0,0,Rex Ryan,Brian Schottenheimer,Mike Pettine,Mark Sanchez
172,14,Sun,0.0,9.0,1.0,Miami Dolphins,2010,Jets,267.0,232.0,...,20.0,2621.0,1050.0,4279.0,3671.0,0,Rex Ryan,Brian Schottenheimer,Mike Pettine,Mark Sanchez
173,15,Sun,1.0,9.0,0.0,Pittsburgh Steelers,2010,Jets,273.0,242.0,...,23.0,2651.0,1151.0,4559.0,3802.0,0,Rex Ryan,Brian Schottenheimer,Mike Pettine,Mark Sanchez
174,16,Sun,0.0,10.0,0.0,Chicago Bears,2010,Jets,295.0,259.0,...,23.0,2883.0,1297.0,4835.0,4180.0,0,Rex Ryan,Brian Schottenheimer,Mike Pettine,Mark Sanchez


In [124]:
jets_df.to_csv('Resources/indv_team_csvs/jets.csv')

### Jaguars Data, 2010-2019

In [113]:
# Data Source: https://www.pro-football-reference.com/teams/jax/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

jaguars_df2010 = pd.read_csv('Resources/Jaguars_Raw/jaguars_sportsref_download2010.csv')
jaguars_df2011 = pd.read_csv('Resources/Jaguars_Raw/jaguars_sportsref_download2011.csv')
jaguars_df2012 = pd.read_csv('Resources/Jaguars_Raw/jaguars_sportsref_download2012.csv')
jaguars_df2013 = pd.read_csv('Resources/Jaguars_Raw/jaguars_sportsref_download2013.csv')
jaguars_df2014 = pd.read_csv('Resources/Jaguars_Raw/jaguars_sportsref_download2014.csv')
jaguars_df2015 = pd.read_csv('Resources/Jaguars_Raw/jaguars_sportsref_download2015.csv')
jaguars_df2016 = pd.read_csv('Resources/Jaguars_Raw/jaguars_sportsref_download2016.csv')
jaguars_df2017 = pd.read_csv('Resources/Jaguars_Raw/jaguars_sportsref_download2017.csv')
jaguars_df2018 = pd.read_csv('Resources/Jaguars_Raw/jaguars_sportsref_download2018.csv')
jaguars_df2019 = pd.read_csv('Resources/Jaguars_Raw/jaguars_sportsref_download2019.csv')
jaguars_2020 = pd.read_csv('Resources/2020data/jaguars_sportsref_download2020.csv')

jaguars_df2010 = yearly_data(df=jaguars_df2010, year=2010, 
                        HC='Jack Del Rio', 
                        OC='Dirk Koetter', 
                        DC='Mel Tucker', 
                        QB='David Garrard',
                        QB2='Todd Bouman', QB2_range=7)
jaguars_df2010.loc[jaguars_df2010['Week'] == 17, 'QB'] = "Trent Edwards"

jaguars_df2011 = yearly_data(df=jaguars_df2011, year=2011, 
                        HC='Jack Del Rio', HC2='Mel Tucker', HC2_range=range(13,18,1),
                        OC='Dirk Koetter', 
                        DC='Mel Tucker', 
                        QB='Blaine Gabbert',
                        QB2='Luke McCown', QB2_range=range(1,3,1))

jaguars_df2012 = yearly_data(df=jaguars_df2012, year=2012, 
                        HC='Mike Mularkey', 
                        OC='Bob Bratkowski', 
                        DC='Mel Tucker',
                        QB='Blaine Gabbert',
                        QB2='Chad Henne', QB2_range=range(12,18,1))

jaguars_df2013 = yearly_data(df=jaguars_df2013, year=2013, 
                        HC='Gus Bradley', 
                        OC='Jedd Fisch', 
                        DC='Bob Babich', 
                        QB='Chad Henne',
                        QB2='Blaine Gabbert', QB2_range=range(1,5,3))

jaguars_df2014 = yearly_data(df=jaguars_df2014, year=2014, 
                        HC='Gus Bradley', 
                        OC='Jedd Fisch', 
                        DC='Bob Babich', 
                        QB='Blake Bortles',
                        QB2='Chad Henne', QB2_range=range(1,4,1))

jaguars_df2015 = yearly_data(df=jaguars_df2015, year=2015, 
                        HC='Gus Bradley', 
                        OC='Greg Olson', 
                        DC='Bob Babich', 
                        QB='Blake Bortles')

jaguars_df2016 = yearly_data(df=jaguars_df2016, year=2016, 
                        HC='Gus Bradley', HC2='Doug Marrone', HC2_range=range(16,18,1),
                        OC='Greg Olson', 
                        DC="Todd Wash", 
                        QB='Blake Bortles')
jaguars_df2016.loc[jaguars_df2016['Week'] == range(16,18,1), 'OC'] = "Nathaniel Hackett"

jaguars_df2017 = yearly_data(df=jaguars_df2017, year=2017, 
                        HC='Doug Marrone', 
                        OC='Nathaniel Hackett', 
                        DC='Todd Wash', 
                        QB='Blake Bortles')

jaguars_df2018 = yearly_data(df=jaguars_df2018, year=2018, 
                        HC='Doug Marrone', 
                        OC='Nathaniel Hackett', 
                        DC='Todd Wash', 
                        QB='Blake Bortles',
                        QB2='Cody Kessler', QB2_range=range(13,17,1))

jaguars_df2019 = yearly_data(df=jaguars_df2019, year=2019, 
                        HC='Doug Marrone', 
                        OC='John DeFilippo', 
                        DC='Todd Wash', 
                        QB='Gardner Minshew',
                        QB2='Nick Foles', QB2_range=range(1,3,1))
jaguars_df2019.loc[jaguars_df2019['Week'] == range(12,14,1), 'QB'] = "Nick Foles"

jaguars_2020 = yearly_data(df=jaguars_2020, year=2020, 
                        HC='Doug Marrone', 
                        OC='Jay Gruden', 
                        DC='Todd Wash', 
                        QB='Gardner Minshew')

# Make master dataframe for team
jaguars_df = jaguars_2020.append(jaguars_df2019, ignore_index=True)
jaguars_df = jaguars_df.append(jaguars_df2018, ignore_index=True)
jaguars_df = jaguars_df.append(jaguars_df2017, ignore_index=True)
jaguars_df = jaguars_df.append(jaguars_df2016, ignore_index=True)
jaguars_df = jaguars_df.append(jaguars_df2015, ignore_index=True)
jaguars_df = jaguars_df.append(jaguars_df2014, ignore_index=True)
jaguars_df = jaguars_df.append(jaguars_df2013, ignore_index=True)
jaguars_df = jaguars_df.append(jaguars_df2012, ignore_index=True)
jaguars_df = jaguars_df.append(jaguars_df2011, ignore_index=True)
jaguars_df = jaguars_df.append(jaguars_df2010, ignore_index=True)

jaguars_df['Base_Team'] = 'Jaguars'
jaguars_df = jaguars_df.sort_values(by=['year', 'Week'], ascending=[False, True])

jaguars_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Sun,1.0,0.0,1.0,Indianapolis Colts,2020,Jaguars,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Doug Marrone,Jay Gruden,Todd Wash,Gardner Minshew
1,2,Sun,0.0,1.0,0.0,Tennessee Titans,2020,Jaguars,27.0,20.0,...,2.0,357.0,88.0,241.0,445.0,0,Doug Marrone,Jay Gruden,Todd Wash,Gardner Minshew
2,3,Thu,0.0,1.0,1.0,Miami Dolphins,2020,Jaguars,57.0,53.0,...,2.0,588.0,211.0,721.0,799.0,0,Doug Marrone,Jay Gruden,Todd Wash,Gardner Minshew
3,4,Sun,0.0,1.0,0.0,Cincinnati Bengals,2020,Jaguars,70.0,84.0,...,2.0,744.0,349.0,1039.0,1093.0,0,Doug Marrone,Jay Gruden,Todd Wash,Gardner Minshew
4,5,Sun,NaN,NaN,0.0,Houston Texans,2020,Jaguars,95.0,117.0,...,3.0,1044.0,554.0,1468.0,1598.0,0,Doug Marrone,Jay Gruden,Todd Wash,Gardner Minshew
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,13,Sun,1.0,6.0,0.0,Tennessee Titans,2010,Jaguars,240.0,294.0,...,13.0,2872.0,1257.0,3684.0,4129.0,0,Jack Del Rio,Dirk Koetter,Mel Tucker,David Garrard
172,14,Sun,1.0,7.0,1.0,Oakland Raiders,2010,Jaguars,257.0,300.0,...,15.0,3035.0,1314.0,4061.0,4349.0,0,Jack Del Rio,Dirk Koetter,Mel Tucker,David Garrard
173,15,Sun,0.0,8.0,0.0,Indianapolis Colts,2010,Jaguars,295.0,331.0,...,17.0,3358.0,1467.0,4446.0,4825.0,0,Jack Del Rio,Dirk Koetter,Mel Tucker,David Garrard
174,16,Sun,0.0,8.0,1.0,Washington Redskins,2010,Jaguars,319.0,365.0,...,17.0,3579.0,1622.0,4802.0,5201.0,0,Jack Del Rio,Dirk Koetter,Mel Tucker,David Garrard


In [114]:
jaguars_df.to_csv('Resources/indv_team_csvs/jaguars.csv')

### (30) Dolphins Data, 2010-2019

In [115]:
# Data Source: https://www.pro-football-reference.com/teams/mia/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

dolphins_df2010 = pd.read_csv('Resources/Dolphins_Raw/dolphins_sportsref_download2010.csv')
dolphins_df2011 = pd.read_csv('Resources/Dolphins_Raw/dolphins_sportsref_download2011.csv')
dolphins_df2012 = pd.read_csv('Resources/Dolphins_Raw/dolphins_sportsref_download2012.csv')
dolphins_df2013 = pd.read_csv('Resources/Dolphins_Raw/dolphins_sportsref_download2013.csv')
dolphins_df2014 = pd.read_csv('Resources/Dolphins_Raw/dolphins_sportsref_download2014.csv')
dolphins_df2015 = pd.read_csv('Resources/Dolphins_Raw/dolphins_sportsref_download2015.csv')
dolphins_df2016 = pd.read_csv('Resources/Dolphins_Raw/dolphins_sportsref_download2016.csv')
dolphins_df2017 = pd.read_csv('Resources/Dolphins_Raw/dolphins_sportsref_download2017.csv')
dolphins_df2018 = pd.read_csv('Resources/Dolphins_Raw/dolphins_sportsref_download2018.csv')
dolphins_df2019 = pd.read_csv('Resources/Dolphins_Raw/dolphins_sportsref_download2019.csv')
dolphins_2020 = pd.read_csv('Resources/2020data/dolphins_sportsref_download2020.csv')

dolphins_df2010 = yearly_data(df=dolphins_df2010, year=2010, 
                        HC='Tony Sparano', 
                        OC='Dan Henning', 
                        DC='Mike Nolan', 
                        QB='Chad Henne',
                        QB2='Tyler Thigpen', QB2_range=11)
dolphins_df2010.loc[dolphins_df2010['Week'] == 10, 'QB'] = "Chad Pennington"

dolphins_df2011 = yearly_data(df=dolphins_df2011, year=2011, 
                        HC='Tony Sparano', 
                        OC='Brian Daboll', 
                        DC='Mike Nolan', 
                        QB='Chad Henne',
                        QB2='Matt Moore', QB2_range=range(1,5,1))

dolphins_df2012 = yearly_data(df=dolphins_df2012, year=2012, 
                        HC='Joe Philbin', 
                        OC='Mike Sherman', 
                        DC='Kevin Coyle',
                        QB='Ryan Tannehill')

dolphins_df2013 = yearly_data(df=dolphins_df2013, year=2013, 
                        HC='Joe Philbin', 
                        OC='Mike Sherman', 
                        DC='Kevin Coyle', 
                        QB='Ryan Tannehill')

dolphins_df2014 = yearly_data(df=dolphins_df2014, year=2014, 
                        HC='Joe Philbin', 
                        OC='Bill Lazor', 
                        DC='Kevin Coyle', 
                        QB='Ryan Tannehill')

dolphins_df2015 = yearly_data(df=dolphins_df2015, year=2015, 
                        HC='Joe Philbin', HC2='Dan Campbell', HC2_range=range(6,18,1), 
                        OC='Bill Lazor', 
                        DC='Kevin Coyle', 
                        QB='Ryan Tannehill')
dolphins_df2015.loc[dolphins_df2015['Week'] == range(6,18,1), 'OC'] = "Zac Taylor"
dolphins_df2015.loc[dolphins_df2015['Week'] == range(6,18,1), 'DC'] = "Lou Anarumo"

dolphins_df2016 = yearly_data(df=dolphins_df2016, year=2016, 
                        HC='Adam Gase',
                        OC='Clyde Christensen', 
                        DC="Vance Joseph", 
                        QB='Ryan Tannehill',
                        QB2='Matt Moore', QB2_range=range(15,18,1))

dolphins_df2017 = yearly_data(df=dolphins_df2017, year=2017, 
                        HC='Adam Gase',
                        OC='Clyde Christensen', 
                        DC='Matt Burke', 
                        QB='Jay Cutler',
                        QB2='Matt Moore', QB2_range=range(8,13,4))

dolphins_df2018 = yearly_data(df=dolphins_df2018, year=2018, 
                        HC='Adam Gase',
                        OC='Dowell Loggains', 
                        DC='Matt Burke', 
                        QB='Ryan Tannehill',
                        QB2='Brock Osweiler', QB2_range=range(6,12,1))

dolphins_df2019 = yearly_data(df=dolphins_df2019, year=2019, 
                        HC='Brian Flores', 
                        OC="Chad O'Shea", 
                        DC='Patrick Graham', 
                        QB='Ryan Fitzpatrick',
                        QB2='Josh Rosen', QB2_range=range(3,7,1))

dolphins_2020 = yearly_data(df=dolphins_2020, year=2020, 
                        HC='Brian Flores', 
                        OC='Chan Gailey', 
                        DC='Josh Boyer', 
                        QB='Ryan Fitzpatrick')

# Make master dataframe for team
dolphins_df = dolphins_2020.append(dolphins_df2019, ignore_index=True)
dolphins_df = dolphins_df.append(dolphins_df2018, ignore_index=True)
dolphins_df = dolphins_df.append(dolphins_df2017, ignore_index=True)
dolphins_df = dolphins_df.append(dolphins_df2016, ignore_index=True)
dolphins_df = dolphins_df.append(dolphins_df2015, ignore_index=True)
dolphins_df = dolphins_df.append(dolphins_df2014, ignore_index=True)
dolphins_df = dolphins_df.append(dolphins_df2013, ignore_index=True)
dolphins_df = dolphins_df.append(dolphins_df2012, ignore_index=True)
dolphins_df = dolphins_df.append(dolphins_df2011, ignore_index=True)
dolphins_df = dolphins_df.append(dolphins_df2010, ignore_index=True)

dolphins_df['Base_Team'] = 'Dolphins'
dolphins_df = dolphins_df.sort_values(by=['year', 'Week'], ascending=[False, True])

dolphins_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Sun,0.0,0.0,0.0,New England Patriots,2020,Dolphins,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Brian Flores,Chan Gailey,Josh Boyer,Ryan Fitzpatrick
1,2,Sun,0.0,0.0,1.0,Buffalo Bills,2020,Dolphins,11.0,21.0,...,1.0,140.0,217.0,269.0,357.0,0,Brian Flores,Chan Gailey,Josh Boyer,Ryan Fitzpatrick
2,3,Thu,1.0,0.0,0.0,Jacksonville Jaguars,2020,Dolphins,39.0,52.0,...,2.0,551.0,329.0,679.0,880.0,0,Brian Flores,Chan Gailey,Josh Boyer,Ryan Fitzpatrick
3,4,Sun,0.0,1.0,1.0,Seattle Seahawks,2020,Dolphins,70.0,65.0,...,4.0,797.0,401.0,973.0,1198.0,0,Brian Flores,Chan Gailey,Josh Boyer,Ryan Fitzpatrick
4,5,Sun,NaN,NaN,0.0,San Francisco 49ers,2020,Dolphins,93.0,96.0,...,5.0,1140.0,499.0,1388.0,1639.0,0,Brian Flores,Chan Gailey,Josh Boyer,Ryan Fitzpatrick
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,13,Sun,0.0,6.0,1.0,Cleveland Browns,2010,Dolphins,205.0,225.0,...,15.0,2215.0,1143.0,3757.0,3358.0,0,Tony Sparano,Dan Henning,Mike Nolan,Chad Henne
172,14,Sun,1.0,6.0,0.0,New York Jets,2010,Dolphins,215.0,238.0,...,15.0,2415.0,1195.0,4038.0,3610.0,0,Tony Sparano,Dan Henning,Mike Nolan,Chad Henne
173,15,Sun,0.0,7.0,1.0,Buffalo Bills,2010,Dolphins,225.0,244.0,...,17.0,2608.0,1282.0,4169.0,3890.0,0,Tony Sparano,Dan Henning,Mike Nolan,Chad Henne
174,16,Sun,0.0,7.0,1.0,Detroit Lions,2010,Dolphins,239.0,261.0,...,18.0,2819.0,1353.0,4495.0,4172.0,0,Tony Sparano,Dan Henning,Mike Nolan,Chad Henne


In [116]:
dolphins_df.to_csv('Resources/indv_team_csvs/dolphins.csv')

### Patriots Data, 2010-2019

In [117]:
# Data Source: https://www.pro-football-reference.com/teams/nwe/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

pats_df2010 = pd.read_csv('Resources/Pats_Raw/pats_sportsref_download2010.csv')
pats_df2011 = pd.read_csv('Resources/Pats_Raw/pats_sportsref_download2011.csv')
pats_df2012 = pd.read_csv('Resources/Pats_Raw/pats_sportsref_download2012.csv')
pats_df2013 = pd.read_csv('Resources/Pats_Raw/pats_sportsref_download2013.csv')
pats_df2014 = pd.read_csv('Resources/Pats_Raw/pats_sportsref_download2014.csv')
pats_df2015 = pd.read_csv('Resources/Pats_Raw/pats_sportsref_download2015.csv')
pats_df2016 = pd.read_csv('Resources/Pats_Raw/pats_sportsref_download2016.csv')
pats_df2017 = pd.read_csv('Resources/Pats_Raw/pats_sportsref_download2017.csv')
pats_df2018 = pd.read_csv('Resources/Pats_Raw/pats_sportsref_download2018.csv')
pats_df2019 = pd.read_csv('Resources/Pats_Raw/pats_sportsref_download2019.csv')
pats_2020 = pd.read_csv('Resources/2020data/patriots_sportsref_download2020.csv')

pats_df2010 = yearly_data(df=pats_df2010, year=2010, 
                        HC='Bill Belichick', 
                        OC='Bill Belichick', 
                        DC='Bill Belichick', 
                        QB='Tom Brady')

pats_df2011 = yearly_data(df=pats_df2011, year=2011, 
                        HC='Bill Belichick', 
                        OC="Bill O'Brien", 
                        DC='Bill Belichick', 
                        QB='Tom Brady')

pats_df2012 = yearly_data(df=pats_df2012, year=2012, 
                        HC='Bill Belichick', 
                        OC='Josh McDaniels', 
                        DC='Matt Patricia',
                        QB='Tom Brady')

pats_df2013 = yearly_data(df=pats_df2013, year=2013, 
                        HC='Bill Belichick', 
                        OC='Josh McDaniels', 
                        DC='Matt Patricia', 
                        QB='Tom Brady')

pats_df2014 = yearly_data(df=pats_df2014, year=2014, 
                        HC='Bill Belichick', 
                        OC='Josh McDaniels', 
                        DC='Matt Patricia', 
                        QB='Tom Brady')

pats_df2015 = yearly_data(df=pats_df2015, year=2015, 
                        HC='Bill Belichick', 
                        OC='Josh McDaniels', 
                        DC='Matt Patricia', 
                        QB='Tom Brady')

pats_df2016 = yearly_data(df=pats_df2016, year=2016, 
                        HC='Bill Belichick', 
                        OC='Josh McDaniels', 
                        DC='Matt Patricia', 
                        QB='Tom Brady', 
                        QB2='Jimmy Garoppolo', QB2_range=range(1,3,1))
pats_df2016.loc[pats_df2016['Week'] == range(3,5,1), 'QB'] = "Jacoby Brissett"

pats_df2017 = yearly_data(df=pats_df2017, year=2017, 
                        HC='Bill Belichick', 
                        OC='Josh McDaniels', 
                        DC='Matt Patricia', 
                        QB='Tom Brady')

pats_df2018 = yearly_data(df=pats_df2018, year=2018, 
                        HC='Bill Belichick', 
                        OC='Josh McDaniels', 
                        DC='Bill Belichick', 
                        QB='Tom Brady')

pats_df2019 = yearly_data(df=pats_df2019, year=2019, 
                        HC='Bill Belichick', 
                        OC='Josh McDaniels', 
                        DC='Bill Belichick', 
                        QB='Tom Brady')

pats_2020 = yearly_data(df=pats_2020, year=2020, 
                        HC='Bill Belichick', 
                        OC='Josh McDaniels', 
                        DC='Bill Belichick', 
                        QB='Cam Newton', 
                        QB2='Brian Hoyer', QB2_range=(4,6,1))

# Make master dataframe for team
pats_df = pats_2020.append(pats_df2019, ignore_index=True)
pats_df = pats_df.append(pats_df2018, ignore_index=True)
pats_df = pats_df.append(pats_df2017, ignore_index=True)
pats_df = pats_df.append(pats_df2016, ignore_index=True)
pats_df = pats_df.append(pats_df2015, ignore_index=True)
pats_df = pats_df.append(pats_df2014, ignore_index=True)
pats_df = pats_df.append(pats_df2013, ignore_index=True)
pats_df = pats_df.append(pats_df2012, ignore_index=True)
pats_df = pats_df.append(pats_df2011, ignore_index=True)
pats_df = pats_df.append(pats_df2010, ignore_index=True)

pats_df['Base_Team'] = 'Patriots'
pats_df = pats_df.sort_values(by=['year', 'Week'], ascending=[False, True])

pats_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Sun,1.0,0.0,1.0,Miami Dolphins,2020,Patriots,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Bill Belichick,Josh McDaniels,Bill Belichick,Cam Newton
1,2,Sun,0.0,1.0,0.0,Seattle Seahawks,2020,Patriots,21.0,11.0,...,3.0,182.0,87.0,357.0,269.0,0,Bill Belichick,Josh McDaniels,Bill Belichick,Cam Newton
2,3,Sun,1.0,1.0,1.0,Las Vegas Raiders,2020,Patriots,51.0,46.0,...,4.0,457.0,241.0,821.0,698.0,0,Bill Belichick,Josh McDaniels,Bill Belichick,Cam Newton
3,4,Mon,0.0,2.0,0.0,Kansas City Chiefs,2020,Patriots,87.0,66.0,...,7.0,706.0,367.0,1227.0,1073.0,0,Bill Belichick,Josh McDaniels,Bill Belichick,Cam Newton
4,5,Sun,NaN,NaN,1.0,Denver Broncos,2020,Patriots,97.0,92.0,...,8.0,935.0,461.0,1584.0,1396.0,0,Bill Belichick,Josh McDaniels,Bill Belichick,Cam Newton
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,13,Mon,1.0,9.0,1.0,New York Jets,2010,Patriots,334.0,266.0,...,20.0,3173.0,1217.0,3841.0,4390.0,0,Bill Belichick,Bill Belichick,Bill Belichick,Tom Brady
172,14,Sun,1.0,10.0,0.0,Chicago Bears,2010,Patriots,379.0,269.0,...,23.0,3322.0,1369.0,4246.0,4691.0,0,Bill Belichick,Bill Belichick,Bill Belichick,Tom Brady
173,15,Sun,1.0,11.0,1.0,Green Bay Packers,2010,Patriots,415.0,276.0,...,27.0,3460.0,1416.0,4721.0,4876.0,0,Bill Belichick,Bill Belichick,Bill Belichick,Tom Brady
174,16,Sun,1.0,12.0,0.0,Buffalo Bills,2010,Patriots,446.0,303.0,...,29.0,3686.0,1559.0,4970.0,5245.0,0,Bill Belichick,Bill Belichick,Bill Belichick,Tom Brady


In [118]:
pats_df.to_csv('Resources/indv_team_csvs/pats.csv')

### Bills Data, 2010-2019

In [119]:
# Data Source: https://www.pro-football-reference.com/teams/buf/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

bills_df2010 = pd.read_csv('Resources/Bills_Raw/bills_sportsref_download2010.csv')
bills_df2011 = pd.read_csv('Resources/Bills_Raw/bills_sportsref_download2011.csv')
bills_df2012 = pd.read_csv('Resources/Bills_Raw/bills_sportsref_download2012.csv')
bills_df2013 = pd.read_csv('Resources/Bills_Raw/bills_sportsref_download2013.csv')
bills_df2014 = pd.read_csv('Resources/Bills_Raw/bills_sportsref_download2014.csv')
bills_df2015 = pd.read_csv('Resources/Bills_Raw/bills_sportsref_download2015.csv')
bills_df2016 = pd.read_csv('Resources/Bills_Raw/bills_sportsref_download2016.csv')
bills_df2017 = pd.read_csv('Resources/Bills_Raw/bills_sportsref_download2017.csv')
bills_df2018 = pd.read_csv('Resources/Bills_Raw/bills_sportsref_download2018.csv')
bills_df2019 = pd.read_csv('Resources/Bills_Raw/bills_sportsref_download2019.csv')
bills_2020 = pd.read_csv('Resources/2020data/bills_sportsref_download2020.csv')

bills_df2010 = yearly_data(df=bills_df2010, year=2010, 
                        HC='Chan Gailey', 
                        OC='Curtis Modkins', 
                        DC='George Edwards', 
                        QB='Ryan Fitzpatrick', 
                        QB2='Trent Edwards', QB2_range=range(1,3,1))
bills_df2010.loc[bills_df2010['Week'] == 17, 'QB'] = "Brian Brohm"

bills_df2011 = yearly_data(df=bills_df2011, year=2011, 
                        HC='Chan Gailey', 
                        OC='Curtis Modkins', 
                        DC='George Edwards', 
                        QB='Ryan Fitzpatrick')

bills_df2012 = yearly_data(df=bills_df2012, year=2012, 
                        HC='Chan Gailey', 
                        OC='Curtis Modkins', 
                        DC='Dave Wannstedt',
                        QB='Ryan Fitzpatrick')

bills_df2013 = yearly_data(df=bills_df2013, year=2013, 
                        HC='Doug Marrone', 
                        OC='Nathaniel Hackett', 
                        DC='Mike Pettine', 
                        QB='EJ Manuel', 
                        QB2='Thad Lewis', QB2_range=range(6,9,1))
bills_df2013.loc[bills_df2013['Week'] == 9, 'QB'] = "Jeff Tuel"
bills_df2013.loc[bills_df2013['Week'] == range(16,18,1), 'QB'] = "Thad Lewis"

bills_df2014 = yearly_data(df=bills_df2014, year=2014, 
                        HC='Doug Marrone', 
                        OC='Nathaniel Hackett', 
                        DC='Jim Schwartz', 
                        QB='Kyle Orton', 
                        QB2='EJ Manuel', QB2_range=range(1,5,1))

bills_df2015 = yearly_data(df=bills_df2015, year=2015, 
                        HC='Rex Ryan', 
                        OC='Greg Roman', 
                        DC='Dennis Thurman', 
                        QB='Tyrod Taylor', 
                        QB2='EJ Manuel', QB2_range=range(6,8,1))

bills_df2016 = yearly_data(df=bills_df2016, year=2016, 
                        HC='Rex Ryan', HC2='Anthony Lynn', HC2_range=17,
                        OC='Greg Roman', 
                        DC="Dennis Thurman", 
                        QB='Tyrod Taylor', 
                        QB2='EJ Manuel', QB2_range=17)
bills_df2016.loc[bills_df2016['Week'] == range(3,18,1), 'OC'] = "Anthony Lynn"

bills_df2017 = yearly_data(df=bills_df2017, year=2017, 
                        HC='Sean McDermott', 
                        OC='Rick Dennison', 
                        DC='Leslie Frazier', 
                        QB='Tyrod Taylor', 
                        QB2='Nathan Peterman', QB2_range=11)
bills_df2017.loc[bills_df2017['Week'] == 14, 'QB'] = "Nathan Peterman"

bills_df2018 = yearly_data(df=bills_df2018, year=2018, 
                        HC='Sean McDermott', 
                        OC='Brian Daboll', 
                        DC='Leslie Frazier', 
                        QB='Josh Allen', 
                        QB2='Nathan Peterman', QB2_range=1)
bills_df2018.loc[bills_df2018['Week'] == 9, 'QB'] = "Nathan Peterman"
bills_df2018.loc[bills_df2018['Week'] == range(7,9,1), 'QB'] = "Derek Anderson"
bills_df2018.loc[bills_df2018['Week'] == 10, 'QB'] = "Matt Barkley"

bills_df2019 = yearly_data(df=bills_df2019, year=2019, 
                        HC='Sean McDermott', 
                        OC='Brian Daboll', 
                        DC='Leslie Frazier', 
                        QB='Josh Allen')

bills_2020 = yearly_data(df=bills_2020, year=2020, 
                        HC='Sean McDermott', 
                        OC='Brian Daboll', 
                        DC='Leslie Frazier', 
                        QB='Josh Allen')

# Make master dataframe for team
bills_df = bills_2020.append(bills_df2019, ignore_index=True)
bills_df = bills_df.append(bills_df2018, ignore_index=True)
bills_df = bills_df.append(bills_df2017, ignore_index=True)
bills_df = bills_df.append(bills_df2016, ignore_index=True)
bills_df = bills_df.append(bills_df2015, ignore_index=True)
bills_df = bills_df.append(bills_df2014, ignore_index=True)
bills_df = bills_df.append(bills_df2013, ignore_index=True)
bills_df = bills_df.append(bills_df2012, ignore_index=True)
bills_df = bills_df.append(bills_df2011, ignore_index=True)
bills_df = bills_df.append(bills_df2010, ignore_index=True)

bills_df['Base_Team'] = 'Bills'
bills_df = bills_df.sort_values(by=['year', 'Week'], ascending=[False, True])

bills_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Sun,1.0,0.0,1.0,New York Jets,2020,Bills,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Sean McDermott,Brian Daboll,Leslie Frazier,Josh Allen
1,2,Sun,1.0,1.0,0.0,Miami Dolphins,2020,Bills,27.0,17.0,...,2.0,202.0,52.0,404.0,254.0,0,Sean McDermott,Brian Daboll,Leslie Frazier,Josh Allen
2,3,Sun,1.0,2.0,1.0,Los Angeles Rams,2020,Bills,58.0,45.0,...,2.0,513.0,151.0,927.0,664.0,0,Sean McDermott,Brian Daboll,Leslie Frazier,Josh Allen
3,4,Sun,1.0,3.0,0.0,Las Vegas Raiders,2020,Bills,93.0,77.0,...,4.0,824.0,318.0,1302.0,1142.0,0,Sean McDermott,Brian Daboll,Leslie Frazier,Josh Allen
4,5,Sun,NaN,NaN,0.0,Tennessee Titans,2020,Bills,123.0,100.0,...,6.0,1121.0,404.0,1638.0,1525.0,0,Sean McDermott,Brian Daboll,Leslie Frazier,Josh Allen
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,13,Sun,0.0,2.0,0.0,Minnesota Vikings,2010,Bills,229.0,295.0,...,12.0,2309.0,1841.0,3503.0,4150.0,0,Chan Gailey,Curtis Modkins,George Edwards,Ryan Fitzpatrick
172,14,Sun,1.0,2.0,1.0,Cleveland Browns,2010,Bills,243.0,333.0,...,16.0,2486.0,2051.0,3742.0,4537.0,0,Chan Gailey,Curtis Modkins,George Edwards,Ryan Fitzpatrick
173,15,Sun,1.0,3.0,0.0,Miami Dolphins,2010,Bills,256.0,339.0,...,19.0,2568.0,2156.0,4065.0,4724.0,0,Chan Gailey,Curtis Modkins,George Edwards,Ryan Fitzpatrick
174,16,Sun,0.0,4.0,1.0,New England Patriots,2010,Bills,273.0,353.0,...,21.0,2829.0,2221.0,4347.0,5050.0,0,Chan Gailey,Curtis Modkins,George Edwards,Ryan Fitzpatrick


In [120]:
bills_df.to_csv('Resources/indv_team_csvs/bills.csv')

### Titans Data, 2010-2019

In [121]:
# Data Source: https://www.pro-football-reference.com/teams/oti/
# Years selected and then data downloaded as xls files
# In Excel, cells were reformatted to be compatible with a csv format

titans_df2010 = pd.read_csv('Resources/Titans_Raw/titans_sportsref_download2010.csv')
titans_df2011 = pd.read_csv('Resources/Titans_Raw/titans_sportsref_download2011.csv')
titans_df2012 = pd.read_csv('Resources/Titans_Raw/titans_sportsref_download2012.csv')
titans_df2013 = pd.read_csv('Resources/Titans_Raw/titans_sportsref_download2013.csv')
titans_df2014 = pd.read_csv('Resources/Titans_Raw/titans_sportsref_download2014.csv')
titans_df2015 = pd.read_csv('Resources/Titans_Raw/titans_sportsref_download2015.csv')
titans_df2016 = pd.read_csv('Resources/Titans_Raw/titans_sportsref_download2016.csv')
titans_df2017 = pd.read_csv('Resources/Titans_Raw/titans_sportsref_download2017.csv')
titans_df2018 = pd.read_csv('Resources/Titans_Raw/titans_sportsref_download2018.csv')
titans_df2019 = pd.read_csv('Resources/Titans_Raw/titans_sportsref_download2019.csv')
titans_2020 = pd.read_csv('Resources/2020data/titans_sportsref_download2020.csv')

titans_df2010 = yearly_data(df=titans_df2010, year=2010, 
                        HC='Jeff Fisher', 
                        OC='Mike Heimerdinger', 
                        DC='Chuck Cecil', 
                        QB='Vince Young', 
                        QB2='Kerry Collins',QB2_range=range(13,18,1))
titans_df2010.loc[titans_df2010['Week'] == 12, 'QB'] = "Rusty Smith"
titans_df2010.loc[titans_df2010['Week'] == 7, 'QB'] = "Rusty Smith"

titans_df2011 = yearly_data(df=titans_df2011, year=2011, 
                        HC='Mike Munchak', 
                        OC='Chris Palmer', 
                        DC='Jerry Gray', 
                        QB='Matt Hasselbeck')

titans_df2012 = yearly_data(df=titans_df2012, year=2012, 
                        HC='Mike Munchak', 
                        OC='Chris Palmer', 
                        DC='Jerry Gray',
                        QB='Jake Locker', 
                        QB2='Matt Hasselbeck',QB2_range=range(5,10,1))

titans_df2013 = yearly_data(df=titans_df2013, year=2013, 
                        HC='Mike Munchak', 
                        OC='Dowell Loggains', 
                        DC='Jerry Gray', 
                        QB='Ryan Fitzpatrick', 
                        QB2='Jake Locker',QB2_range=range(4,11,3))
titans_df2013.loc[titans_df2013['Week'] == range(3,10,6), 'QB'] = "Jake Locker"

titans_df2014 = yearly_data(df=titans_df2014, year=2014, 
                        HC='Ken Whisenhunt', 
                        OC='Jason Michael', 
                        DC='Ray Horton', 
                        QB='Zach Mettenberger',
                        QB2='Jake Locker',QB2_range=range(1,4,1))
titans_df2014.loc[titans_df2014['Week'] == 4, 'QB'] = "Charlie Whitehurst"
titans_df2014.loc[titans_df2014['Week'] == range(6,8,1), 'QB'] = "Charlie Whitehurst"
titans_df2014.loc[titans_df2014['Week'] == range(16,18,1), 'QB'] = "Charlie Whitehurst"
titans_df2014.loc[titans_df2014['Week'] == range(5,16,10), 'QB'] = "Jake Locker"

titans_df2015 = yearly_data(df=titans_df2015, year=2015, 
                        HC='Ken Whisenhunt',HC2='Mike Mularkey', HC2_range=range(9,18,1), 
                        OC='Jason Michael', 
                        DC='Ray Horton', 
                        QB='Marcus Mariota', 
                        QB2='Zach Mettenberger',QB2_range=range(16,18,1))
titans_df2015.loc[titans_df2015['Week'] == range(7,9,1), 'QB'] = "Zach Mettenberger"

titans_df2016 = yearly_data(df=titans_df2016, year=2016, 
                        HC='Mike Mularkey',
                        OC='Terry Robiskie', 
                        DC="Dick LeBeau", 
                        QB='Marcus Mariota', 
                        QB2='Matt Cassel',QB2_range=17)

titans_df2017 = yearly_data(df=titans_df2017, year=2017, 
                        HC='Mike Mularkey',
                        OC='Terry Robiskie', 
                        DC="Dick LeBeau", 
                        QB='Marcus Mariota', 
                        QB2='Matt Cassel',QB2_range=5)

titans_df2018 = yearly_data(df=titans_df2018, year=2018, 
                        HC='Mike Vrabel', 
                        OC='Matt LaFleur', 
                        DC='Dean Pees', 
                        QB='Marcus Mariota', 
                        QB2='Blaine Gabbert',QB2_range=range(2,4,1))
titans_df2018.loc[titans_df2018['Week'] == 17, 'QB'] = "Blaine Gabbert"

titans_df2019 = yearly_data(df=titans_df2019, year=2019, 
                        HC='Mike Vrabel', 
                        OC='Arthur Smith', 
                        DC='Dean Pees', 
                        QB='Ryan Tannehill', 
                        QB2='Marcus Mariota',QB2_range=range(1,7,1))

titans_2020 = yearly_data(df=titans_2020, year=2020, 
                        HC='Mike Vrabel', 
                        OC='Arthur Smith', 
                        DC='Mike Vrabel', 
                        QB='Ryan Tannehill')

# Make master dataframe for team
titans_df = titans_2020.append(titans_df2019, ignore_index=True)
titans_df = titans_df.append(titans_df2018, ignore_index=True)
titans_df = titans_df.append(titans_df2017, ignore_index=True)
titans_df = titans_df.append(titans_df2016, ignore_index=True)
titans_df = titans_df.append(titans_df2015, ignore_index=True)
titans_df = titans_df.append(titans_df2014, ignore_index=True)
titans_df = titans_df.append(titans_df2013, ignore_index=True)
titans_df = titans_df.append(titans_df2012, ignore_index=True)
titans_df = titans_df.append(titans_df2011, ignore_index=True)
titans_df = titans_df.append(titans_df2010, ignore_index=True)

titans_df['Base_Team'] = 'Titans'
titans_df = titans_df.sort_values(by=['year', 'Week'], ascending=[False, True])

titans_df

,Week,Day,Results,Wins Tally,Home,Opp,year,Base_Team,PF Tally,PA Tally,...,TO_won Tally,DPassY Tally,DRushY Tally,OTotYd Tally,DTotYd Tally,after_bye,Head_Coach,OC,DC,QB
0,1,Mon,1.0,0.0,0.0,Denver Broncos,2020,Titans,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0,Mike Vrabel,Arthur Smith,Mike Vrabel,Ryan Tannehill
1,2,Sun,1.0,1.0,1.0,Jacksonville Jaguars,2020,Titans,16.0,14.0,...,1.0,216.0,107.0,377.0,323.0,0,Mike Vrabel,Arthur Smith,Mike Vrabel,Ryan Tannehill
2,3,Sun,1.0,2.0,0.0,Minnesota Vikings,2020,Titans,49.0,44.0,...,3.0,531.0,272.0,731.0,803.0,0,Mike Vrabel,Arthur Smith,Mike Vrabel,Ryan Tannehill
3,5,Sun,NaN,NaN,1.0,Buffalo Bills,2020,Titans,80.0,74.0,...,6.0,769.0,498.0,1175.0,1267.0,1,Mike Vrabel,Arthur Smith,Mike Vrabel,Ryan Tannehill
4,6,Sun,NaN,NaN,1.0,Houston Texans,2020,Titans,80.0,74.0,...,6.0,769.0,498.0,1175.0,1267.0,0,Mike Vrabel,Arthur Smith,Mike Vrabel,Ryan Tannehill
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
171,13,Sun,0.0,5.0,1.0,Jacksonville Jaguars,2010,Titans,257.0,218.0,...,21.0,2745.0,1223.0,3279.0,3968.0,0,Jeff Fisher,Mike Heimerdinger,Chuck Cecil,Vince Young
172,14,Thu,0.0,5.0,1.0,Indianapolis Colts,2010,Titans,263.0,235.0,...,21.0,2864.0,1481.0,3499.0,4345.0,0,Jeff Fisher,Mike Heimerdinger,Chuck Cecil,Vince Young
173,15,Sun,1.0,5.0,1.0,Houston Texans,2010,Titans,291.0,265.0,...,21.0,3176.0,1568.0,3864.0,4744.0,0,Jeff Fisher,Mike Heimerdinger,Chuck Cecil,Vince Young
174,16,Sun,0.0,6.0,0.0,Kansas City Chiefs,2010,Titans,322.0,282.0,...,22.0,3469.0,1598.0,4223.0,5067.0,0,Jeff Fisher,Mike Heimerdinger,Chuck Cecil,Vince Young


In [122]:
titans_df.to_csv('Resources/indv_team_csvs/titans.csv')